# Importing packages and files

In [1]:
from importlib import reload
import Scenario
import Technology
import statistics
from pathlib import Path
reload(Scenario)
reload(Technology)
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
import copy
import sys
import plotly.graph_objects as go
from Scenario import Scenario
from Technology import Technology
import requests
from io import StringIO
from scipy.special import logsumexp
from pyomo.environ import *
from pyomo.environ import RangeSet
from pyomo.environ import value

In [2]:
data_path = 'data'

In [3]:
savefigs = True

# Constructing numerical matrices and vectors for LCA

In [5]:
# Helper to read a CSV from a public GitHub raw URL
def read_csv_public(url: str) -> pd.DataFrame:
    """Load a CSV hosted on a public GitHub raw link."""
    return pd.read_csv(url, header=None)

# Data locations
A_url = "https://raw.githubusercontent.com/FarshidNazemi/Supplementary-files-for-Computers-Chem-E/refs/heads/main/data/technology_matrix.csv"
B_url = "https://raw.githubusercontent.com/FarshidNazemi/Supplementary-files-for-Computers-Chem-E/refs/heads/main/data/intervention_matrix.csv"
C_url = "https://raw.githubusercontent.com/FarshidNazemi/Supplementary-files-for-Computers-Chem-E/refs/heads/main/data/impact_factor_matrix.csv"

# Read data
A_df_org    = read_csv_public(A_url)
A_df_design = A_df_org.copy()
A_df        = A_df_org.copy()

B_df_org    = read_csv_public(B_url)
B_df_design = B_df_org.copy()
B_df        = B_df_org.copy()

C_df_org    = read_csv_public(C_url)
C_df_design = C_df_org.copy()
C_df        = C_df_org.copy()

In [6]:
#Building the numerical A matrix from the dataframe
# Step 1: Delete the first 10 columns (non-numerical)
A_df = A_df_design.drop(A_df.columns[:10], axis=1)

# Step 2: Delete the first 10 rows (non-numerical)
A_df = A_df.iloc[10:]

#Replacing empty values with zero and getting the final A matrix
A_design=A_df
A_design=A_design.replace(np.nan, 0)
A_design=np.array(A_design,dtype='float64')

In [7]:
# Building B matrix
# Step 1: Delete the first 7 columns
B_df = B_df.drop(B_df.columns[:7], axis=1)

# Step 2: Delete the first 10 rows (first row is heading, so put 3)
B_df = B_df.iloc[10:]

# Replacing empty values with zero
B_df = B_df.replace(np.nan, 0)

# Converting all elements to float, handling empty strings and stripping whitespace
def convert_to_float(x):
    if isinstance(x, str):
        x = x.strip()  # Remove any leading/trailing whitespace
        if x == '':  # Handle empty strings
            return 0.0
        try:
            return float(x)
        except ValueError:
            return 0.0  # Or raise an error if you want to catch non-numeric strings
    return x

B_df = B_df.applymap(convert_to_float)

# Converting to a NumPy array
B_design = np.array(B_df, dtype=float)

In [8]:
#Building C matrix
# Step 1: Delete the first 7 columns
C_df = C_df.drop(C_df.columns[:7], axis=1)

# Step 2: Delete the first 5 rows (first row is heading, so put 3)
C_df = C_df.iloc[5:]

#Replacing empty values with zero and getting the final B matrix
C_design=C_df
C_design=C_design.replace(np.nan, 0)
C_design=np.array(C_design,dtype='float64')

In [9]:
C_gwp= np.transpose(C_design)[[4]]
#Creating GWP factors matrix
coef_GWP=C_gwp@B_design
coef_GWP=np.array(coef_GWP)
coef_GWP = coef_GWP.reshape(-1)

# Constructing numerical TEA structures for each end of life technology

In [ ]:
landf = Scenario(func_unit = 2.204)
incin = Scenario(
    func_unit = 2.204,
    eol_sc = {'Incineration': [1.0, 0]},
    products = {'Incineration': 'Electricity'}
)
pyrol = Scenario(
    func_unit = 2.204,
    eol_sc = {'Pyrolysis': [1.0, 0]},
    products = {'Pyrolysis': 'Liquid fuel'}
)
pellet_noheat = Scenario(
    func_unit = 2.204,
    eol_sc = {'Pelletizing without Preheating': [1.0, 0]},
    products = {'Pelletizing without Preheating': 'Granulates'}
)
pellet_withheat = Scenario(
    func_unit = 2.204,
    eol_sc = {'Pelletizing with Preheating': [1.0, 0]},
    products = {'Pelletizing with Preheating': 'Granulates'}
)
masc_design = designs.copy().loc[designs.Technology == 'Mechanical and Solvent Cleaning']
strap_design = designs.copy().loc[designs.Technology == 'Solvent Treatment and Precipitation']

# Circulairty Indicator

In [10]:
#Circularity Indicator 
# Price data for calculating gamma
cir_url = "https://raw.githubusercontent.com/FarshidNazemi/Supplementary-files-for-Computers-Chem-E/refs/heads/main/data/Circularity%20(Prices).csv"
# Directly load the public CSV
cir = pd.read_csv(cir_url)
cir = cir.to_numpy()

In [11]:
#Efficiency of the recycling process
eta_1 = 0 #landfill
eta_2 = 0.093 #incineration (HHV of colored film = 42 MJ/kg, Actual electiricity production = 3.92 MJ/kg)
eta_3 = 0.596*0.74 #pyrolysis (HHV of colored film = 42 MJ/kg, Energy content for pyrolysis oil = 25 MJ/kg) and process efficiency: 0.74 kg/kg
eta_4 = 0.95 #STRAP
eta_5_s1 = 0.95 #downcycling-s1
eta_5_s2 = 0.6365 #downcycling-s2
eta_6 = 0.95 #MASC

In [12]:
#Quality parameter for each scenario for the allocation method
gama_1 = 0 #landfill
gama_2 = cir[0,1]/cir[6,1] #incineration
gama_3 = cir[1,1]/cir[6,1] #pyrolysis
gama_4_PE = cir[4,1]/cir[6,1] #STRAP, polyethylene
gama_4_nylon = cir[3,1]/cir[6,1] #STRAP, nylon
gama_5 = cir[5,1]/cir[6,1] #Pelletizing
gama_6 = cir[6,1]/cir[6,1] #MASC

# Economic Indicator

In [13]:
chosen_economic_indicator_show=["Total Production Cost"]

# Functional Unit

In [14]:
FU=input('Enter the functional unit (metric ton of multilayer plastic film consumption):')
FU=float(FU)
f_design=np.zeros(len(A_design))
f_design[0]=FU*1000

Enter the functional unit (metric ton of multilayer plastic film consumption): 1


# 1. Optimization without resilience constraint

## 1.1. Best solution

In [ ]:
#Uncertainty scenario
scen = 0

if scen ==0:
    #Best case values
    masc_solv_draws = 0.01
    masc_ener_draws = 0.0003694444
    masc_eff_draws = 0.95
    masc_cycles = 100

    strap_solv_draws = 0.002
    strap_ener_draws = 0.00001555556
    strap_eff_draws = 0.95
    strap_cycles = 100
    
elif scen ==1:
    # Expected values
    masc_solv_draws = 0.1
    masc_ener_draws = 0.0007388887
    masc_eff_draws = 0.875
    masc_cycles = 5

    strap_solv_draws = 0.1
    strap_ener_draws = 0.000023472225
    strap_eff_draws = 0.875
    strap_cycles = 5

elif scen==2:
    #Worst case values
    masc_solv_draws = 0.2
    masc_ener_draws = 0.001108333
    masc_eff_draws = 0.80
    masc_cycles = 1

    strap_solv_draws = 0.2
    strap_ener_draws = 0.000023472225
    strap_eff_draws = 0.80
    strap_cycles = 1
    
#TEA
masc_des = masc_design.loc[(masc_design.Index.isin(['Electricity','Diacetone alcohol']))  | (masc_design.Variable == 'Output efficiency')]
strap_des = strap_design.loc[(strap_design.Index.isin(['Electricity','Ethylene glycol']))  | (strap_design.Variable == 'Output efficiency')]


A_design_scenario = []
masc_unc_costs = []
strap_unc_costs = []
masc_unc_waste = []
strap_unc_waste = []



A_design_scenario = A_design.copy()

## MASC ##

A_design_scenario [A_df_design.index[A_df_design.iloc[:, 1] == 'Diacetone Alcohol'].tolist()[0] - 10,
                 np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'MASC'].tolist()[0] - 10] = - 5.56 * (masc_solv_draws)*(1/masc_eff_draws)*calculate_sum_mi_masc(masc_eff_draws, masc_cycles)

A_design_scenario [A_df_design.index[A_df_design.iloc[:, 0] == 'Electricity, at Grid, US, 2010'].tolist()[0] - 10,
                 np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'MASC'].tolist()[0] - 10] = - 5.56 * 2.7 * (1/masc_eff_draws)*calculate_sum_mi_masc(masc_eff_draws, masc_cycles)

A_design_scenario [A_df_design.index[A_df_design.iloc[:, 1] == 'Ultimate EoL waste'].tolist()[0] - 10,
                 np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'MASC'].tolist()[0] - 10] = ((1-masc_eff_draws)/masc_eff_draws)*calculate_sum_mi_masc(masc_eff_draws, masc_cycles) + calculate_mi_masc(masc_eff_draws,masc_cycles,masc_cycles)

A_design_scenario [A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film'].tolist()[0] - 10,
                 np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'MASC'].tolist()[0] - 10] = 1 - (calculate_mp_masc(masc_eff_draws, masc_cycles))
## STRAP ##

A_design_scenario [A_df_design.index[A_df_design.iloc[:, 0] == 'Electricity, at Grid, US, 2010'].tolist()[0] - 10,
                 np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'STRAP'].tolist()[0] - 10] = - (1/0.9) * 0.648 * (1/strap_eff_draws) * calculate_sum_mi_strap(strap_eff_draws, strap_cycles)

A_design_scenario [A_df_design.index[A_df_design.iloc[:, 1] == 'Ethylene glycol, at plant, kg'].tolist()[0] - 10,
                 np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'STRAP'].tolist()[0] - 10] = - 11.5 * (1/0.9) * (strap_solv_draws)*(1/strap_eff_draws)*calculate_sum_mi_strap(strap_eff_draws, strap_cycles)

A_design_scenario [A_df_design.index[A_df_design.iloc[:, 1] == 'heat, from natural gas'].tolist()[0] - 10,
                 np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'STRAP'].tolist()[0] - 10] = - (1/0.9) * 0.00351 * (1/strap_eff_draws) * calculate_sum_mi_strap(strap_eff_draws, strap_cycles) 

A_design_scenario [A_df_design.index[A_df_design.iloc[:, 1] == 'Ultimate EoL waste'].tolist()[0] - 10,
                 np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'STRAP'].tolist()[0] - 10] = ((1-strap_eff_draws)/strap_eff_draws)*(1/0.9)*calculate_sum_mi_strap(strap_eff_draws, strap_cycles) + calculate_mi_strap(strap_eff_draws,strap_cycles,strap_cycles)

A_design_scenario [A_df_design.index[A_df_design.iloc[:, 1] == 'POLYAMIDE 6 (Nylon 6) (at plant)'].tolist()[0] - 10,
                 np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'STRAP'].tolist()[0] - 10] = 0.26/0.9 - (0.26/0.9)*calculate_mp_strap(strap_eff_draws, strap_cycles)

A_design_scenario [A_df_design.index[A_df_design.iloc[:, 1] == 'Polyethylene, linear low-density, LLDPE, virgin resin, at plant'].tolist()[0] - 10,
                 np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'STRAP'].tolist()[0] - 10] = 0.74/0.9 - (0.74/0.9)*calculate_mp_strap(strap_eff_draws, strap_cycles)



masc_des.loc[masc_des.Index == 'Electricity','Value'] = 2521.974 * masc_ener_draws
masc_des.loc[masc_des.Index == 'Diacetone alcohol','Value'] = 5560 * masc_solv_draws
masc_des.loc[masc_des.Variable == 'Output efficiency','Value'] = masc_eff_draws

_masc = Scenario(
    func_unit = 2.204,
    sens_df = masc_des,
    eol_sc = {'Mechanical and Solvent Cleaning': [1.0, masc_cycles]},
    products = {'Mechanical and Solvent Cleaning': 'Barrier film'}
)


strap_des.loc[strap_des.Index == 'Electricity', 'Value'] = 11400 * strap_ener_draws
strap_des.loc[strap_des.Index == 'Ethylene glycol', 'Value'] = 25132.698 * strap_solv_draws
strap_des.loc[strap_des.Variable == 'Output efficiency', 'Value'] = strap_eff_draws

_strap = Scenario(
    func_unit = 2.204,
    sens_df = strap_des,
    eol_sc = {'Solvent Treatment and Precipitation': [1.0, strap_cycles]},
    products = {'Solvent Treatment and Precipitation':'Polyethylene'}
)

masc_waste = ((1-masc_eff_draws)/masc_eff_draws)*calculate_sum_mi_masc(masc_eff_draws, masc_cycles) + calculate_mi_masc(masc_eff_draws,masc_cycles,masc_cycles)
strap_waste = ((1-strap_eff_draws)/strap_eff_draws)*(1/0.9)*calculate_sum_mi_strap(strap_eff_draws, strap_cycles) + calculate_mi_strap(strap_eff_draws,strap_cycles,strap_cycles)

### Multi-objective optimization formulation

In [ ]:
#Model Formulation
# Create the model
model = ConcreteModel()

res = 0

model.s_eps1 = Var(within=NonNegativeReals, bounds=(1e-6, None))  # Slack for epsilon constraint 1
model.s_eps2 = Var(within=NonNegativeReals, bounds=(1e-6, None))  # Slack for epsilon constraint 2


#epsilon parameter
model.epsilon1=Param(initialize=1000,mutable=True)
model.epsilon2=Param(initialize=1000,mutable=True)

# Define the decision variable
model.set_s = RangeSet(len(np.transpose(A_design)))
model.s = Var(model.set_s)
for i in model.set_s:
    model.s[i].set_value(1.0)  # Adjust based on the problem's context
# List of processes with negative scaling factor due to the substitution approach
negative_s_indices = []
positive_s_indices = []
all_s_indices = []
search_elements = [
    'Substitued Electricity US Market',
    'Substitued Polypropylene Market',
    'Substitued Refinery Oil Market'
]

search_elements_zero = [
    #'pyrolysis',
    #'STRAP',
    #'MASC',
    #'landfill (film)',
    #'downcycling (allocation)-s1 (heat pretreatment)',
    #'MASC',
    #'incineration (film)'
]

# Search for elements in the first row of the DataFrame
negative_s_indices = [A_df_design.iloc[0,:].tolist().index(elem) for elem in search_elements]
negative_s_indices = [i-9 for i in negative_s_indices]
all_s_indices = list(range(1, len(np.transpose(A_design))+1))
positive_s_indices = [index for index in all_s_indices if index not in negative_s_indices]
zero_s_indices = [A_df_design.iloc[0,:].tolist().index(elem) for elem in search_elements_zero]
zero_s_indices = [i-9 for i in zero_s_indices]

model.set_negative_scale = Set(initialize=negative_s_indices)
model.set_positive_scale = Set(initialize=positive_s_indices)
model.set_zero_scale = Set(initialize=zero_s_indices)

#model constraint: As=f
model.set_balance = RangeSet(len(f_design))
def cons_rule(model, p):
    return sum(A_design_scenario[p-1,i-1]*model.s[i] for i in model.set_s) == f_design[p-1]
def negative_scale(model, i):
    return (model.s[i]<=0)
def positive_scale(model, i):
    return (model.s[i]>=0)
def zero_scale(model, i):
    return (model.s[i]==0)

##### Resilience Metric
# Define indices
num_rows = 10  # Number of rows/columns in R
indices = range(num_rows)
# Define epsilon for numerical stability
epsilon = 1e-10

def R_rule(model, i, j):
    if i == 0 and j == 1:
        return abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film'].tolist()[0] - 10,
                            A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'multilayer film production'].tolist()[0] - 10]) \
               * abs(model.s[A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'multilayer film production'].tolist()[0] - 9])
    elif i == 0 and j == 9:
        return abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'Ultimate EoL waste'].tolist()[0] - 10,
                            A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'multilayer film production'].tolist()[0] - 10]) \
               * abs(model.s[A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'multilayer film production'].tolist()[0] - 9])
    elif i == 1 and j == 2:
        return abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                            A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'landfill (film)'].tolist()[0] - 10]) \
               * abs(model.s[A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'landfill (film)'].tolist()[0] - 9])
    elif i == 1 and j == 3:
        return abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                            A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'incineration (film)'].tolist()[0] - 10]) \
               * abs(model.s[A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'incineration (film)'].tolist()[0] - 9])
    elif i == 1 and j == 4:
        return abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                            A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'downcycling (allocation)-s1 (heat pretreatment)'].tolist()[0] - 10]) \
               * abs(model.s[A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'downcycling (allocation)-s1 (heat pretreatment)'].tolist()[0] - 9])
    elif i == 1 and j == 5:
        return abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                            A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'downcycling (allocation)-s2 (1/3 discard)'].tolist()[0] - 10]) \
               * abs(model.s[A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'downcycling (allocation)-s2 (1/3 discard)'].tolist()[0] - 9])
    elif i == 1 and j == 6:
        return abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                            A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'STRAP'].tolist()[0] - 10]) \
               * abs(model.s[A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'STRAP'].tolist()[0] - 9])
    elif i == 1 and j == 7:
        return abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                            A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'MASC'].tolist()[0] - 10]) \
               * abs(model.s[A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'MASC'].tolist()[0] - 9])
    elif i == 1 and j == 8:
        return abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                            A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'pyrolysis'].tolist()[0] - 10]) \
               * abs(model.s[A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'pyrolysis'].tolist()[0] - 9])
    elif i == 2 and j == 9:
        return abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                   A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'landfill (film)'].tolist()[0] - 10]) \
           * abs(model.s[A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'landfill (film)'].tolist()[0] - 9])
    elif i == 3 and j == 9:
        return abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                   A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'incineration (film)'].tolist()[0] - 10]) \
           * abs(model.s[A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'incineration (film)'].tolist()[0] - 9])
    elif i == 4 and j == 9:
        return abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                   A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'downcycling (allocation)-s1 (heat pretreatment)'].tolist()[0] - 10]) \
           * abs(model.s[A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'downcycling (allocation)-s1 (heat pretreatment)'].tolist()[0] - 9])
    elif i == 5 and j == 9:
        return abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                   A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'downcycling (allocation)-s2 (1/3 discard)'].tolist()[0] - 10]) \
           * abs(model.s[A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'downcycling (allocation)-s2 (1/3 discard)'].tolist()[0] - 9])
    elif i == 6 and j == 0:
        return abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'Polyethylene, linear low-density, LLDPE, virgin resin, at plant'].tolist()[0] - 10,
                   A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'STRAP'].tolist()[0] - 10]) \
           * abs(model.s[A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'STRAP'].tolist()[0] - 9]) \
     +abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'POLYAMIDE 6 (Nylon 6) (at plant)'].tolist()[0] - 10,
                   A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'STRAP'].tolist()[0] - 10]) \
           * abs(model.s[A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'STRAP'].tolist()[0] - 9])
    elif i == 6 and j == 9:
        return abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'Ultimate EoL waste'].tolist()[0] - 10,
                   A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'STRAP'].tolist()[0] - 10]) \
           * abs(model.s[A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'STRAP'].tolist()[0] - 9])
    elif i == 7 and j == 1:
        return abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film'].tolist()[0] - 10,
                   A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'MASC'].tolist()[0] - 10]) \
           * abs(model.s[A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'MASC'].tolist()[0] - 9])
    elif i == 7 and j == 9:
        return abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'Ultimate EoL waste'].tolist()[0] - 10,
                   A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'MASC'].tolist()[0] - 10]) \
           * abs(model.s[A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'MASC'].tolist()[0] - 9])
    elif i == 8 and j == 9:
        return abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                   A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'pyrolysis'].tolist()[0] - 10]) \
           * abs(model.s[A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'pyrolysis'].tolist()[0] - 9])
    elif i == 9 and j == 0:
        return abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'Polyethylene, linear low-density, LLDPE, virgin resin, at plant'].tolist()[0] - 10,
                            A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'Polyethylene, linear low-density, LLDPE, virgin resin, at plant'].tolist()[0] - 10]) \
               * abs(model.s[A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'Polyethylene, linear low-density, LLDPE, virgin resin, at plant'].tolist()[0] - 9]) \
               + abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'POLYAMIDE 6 (Nylon 6) (at plant)'].tolist()[0] - 10,
                          A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'POLYAMIDE 6 (Nylon 6)'].tolist()[0] - 10]) \
               * abs(model.s[A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'POLYAMIDE 6 (Nylon 6)'].tolist()[0] - 9])
    else:
        return 0

model.R = Expression(indices, indices, rule=R_rule)

# Define the total sum of R
model.R_sum = Expression(expr=sum(model.R[i, j] for i in indices for j in indices))

# Define Development Capacity (DC)
def DC_rule(model):
    return sum(
        -model.R[i, j] * log((model.R[i, j] / model.R_sum) + epsilon) 
        for i in indices for j in indices if i != j
    )
model.DC = Expression(rule=DC_rule)

# Define Ascendency (ASC)
def ASC_rule(model):
    return sum(
        model.R[i, j] * log(
            (model.R[i, j] * model.R_sum) / 
            ((sum(model.R[i, k] for k in indices) + epsilon) * 
             (sum(model.R[k, j] for k in indices) + epsilon)) + epsilon
        ) 
        for i in indices for j in indices if i != j
    )
model.ASC = Expression(rule=ASC_rule)

# Define R_ASC
model.R_ASC = Expression(expr=model.ASC / (model.DC+ epsilon))


model.balance_constraints = Constraint(model.set_balance, rule=cons_rule)
model.negative_scale_constraints = Constraint(model.set_negative_scale, rule=negative_scale)
model.positive_scale_constraints = Constraint(model.set_positive_scale, rule=positive_scale)
model.zero_scale_constraints = Constraint(model.set_zero_scale, rule=zero_scale)
if res==1:  
    model.R_ASC_lower_bound = Constraint(expr=0.213 <= model.R_ASC)
    model.R_ASC_upper_bound = Constraint(expr=model.R_ASC <= 0.589)


model.eps_constraint_1 = Constraint(expr = -(abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'Polyethylene, linear low-density, LLDPE, virgin resin, at plant'].tolist()[0] - 10,
                     np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'Polyethylene, linear low-density, LLDPE, virgin resin, at plant'].tolist()[0] - 10])
    *model.s[np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'Polyethylene, linear low-density, LLDPE, virgin resin, at plant'].tolist()[0] - 9]
        
        + abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'POLYAMIDE 6 (Nylon 6) (at plant)'].tolist()[0] - 10,
                     np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'POLYAMIDE 6 (Nylon 6)'].tolist()[0] - 10])
    *model.s[np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'POLYAMIDE 6 (Nylon 6)'].tolist()[0] - 9]
        
        + (abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                     np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'landfill (film)'].tolist()[0] - 10])
    *model.s[np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'landfill (film)'].tolist()[0] - 9])*(1-0)

                                  + (abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                     np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'incineration (film)'].tolist()[0] - 10])
    *model.s[np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'incineration (film)'].tolist()[0] - 9])*(1-eta_2*gama_2)

                                       + (abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                     np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'pyrolysis'].tolist()[0] - 10])
    *model.s[np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'pyrolysis'].tolist()[0] - 9])*(1-eta_3*gama_3) 

                                  + (abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                     np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'STRAP'].tolist()[0] - 10])
    *model.s[np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'STRAP'].tolist()[0] - 9])*strap_waste

                                  + (abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                     np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'downcycling (allocation)-s1 (heat pretreatment)'].tolist()[0] - 10])
    *model.s[np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'downcycling (allocation)-s1 (heat pretreatment)'].tolist()[0] - 9])*(1-eta_5_s1*gama_5) 

                                     +(abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                     np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'downcycling (allocation)-s2 (1/3 discard)'].tolist()[0] - 10])
    *model.s[np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'downcycling (allocation)-s2 (1/3 discard)'].tolist()[0] - 9])*(1-eta_5_s2*gama_5) 

                                  + (abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                     np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'MASC'].tolist()[0] - 10])
    *model.s[np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'MASC'].tolist()[0] - 9])*masc_waste) - model.s_eps1 == model.epsilon1)


model.eps_constraint_2 = Constraint(expr = -((abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                         np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'landfill (film)'].tolist()[0] - 10])
        *model.s[np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'landfill (film)'].tolist()[0] - 9])
                                      *landf.total_cost

                                      + (abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                         np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'incineration (film)'].tolist()[0] - 10])
        *model.s[np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'incineration (film)'].tolist()[0] - 9])
                                      *landf.total_cost

                                           + (abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                         np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'pyrolysis'].tolist()[0] - 10])
        *model.s[np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'pyrolysis'].tolist()[0] - 9])
                                      *pyrol.total_cost

                                      + (abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                         np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'STRAP'].tolist()[0] - 10])
        *model.s[np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'STRAP'].tolist()[0] - 9])
                                      *_strap.total_cost

                                      + (abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                         np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'downcycling (allocation)-s1 (heat pretreatment)'].tolist()[0] - 10])
        *model.s[np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'downcycling (allocation)-s1 (heat pretreatment)'].tolist()[0] - 9])
                                      *pellet_withheat.total_cost

                                         +(abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                         np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'downcycling (allocation)-s2 (1/3 discard)'].tolist()[0] - 10])
        *model.s[np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'downcycling (allocation)-s2 (1/3 discard)'].tolist()[0] - 9])
                                      *pellet_noheat.total_cost

                                      + (abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                         np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'MASC'].tolist()[0] - 10])
        *model.s[np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'MASC'].tolist()[0] - 9])
                                      *_masc.total_cost) - model.s_eps2 == model.epsilon2)



model.obj1 = Objective(expr = -sum(coef_GWP[i-1]*model.s[i] for i in model.set_s), sense=maximize)

model.obj2 = Objective(expr = -(abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'Polyethylene, linear low-density, LLDPE, virgin resin, at plant'].tolist()[0] - 10,
                     np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'Polyethylene, linear low-density, LLDPE, virgin resin, at plant'].tolist()[0] - 10])
    *model.s[np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'Polyethylene, linear low-density, LLDPE, virgin resin, at plant'].tolist()[0] - 9]
        
        + abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'POLYAMIDE 6 (Nylon 6) (at plant)'].tolist()[0] - 10,
                     np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'POLYAMIDE 6 (Nylon 6)'].tolist()[0] - 10])
    *model.s[np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'POLYAMIDE 6 (Nylon 6)'].tolist()[0] - 9]
        
        + (abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                     np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'landfill (film)'].tolist()[0] - 10])
    *model.s[np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'landfill (film)'].tolist()[0] - 9])*(1-0)

                                  + (abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                     np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'incineration (film)'].tolist()[0] - 10])
    *model.s[np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'incineration (film)'].tolist()[0] - 9])*(1-eta_2*gama_2)

                                       + (abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                     np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'pyrolysis'].tolist()[0] - 10])
    *model.s[np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'pyrolysis'].tolist()[0] - 9])*(1-eta_3*gama_3) 

                                  + (abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                     np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'STRAP'].tolist()[0] - 10])
    *model.s[np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'STRAP'].tolist()[0] - 9])*strap_waste

                                  + (abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                     np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'downcycling (allocation)-s1 (heat pretreatment)'].tolist()[0] - 10])
    *model.s[np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'downcycling (allocation)-s1 (heat pretreatment)'].tolist()[0] - 9])*(1-eta_5_s1*gama_5) 

                                     +(abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                     np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'downcycling (allocation)-s2 (1/3 discard)'].tolist()[0] - 10])
    *model.s[np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'downcycling (allocation)-s2 (1/3 discard)'].tolist()[0] - 9])*(1-eta_5_s2*gama_5) 

                                  + (abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                     np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'MASC'].tolist()[0] - 10])
    *model.s[np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'MASC'].tolist()[0] - 9])*masc_waste), sense=maximize
            )

model.obj3 = Objective(expr = -((abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                         np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'landfill (film)'].tolist()[0] - 10])
        *model.s[np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'landfill (film)'].tolist()[0] - 9])
                                      *landf.total_cost

                                      + (abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                         np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'incineration (film)'].tolist()[0] - 10])
        *model.s[np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'incineration (film)'].tolist()[0] - 9])
                                      *landf.total_cost

                                           + (abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                         np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'pyrolysis'].tolist()[0] - 10])
        *model.s[np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'pyrolysis'].tolist()[0] - 9])
                                      *pyrol.total_cost

                                      + (abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                         np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'STRAP'].tolist()[0] - 10])
        *model.s[np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'STRAP'].tolist()[0] - 9])
                                      *_strap.total_cost

                                      + (abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                         np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'downcycling (allocation)-s1 (heat pretreatment)'].tolist()[0] - 10])
        *model.s[np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'downcycling (allocation)-s1 (heat pretreatment)'].tolist()[0] - 9])
                                      *pellet_withheat.total_cost

                                         +(abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                         np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'downcycling (allocation)-s2 (1/3 discard)'].tolist()[0] - 10])
        *model.s[np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'downcycling (allocation)-s2 (1/3 discard)'].tolist()[0] - 9])
                                      *pellet_noheat.total_cost

                                      + (abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                         np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'MASC'].tolist()[0] - 10])
        *model.s[np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'MASC'].tolist()[0] - 9])
                                      *_masc.total_cost), sense=maximize
         )

#Solver
if res==0:
    solver = SolverFactory('gurobi')
elif res==1:
    solver = SolverFactory('ipopt')

### Epsilon bonds

In [ ]:
model.obj1.activate() 
model.obj2.deactivate()
model.obj3.deactivate()
model.eps_constraint_1.deactivate()
model.eps_constraint_2.deactivate()
results = solver.solve(model, tee=True) # solves and updates instance
Z1_1_best=value(model.obj1)
Z2_1_best=value(model.obj2)
Z3_1_best=value(model.obj3)

In [ ]:
model.obj1.deactivate() 
model.obj2.activate() 
model.obj3.deactivate()
model.eps_constraint_1.deactivate()
model.eps_constraint_2.deactivate()
results = solver.solve(model, tee=True) # solves and updates instance
Z1_2_best= value(model.obj1)
Z2_2_best= value(model.obj2)
Z3_2_best= value(model.obj3)
scaling_factors_circ = []
for j in model.set_s:
    scaling_factors_circ.append(model.s[j].value)

In [ ]:
model.obj1.deactivate() 
model.obj2.deactivate()
model.obj3.activate()
model.eps_constraint_1.deactivate()
model.eps_constraint_2.deactivate()
results = solver.solve(model, tee=True) # solves and updates instance
Z1_3_best= value(model.obj1)
Z2_3_best= value(model.obj2)
Z3_3_best= value(model.obj3)

### Solving the Optimization Problem

In [ ]:
M1_best = max(Z1_1_best,Z1_2_best,Z1_3_best)
n1_best = min(Z1_1_best,Z1_2_best,Z1_3_best)

M2_best = max(Z2_1_best,Z2_2_best,Z2_3_best)
n2_best = min(Z2_1_best,Z2_2_best,Z2_3_best)

M3_best = max(Z3_1_best,Z3_2_best,Z3_3_best)
n3_best = min(Z3_1_best,Z3_2_best,Z3_3_best)

In [ ]:
M_penalty = 1e-4
delta_1 =  M2_best - n2_best
delta_2 = M3_best - n3_best
model.obj1_aug = Objective(expr = -sum(coef_GWP[i-1]*model.s[i] for i in model.set_s) + M_penalty*(model.s_eps1/delta_1 + model.s_eps2/delta_2), sense=maximize)

In [ ]:
#Z1, Z2, and Z3 values!!! for pareto surface generation

#Best
model.obj1.deactivate()
model.obj1_aug.activate()
model.obj2.deactivate() 
model.obj3.deactivate()
model.eps_constraint_1.activate()
model.eps_constraint_2.activate()


r=100
Z1=[]
Z2=[]
Z3=[]
solution_matrix = []

for t in range(0,r):
    for i in range (0,r):
        model.epsilon1 = n2_best + (M2_best - n2_best) * t / (r - 1)
        model.epsilon2 = n3_best + (M3_best - n3_best) * i / (r - 1)
        results = solver.solve(model) # solves and updates instance
        
        if results.solver.termination_condition == TerminationCondition.optimal:           
            Z1.append(-value(model.obj1))
            Z2.append(-value(model.obj2))
            Z3.append(-value(model.obj3))
            
            
            scaling_factors = []
            for j in model.set_s:
                scaling_factors.append(model.s[j].value)
            solution_matrix.append(scaling_factors) 

Z1.append(-Z1_2_best)
Z2.append(-Z2_2_best)
Z3.append(-Z3_2_best)

In [ ]:
solution_matrix.append(scaling_factors_circ)

### Generating the pareto frontier numbers

In [ ]:
# Assuming Z1, Z2, and Z3 are lists of values

# Create a dictionary to store unique combinations
unique_combinations = {}

# Iterate over the lists and store unique combinations in the dictionary
for z1, z2, z3 in zip(Z1, Z2, Z3):
    combination = (z1, z2, z3)
    if combination not in unique_combinations:
        unique_combinations[combination] = 1  # Value doesn't matter, just using the key for uniqueness

# Extract unique values from the dictionary
unique_Z1, unique_Z2, unique_Z3 = zip(*unique_combinations.keys())

In [ ]:
# Assuming Z1, Z2, and Z3 are lists of values
# Create a dictionary to store unique combinations and their indices
unique_combinations = {}
indices = []
unique_solution_matrix = []

# Iterate over the lists and store unique combinations and their indices in the dictionary
for i, (z1, z2, z3) in enumerate(zip(Z1, Z2, Z3)):
    combination = (z1, z2, z3)
    if combination not in unique_combinations:
        unique_combinations[combination] = [i]  # Store the index as a list
    else:
        unique_combinations[combination].append(i)  # Append the index to the existing list

# Extract unique values and their corresponding indices from the dictionary
indices = list(unique_combinations.values())

# New list with only the first elements of each sublist
unique_solution_indices = [sublist[0] for sublist in indices]

unique_solution_matrix = [solution_matrix[index] for index in unique_solution_indices]

In [ ]:
pareto_GWP = unique_Z1
pareto_circularity = [(1 - (value / (2 * f_design[0]))) for value in unique_Z2]
pareto_cost = unique_Z3

#pareto front points
pareto_points = [[pareto_GWP[i], pareto_circularity[i], pareto_cost[i]] for i in range(len(pareto_GWP))]

In [ ]:
#utopia point

utopia_point_GWP = min(pareto_GWP)
scaled_utopia_point_GWP = (utopia_point_GWP - min(pareto_GWP)) / (max(pareto_GWP) - min(pareto_GWP))

utopia_point_circularity = max(pareto_circularity)
scaled_utopia_point_circularity = (utopia_point_circularity - min(pareto_circularity)) / (max(pareto_circularity) - min(pareto_circularity))

utopia_point_cost = min(pareto_cost)
scaled_utopia_point_cost = (utopia_point_cost - min(pareto_cost)) / (max(pareto_cost) - min(pareto_cost))                                                     

In [ ]:
#Compromised Solution

scaled_pareto_GWP = [(element - min(pareto_GWP)) / (max(pareto_GWP) - min(pareto_GWP)) for element in pareto_GWP]
                                                                     

scaled_pareto_circularity = [(element - min(pareto_circularity)) / (max(pareto_circularity) - min(pareto_circularity)) for element in pareto_circularity]


scaled_pareto_cost = [(element - min(pareto_cost)) / (max(pareto_cost) - min(pareto_cost)) for element in pareto_cost]


# Calculate the distance to the utopia point
distances = []
for i in range(len(scaled_pareto_GWP)):
    distances.append(
        np.sqrt((scaled_pareto_GWP[i] - scaled_utopia_point_GWP) ** 2  
        + (scaled_pareto_circularity[i] - scaled_utopia_point_circularity) ** 2)
        + (scaled_pareto_cost[i] - scaled_utopia_point_cost) ** 2)
index_min_distance = np.argmin(distances)

In [ ]:
#Solutions' scaling factors
index_min_GWP_solution = np.argmin(pareto_GWP)
index_max_circularity_solution = np.argmax(pareto_circularity)
index_min_cost_solution = np.argmin(pareto_cost)
index_compromised_solution = index_min_distance

min_GWP_solution = unique_solution_matrix[index_min_GWP_solution]
max_circularity_solution = unique_solution_matrix[index_max_circularity_solution]
min_cost_solution = unique_solution_matrix[index_min_cost_solution]
compromised_solution = unique_solution_matrix[index_compromised_solution]

## 1.2. Median solution

In [ ]:
#Uncertainty scenario
scen = 1

if scen ==0:
    #Best case values
    masc_solv_draws = 0.01
    masc_ener_draws = 0.0003694444
    masc_eff_draws = 0.95
    masc_cycles = 100

    strap_solv_draws = 0.002
    strap_ener_draws = 0.00001555556
    strap_eff_draws = 0.95
    strap_cycles = 100
    
elif scen ==1:
    # Expected values
    masc_solv_draws = 0.1
    masc_ener_draws = 0.0007388887
    masc_eff_draws = 0.875
    masc_cycles = 5

    strap_solv_draws = 0.1
    strap_ener_draws = 0.000023472225
    strap_eff_draws = 0.875
    strap_cycles = 5

elif scen==2:
    #Worst case values
    masc_solv_draws = 0.2
    masc_ener_draws = 0.001108333
    masc_eff_draws = 0.80
    masc_cycles = 1

    strap_solv_draws = 0.2
    strap_ener_draws = 0.000023472225
    strap_eff_draws = 0.80
    strap_cycles = 1
    
#TEA
masc_des = masc_design.loc[(masc_design.Index.isin(['Electricity','Diacetone alcohol']))  | (masc_design.Variable == 'Output efficiency')]
strap_des = strap_design.loc[(strap_design.Index.isin(['Electricity','Ethylene glycol']))  | (strap_design.Variable == 'Output efficiency')]


A_design_scenario = []
masc_unc_costs = []
strap_unc_costs = []
masc_unc_waste = []
strap_unc_waste = []



A_design_scenario = A_design.copy()

## MASC ##

A_design_scenario [A_df_design.index[A_df_design.iloc[:, 1] == 'Diacetone Alcohol'].tolist()[0] - 10,
                 np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'MASC'].tolist()[0] - 10] = - 5.56 * (masc_solv_draws)*(1/masc_eff_draws)*calculate_sum_mi_masc(masc_eff_draws, masc_cycles)

A_design_scenario [A_df_design.index[A_df_design.iloc[:, 0] == 'Electricity, at Grid, US, 2010'].tolist()[0] - 10,
                 np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'MASC'].tolist()[0] - 10] = - 5.56 * 2.7 * (1/masc_eff_draws)*calculate_sum_mi_masc(masc_eff_draws, masc_cycles)

A_design_scenario [A_df_design.index[A_df_design.iloc[:, 1] == 'Ultimate EoL waste'].tolist()[0] - 10,
                 np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'MASC'].tolist()[0] - 10] = ((1-masc_eff_draws)/masc_eff_draws)*calculate_sum_mi_masc(masc_eff_draws, masc_cycles) + calculate_mi_masc(masc_eff_draws,masc_cycles,masc_cycles)

A_design_scenario [A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film'].tolist()[0] - 10,
                 np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'MASC'].tolist()[0] - 10] = 1 - (calculate_mp_masc(masc_eff_draws, masc_cycles))
## STRAP ##

A_design_scenario [A_df_design.index[A_df_design.iloc[:, 0] == 'Electricity, at Grid, US, 2010'].tolist()[0] - 10,
                 np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'STRAP'].tolist()[0] - 10] = - (1/0.9) * 0.648 * (1/strap_eff_draws) * calculate_sum_mi_strap(strap_eff_draws, strap_cycles)

A_design_scenario [A_df_design.index[A_df_design.iloc[:, 1] == 'Ethylene glycol, at plant, kg'].tolist()[0] - 10,
                 np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'STRAP'].tolist()[0] - 10] = - 11.5 * (1/0.9) * (strap_solv_draws)*(1/strap_eff_draws)*calculate_sum_mi_strap(strap_eff_draws, strap_cycles)

A_design_scenario [A_df_design.index[A_df_design.iloc[:, 1] == 'heat, from natural gas'].tolist()[0] - 10,
                 np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'STRAP'].tolist()[0] - 10] = - (1/0.9) * 0.00351 * (1/strap_eff_draws) * calculate_sum_mi_strap(strap_eff_draws, strap_cycles) 

A_design_scenario [A_df_design.index[A_df_design.iloc[:, 1] == 'Ultimate EoL waste'].tolist()[0] - 10,
                 np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'STRAP'].tolist()[0] - 10] = ((1-strap_eff_draws)/strap_eff_draws)*(1/0.9)*calculate_sum_mi_strap(strap_eff_draws, strap_cycles) + calculate_mi_strap(strap_eff_draws,strap_cycles,strap_cycles)

A_design_scenario [A_df_design.index[A_df_design.iloc[:, 1] == 'POLYAMIDE 6 (Nylon 6) (at plant)'].tolist()[0] - 10,
                 np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'STRAP'].tolist()[0] - 10] = 0.26/0.9 - (0.26/0.9)*calculate_mp_strap(strap_eff_draws, strap_cycles)

A_design_scenario [A_df_design.index[A_df_design.iloc[:, 1] == 'Polyethylene, linear low-density, LLDPE, virgin resin, at plant'].tolist()[0] - 10,
                 np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'STRAP'].tolist()[0] - 10] = 0.74/0.9 - (0.74/0.9)*calculate_mp_strap(strap_eff_draws, strap_cycles)



masc_des.loc[masc_des.Index == 'Electricity','Value'] = 2521.974 * masc_ener_draws
masc_des.loc[masc_des.Index == 'Diacetone alcohol','Value'] = 5560 * masc_solv_draws
masc_des.loc[masc_des.Variable == 'Output efficiency','Value'] = masc_eff_draws

_masc = Scenario(
    func_unit = 2.204,
    sens_df = masc_des,
    eol_sc = {'Mechanical and Solvent Cleaning': [1.0, masc_cycles]},
    products = {'Mechanical and Solvent Cleaning': 'Barrier film'}
)


strap_des.loc[strap_des.Index == 'Electricity', 'Value'] = 11400 * strap_ener_draws
strap_des.loc[strap_des.Index == 'Ethylene glycol', 'Value'] = 25132.698 * strap_solv_draws
strap_des.loc[strap_des.Variable == 'Output efficiency', 'Value'] = strap_eff_draws

_strap = Scenario(
    func_unit = 2.204,
    sens_df = strap_des,
    eol_sc = {'Solvent Treatment and Precipitation': [1.0, strap_cycles]},
    products = {'Solvent Treatment and Precipitation':'Polyethylene'}
)

masc_waste = ((1-masc_eff_draws)/masc_eff_draws)*calculate_sum_mi_masc(masc_eff_draws, masc_cycles) + calculate_mi_masc(masc_eff_draws,masc_cycles,masc_cycles)
strap_waste = ((1-strap_eff_draws)/strap_eff_draws)*(1/0.9)*calculate_sum_mi_strap(strap_eff_draws, strap_cycles) + calculate_mi_strap(strap_eff_draws,strap_cycles,strap_cycles)

### Multi-objective optimization formulation

In [ ]:
#Model Formulation
# Create the model
model = ConcreteModel()

res = 0

model.s_eps1 = Var(within=NonNegativeReals, bounds=(1e-6, None))  # Slack for epsilon constraint 1
model.s_eps2 = Var(within=NonNegativeReals, bounds=(1e-6, None))  # Slack for epsilon constraint 2


#epsilon parameter
model.epsilon1=Param(initialize=1000,mutable=True)
model.epsilon2=Param(initialize=1000,mutable=True)

# Define the decision variable
model.set_s = RangeSet(len(np.transpose(A_design)))
model.s = Var(model.set_s)
for i in model.set_s:
    model.s[i].set_value(1.0)  # Adjust based on the problem's context
# List of processes with negative scaling factor due to the substitution approach
negative_s_indices = []
positive_s_indices = []
all_s_indices = []
search_elements = [
    'Substitued Electricity US Market',
    'Substitued Polypropylene Market',
    'Substitued Refinery Oil Market'
]

search_elements_zero = [
    #'pyrolysis',
    #'STRAP',
    #'MASC',
    #'landfill (film)',
    #'downcycling (allocation)-s1 (heat pretreatment)',
    #'MASC',
    #'incineration (film)'
]

# Search for elements in the first row of the DataFrame
negative_s_indices = [A_df_design.iloc[0,:].tolist().index(elem) for elem in search_elements]
negative_s_indices = [i-9 for i in negative_s_indices]
all_s_indices = list(range(1, len(np.transpose(A_design))+1))
positive_s_indices = [index for index in all_s_indices if index not in negative_s_indices]
zero_s_indices = [A_df_design.iloc[0,:].tolist().index(elem) for elem in search_elements_zero]
zero_s_indices = [i-9 for i in zero_s_indices]

model.set_negative_scale = Set(initialize=negative_s_indices)
model.set_positive_scale = Set(initialize=positive_s_indices)
model.set_zero_scale = Set(initialize=zero_s_indices)

#model constraint: As=f
model.set_balance = RangeSet(len(f_design))
def cons_rule(model, p):
    return sum(A_design_scenario[p-1,i-1]*model.s[i] for i in model.set_s) == f_design[p-1]
def negative_scale(model, i):
    return (model.s[i]<=0)
def positive_scale(model, i):
    return (model.s[i]>=0)
def zero_scale(model, i):
    return (model.s[i]==0)

##### Resilience Metric
# Define indices
num_rows = 10  # Number of rows/columns in R
indices = range(num_rows)
# Define epsilon for numerical stability
epsilon = 1e-10

def R_rule(model, i, j):
    if i == 0 and j == 1:
        return abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film'].tolist()[0] - 10,
                            A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'multilayer film production'].tolist()[0] - 10]) \
               * abs(model.s[A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'multilayer film production'].tolist()[0] - 9])
    elif i == 0 and j == 9:
        return abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'Ultimate EoL waste'].tolist()[0] - 10,
                            A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'multilayer film production'].tolist()[0] - 10]) \
               * abs(model.s[A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'multilayer film production'].tolist()[0] - 9])
    elif i == 1 and j == 2:
        return abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                            A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'landfill (film)'].tolist()[0] - 10]) \
               * abs(model.s[A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'landfill (film)'].tolist()[0] - 9])
    elif i == 1 and j == 3:
        return abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                            A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'incineration (film)'].tolist()[0] - 10]) \
               * abs(model.s[A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'incineration (film)'].tolist()[0] - 9])
    elif i == 1 and j == 4:
        return abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                            A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'downcycling (allocation)-s1 (heat pretreatment)'].tolist()[0] - 10]) \
               * abs(model.s[A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'downcycling (allocation)-s1 (heat pretreatment)'].tolist()[0] - 9])
    elif i == 1 and j == 5:
        return abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                            A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'downcycling (allocation)-s2 (1/3 discard)'].tolist()[0] - 10]) \
               * abs(model.s[A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'downcycling (allocation)-s2 (1/3 discard)'].tolist()[0] - 9])
    elif i == 1 and j == 6:
        return abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                            A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'STRAP'].tolist()[0] - 10]) \
               * abs(model.s[A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'STRAP'].tolist()[0] - 9])
    elif i == 1 and j == 7:
        return abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                            A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'MASC'].tolist()[0] - 10]) \
               * abs(model.s[A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'MASC'].tolist()[0] - 9])
    elif i == 1 and j == 8:
        return abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                            A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'pyrolysis'].tolist()[0] - 10]) \
               * abs(model.s[A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'pyrolysis'].tolist()[0] - 9])
    elif i == 2 and j == 9:
        return abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                   A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'landfill (film)'].tolist()[0] - 10]) \
           * abs(model.s[A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'landfill (film)'].tolist()[0] - 9])
    elif i == 3 and j == 9:
        return abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                   A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'incineration (film)'].tolist()[0] - 10]) \
           * abs(model.s[A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'incineration (film)'].tolist()[0] - 9])
    elif i == 4 and j == 9:
        return abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                   A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'downcycling (allocation)-s1 (heat pretreatment)'].tolist()[0] - 10]) \
           * abs(model.s[A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'downcycling (allocation)-s1 (heat pretreatment)'].tolist()[0] - 9])
    elif i == 5 and j == 9:
        return abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                   A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'downcycling (allocation)-s2 (1/3 discard)'].tolist()[0] - 10]) \
           * abs(model.s[A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'downcycling (allocation)-s2 (1/3 discard)'].tolist()[0] - 9])
    elif i == 6 and j == 0:
        return abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'Polyethylene, linear low-density, LLDPE, virgin resin, at plant'].tolist()[0] - 10,
                   A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'STRAP'].tolist()[0] - 10]) \
           * abs(model.s[A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'STRAP'].tolist()[0] - 9]) \
     +abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'POLYAMIDE 6 (Nylon 6) (at plant)'].tolist()[0] - 10,
                   A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'STRAP'].tolist()[0] - 10]) \
           * abs(model.s[A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'STRAP'].tolist()[0] - 9])
    elif i == 6 and j == 9:
        return abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'Ultimate EoL waste'].tolist()[0] - 10,
                   A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'STRAP'].tolist()[0] - 10]) \
           * abs(model.s[A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'STRAP'].tolist()[0] - 9])
    elif i == 7 and j == 1:
        return abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film'].tolist()[0] - 10,
                   A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'MASC'].tolist()[0] - 10]) \
           * abs(model.s[A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'MASC'].tolist()[0] - 9])
    elif i == 7 and j == 9:
        return abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'Ultimate EoL waste'].tolist()[0] - 10,
                   A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'MASC'].tolist()[0] - 10]) \
           * abs(model.s[A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'MASC'].tolist()[0] - 9])
    elif i == 8 and j == 9:
        return abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                   A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'pyrolysis'].tolist()[0] - 10]) \
           * abs(model.s[A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'pyrolysis'].tolist()[0] - 9])
    elif i == 9 and j == 0:
        return abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'Polyethylene, linear low-density, LLDPE, virgin resin, at plant'].tolist()[0] - 10,
                            A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'Polyethylene, linear low-density, LLDPE, virgin resin, at plant'].tolist()[0] - 10]) \
               * abs(model.s[A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'Polyethylene, linear low-density, LLDPE, virgin resin, at plant'].tolist()[0] - 9]) \
               + abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'POLYAMIDE 6 (Nylon 6) (at plant)'].tolist()[0] - 10,
                          A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'POLYAMIDE 6 (Nylon 6)'].tolist()[0] - 10]) \
               * abs(model.s[A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'POLYAMIDE 6 (Nylon 6)'].tolist()[0] - 9])
    else:
        return 0

model.R = Expression(indices, indices, rule=R_rule)

# Define the total sum of R
model.R_sum = Expression(expr=sum(model.R[i, j] for i in indices for j in indices))

# Define Development Capacity (DC)
def DC_rule(model):
    return sum(
        -model.R[i, j] * log((model.R[i, j] / model.R_sum) + epsilon) 
        for i in indices for j in indices if i != j
    )
model.DC = Expression(rule=DC_rule)

# Define Ascendency (ASC)
def ASC_rule(model):
    return sum(
        model.R[i, j] * log(
            (model.R[i, j] * model.R_sum) / 
            ((sum(model.R[i, k] for k in indices) + epsilon) * 
             (sum(model.R[k, j] for k in indices) + epsilon)) + epsilon
        ) 
        for i in indices for j in indices if i != j
    )
model.ASC = Expression(rule=ASC_rule)

# Define R_ASC
model.R_ASC = Expression(expr=model.ASC / (model.DC+ epsilon))


model.balance_constraints = Constraint(model.set_balance, rule=cons_rule)
model.negative_scale_constraints = Constraint(model.set_negative_scale, rule=negative_scale)
model.positive_scale_constraints = Constraint(model.set_positive_scale, rule=positive_scale)
model.zero_scale_constraints = Constraint(model.set_zero_scale, rule=zero_scale)
if res==1:  
    model.R_ASC_lower_bound = Constraint(expr=0.213 <= model.R_ASC)
    model.R_ASC_upper_bound = Constraint(expr=model.R_ASC <= 0.589)


model.eps_constraint_1 = Constraint(expr = -(abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'Polyethylene, linear low-density, LLDPE, virgin resin, at plant'].tolist()[0] - 10,
                     np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'Polyethylene, linear low-density, LLDPE, virgin resin, at plant'].tolist()[0] - 10])
    *model.s[np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'Polyethylene, linear low-density, LLDPE, virgin resin, at plant'].tolist()[0] - 9]
        
        + abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'POLYAMIDE 6 (Nylon 6) (at plant)'].tolist()[0] - 10,
                     np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'POLYAMIDE 6 (Nylon 6)'].tolist()[0] - 10])
    *model.s[np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'POLYAMIDE 6 (Nylon 6)'].tolist()[0] - 9]
        
        + (abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                     np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'landfill (film)'].tolist()[0] - 10])
    *model.s[np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'landfill (film)'].tolist()[0] - 9])*(1-0)

                                  + (abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                     np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'incineration (film)'].tolist()[0] - 10])
    *model.s[np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'incineration (film)'].tolist()[0] - 9])*(1-eta_2*gama_2)

                                       + (abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                     np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'pyrolysis'].tolist()[0] - 10])
    *model.s[np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'pyrolysis'].tolist()[0] - 9])*(1-eta_3*gama_3) 

                                  + (abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                     np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'STRAP'].tolist()[0] - 10])
    *model.s[np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'STRAP'].tolist()[0] - 9])*strap_waste

                                  + (abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                     np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'downcycling (allocation)-s1 (heat pretreatment)'].tolist()[0] - 10])
    *model.s[np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'downcycling (allocation)-s1 (heat pretreatment)'].tolist()[0] - 9])*(1-eta_5_s1*gama_5) 

                                     +(abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                     np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'downcycling (allocation)-s2 (1/3 discard)'].tolist()[0] - 10])
    *model.s[np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'downcycling (allocation)-s2 (1/3 discard)'].tolist()[0] - 9])*(1-eta_5_s2*gama_5) 

                                  + (abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                     np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'MASC'].tolist()[0] - 10])
    *model.s[np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'MASC'].tolist()[0] - 9])*masc_waste) - model.s_eps1 == model.epsilon1)


model.eps_constraint_2 = Constraint(expr = -((abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                         np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'landfill (film)'].tolist()[0] - 10])
        *model.s[np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'landfill (film)'].tolist()[0] - 9])
                                      *landf.total_cost

                                      + (abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                         np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'incineration (film)'].tolist()[0] - 10])
        *model.s[np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'incineration (film)'].tolist()[0] - 9])
                                      *landf.total_cost

                                           + (abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                         np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'pyrolysis'].tolist()[0] - 10])
        *model.s[np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'pyrolysis'].tolist()[0] - 9])
                                      *pyrol.total_cost

                                      + (abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                         np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'STRAP'].tolist()[0] - 10])
        *model.s[np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'STRAP'].tolist()[0] - 9])
                                      *_strap.total_cost

                                      + (abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                         np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'downcycling (allocation)-s1 (heat pretreatment)'].tolist()[0] - 10])
        *model.s[np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'downcycling (allocation)-s1 (heat pretreatment)'].tolist()[0] - 9])
                                      *pellet_withheat.total_cost

                                         +(abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                         np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'downcycling (allocation)-s2 (1/3 discard)'].tolist()[0] - 10])
        *model.s[np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'downcycling (allocation)-s2 (1/3 discard)'].tolist()[0] - 9])
                                      *pellet_noheat.total_cost

                                      + (abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                         np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'MASC'].tolist()[0] - 10])
        *model.s[np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'MASC'].tolist()[0] - 9])
                                      *_masc.total_cost) - model.s_eps2 == model.epsilon2)



model.obj1 = Objective(expr = -sum(coef_GWP[i-1]*model.s[i] for i in model.set_s), sense=maximize)

model.obj2 = Objective(expr = -(abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'Polyethylene, linear low-density, LLDPE, virgin resin, at plant'].tolist()[0] - 10,
                     np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'Polyethylene, linear low-density, LLDPE, virgin resin, at plant'].tolist()[0] - 10])
    *model.s[np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'Polyethylene, linear low-density, LLDPE, virgin resin, at plant'].tolist()[0] - 9]
        
        + abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'POLYAMIDE 6 (Nylon 6) (at plant)'].tolist()[0] - 10,
                     np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'POLYAMIDE 6 (Nylon 6)'].tolist()[0] - 10])
    *model.s[np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'POLYAMIDE 6 (Nylon 6)'].tolist()[0] - 9]
        
        + (abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                     np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'landfill (film)'].tolist()[0] - 10])
    *model.s[np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'landfill (film)'].tolist()[0] - 9])*(1-0)

                                  + (abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                     np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'incineration (film)'].tolist()[0] - 10])
    *model.s[np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'incineration (film)'].tolist()[0] - 9])*(1-eta_2*gama_2)

                                       + (abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                     np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'pyrolysis'].tolist()[0] - 10])
    *model.s[np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'pyrolysis'].tolist()[0] - 9])*(1-eta_3*gama_3) 

                                  + (abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                     np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'STRAP'].tolist()[0] - 10])
    *model.s[np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'STRAP'].tolist()[0] - 9])*strap_waste

                                  + (abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                     np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'downcycling (allocation)-s1 (heat pretreatment)'].tolist()[0] - 10])
    *model.s[np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'downcycling (allocation)-s1 (heat pretreatment)'].tolist()[0] - 9])*(1-eta_5_s1*gama_5) 

                                     +(abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                     np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'downcycling (allocation)-s2 (1/3 discard)'].tolist()[0] - 10])
    *model.s[np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'downcycling (allocation)-s2 (1/3 discard)'].tolist()[0] - 9])*(1-eta_5_s2*gama_5) 

                                  + (abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                     np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'MASC'].tolist()[0] - 10])
    *model.s[np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'MASC'].tolist()[0] - 9])*masc_waste), sense=maximize
            )

model.obj3 = Objective(expr = -((abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                         np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'landfill (film)'].tolist()[0] - 10])
        *model.s[np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'landfill (film)'].tolist()[0] - 9])
                                      *landf.total_cost

                                      + (abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                         np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'incineration (film)'].tolist()[0] - 10])
        *model.s[np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'incineration (film)'].tolist()[0] - 9])
                                      *landf.total_cost

                                           + (abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                         np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'pyrolysis'].tolist()[0] - 10])
        *model.s[np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'pyrolysis'].tolist()[0] - 9])
                                      *pyrol.total_cost

                                      + (abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                         np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'STRAP'].tolist()[0] - 10])
        *model.s[np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'STRAP'].tolist()[0] - 9])
                                      *_strap.total_cost

                                      + (abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                         np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'downcycling (allocation)-s1 (heat pretreatment)'].tolist()[0] - 10])
        *model.s[np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'downcycling (allocation)-s1 (heat pretreatment)'].tolist()[0] - 9])
                                      *pellet_withheat.total_cost

                                         +(abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                         np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'downcycling (allocation)-s2 (1/3 discard)'].tolist()[0] - 10])
        *model.s[np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'downcycling (allocation)-s2 (1/3 discard)'].tolist()[0] - 9])
                                      *pellet_noheat.total_cost

                                      + (abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                         np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'MASC'].tolist()[0] - 10])
        *model.s[np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'MASC'].tolist()[0] - 9])
                                      *_masc.total_cost), sense=maximize
         )

#Solver
if res==0:
    solver = SolverFactory('gurobi')
elif res==1:
    solver = SolverFactory('ipopt')

### Epsilon bonds

In [ ]:
model.obj1.activate() 
model.obj2.deactivate()
model.obj3.deactivate()
model.eps_constraint_1.deactivate()
model.eps_constraint_2.deactivate()
results = solver.solve(model, tee=True) # solves and updates instance
Z1_1_median=value(model.obj1)
Z2_1_median=value(model.obj2)
Z3_1_median=value(model.obj3)

In [ ]:
model.obj1.deactivate() 
model.obj2.activate() 
model.obj3.deactivate()
model.eps_constraint_1.deactivate()
model.eps_constraint_2.deactivate()
results = solver.solve(model, tee=True) # solves and updates instance
Z1_2_median= value(model.obj1)
Z2_2_median= value(model.obj2)
Z3_2_median= value(model.obj3)
scaling_factors_circ = []
for j in model.set_s:
    scaling_factors_circ.append(model.s[j].value)

In [ ]:
model.obj1.deactivate() 
model.obj2.deactivate()
model.obj3.activate()
model.eps_constraint_1.deactivate()
model.eps_constraint_2.deactivate()
results = solver.solve(model, tee=True) # solves and updates instance
Z1_3_median= value(model.obj1)
Z2_3_median= value(model.obj2)
Z3_3_median= value(model.obj3)

### Solving the Optimization Problem

In [ ]:
M1_median = max(Z1_1_median,Z1_2_median,Z1_3_median)
n1_median = min(Z1_1_best,Z1_2_best,Z1_3_best)

M2_median = max(Z2_1_median,Z2_2_median,Z2_3_median)
n2_median = min(Z2_1_median,Z2_2_median,Z2_3_median)

M3_median = max(Z3_1_median,Z3_2_median,Z3_3_median)
n3_median = min(Z3_1_median,Z3_2_median,Z3_3_median)

In [ ]:
M_penalty = 1e-4
delta_1 =  M2_median - n2_median
delta_2 = M3_median - n3_median
model.obj1_aug = Objective(expr = -sum(coef_GWP[i-1]*model.s[i] for i in model.set_s) + M_penalty*(model.s_eps1/delta_1 + model.s_eps2/delta_2), sense=maximize)

In [ ]:
#Z1, Z2, and Z3 values!!! for pareto surface generation

#Best
model.obj1.deactivate()
model.obj1_aug.activate()
model.obj2.deactivate() 
model.obj3.deactivate()
model.eps_constraint_1.activate()
model.eps_constraint_2.activate()


r=100
Z1=[]
Z2=[]
Z3=[]
solution_matrix = []

for t in range(0,r):
    for i in range (0,r):
        model.epsilon1 = n2_median + (M2_median - n2_median) * t / (r - 1)
        model.epsilon2 = n3_median + (M3_median - n3_median) * i / (r - 1)
        results = solver.solve(model) # solves and updates instance
        
        if results.solver.termination_condition == TerminationCondition.optimal:           
            Z1.append(-value(model.obj1))
            Z2.append(-value(model.obj2))
            Z3.append(-value(model.obj3))
            
            
            scaling_factors = []
            for j in model.set_s:
                scaling_factors.append(model.s[j].value)
            solution_matrix.append(scaling_factors) 

Z1_median.append(-Z1_2_median)
Z2_median.append(-Z2_2_median)
Z3_median.append(-Z3_2_median)

In [ ]:
solution_matrix.append(scaling_factors_circ)

### Generating the pareto frontier numbers

In [ ]:
# Assuming Z1, Z2, and Z3 are lists of values

# Create a dictionary to store unique combinations
unique_combinations = {}

# Iterate over the lists and store unique combinations in the dictionary
for z1, z2, z3 in zip(Z1_median, Z2_median, Z3_median):
    combination = (z1, z2, z3)
    if combination not in unique_combinations:
        unique_combinations[combination] = 1  # Value doesn't matter, just using the key for uniqueness

# Extract unique values from the dictionary
unique_Z1, unique_Z2, unique_Z3 = zip(*unique_combinations.keys())

In [ ]:
# Assuming Z1, Z2, and Z3 are lists of values
# Create a dictionary to store unique combinations and their indices
unique_combinations = {}
indices = []
unique_solution_matrix = []

# Iterate over the lists and store unique combinations and their indices in the dictionary
for i, (z1, z2, z3) in enumerate(zip(Z1_median, Z2_median, Z3_median)):
    combination = (z1, z2, z3)
    if combination not in unique_combinations:
        unique_combinations[combination] = [i]  # Store the index as a list
    else:
        unique_combinations[combination].append(i)  # Append the index to the existing list

# Extract unique values and their corresponding indices from the dictionary
indices = list(unique_combinations.values())

# New list with only the first elements of each sublist
unique_solution_indices = [sublist[0] for sublist in indices]

unique_solution_matrix = [solution_matrix[index] for index in unique_solution_indices]

In [ ]:
pareto_GWP = unique_Z1
pareto_circularity = [(1 - (value / (2 * f_design[0]))) for value in unique_Z2]
pareto_cost = unique_Z3

#pareto front points
pareto_points = [[pareto_GWP[i], pareto_circularity[i], pareto_cost[i]] for i in range(len(pareto_GWP))]

In [ ]:
#utopia point

utopia_point_GWP = min(pareto_GWP)
scaled_utopia_point_GWP = (utopia_point_GWP - min(pareto_GWP)) / (max(pareto_GWP) - min(pareto_GWP))

utopia_point_circularity = max(pareto_circularity)
scaled_utopia_point_circularity = (utopia_point_circularity - min(pareto_circularity)) / (max(pareto_circularity) - min(pareto_circularity))

utopia_point_cost = min(pareto_cost)
scaled_utopia_point_cost = (utopia_point_cost - min(pareto_cost)) / (max(pareto_cost) - min(pareto_cost))                                                     

In [ ]:
#Compromised Solution

scaled_pareto_GWP = [(element - min(pareto_GWP)) / (max(pareto_GWP) - min(pareto_GWP)) for element in pareto_GWP]
                                                                     

scaled_pareto_circularity = [(element - min(pareto_circularity)) / (max(pareto_circularity) - min(pareto_circularity)) for element in pareto_circularity]


scaled_pareto_cost = [(element - min(pareto_cost)) / (max(pareto_cost) - min(pareto_cost)) for element in pareto_cost]


# Calculate the distance to the utopia point
distances = []
for i in range(len(scaled_pareto_GWP)):
    distances.append(
        np.sqrt((scaled_pareto_GWP[i] - scaled_utopia_point_GWP) ** 2  
        + (scaled_pareto_circularity[i] - scaled_utopia_point_circularity) ** 2)
        + (scaled_pareto_cost[i] - scaled_utopia_point_cost) ** 2)
index_min_distance = np.argmin(distances)

In [ ]:
#Solutions' scaling factors
index_min_GWP_solution = np.argmin(pareto_GWP)
index_max_circularity_solution = np.argmax(pareto_circularity)
index_min_cost_solution = np.argmin(pareto_cost)
index_compromised_solution = index_min_distance

min_GWP_solution = unique_solution_matrix[index_min_GWP_solution]
max_circularity_solution = unique_solution_matrix[index_max_circularity_solution]
min_cost_solution = unique_solution_matrix[index_min_cost_solution]
compromised_solution = unique_solution_matrix[index_compromised_solution]

## 1.3. Worst solution

In [ ]:
#Uncertainty scenario
scen = 2

if scen ==0:
    #Best case values
    masc_solv_draws = 0.01
    masc_ener_draws = 0.0003694444
    masc_eff_draws = 0.95
    masc_cycles = 100

    strap_solv_draws = 0.002
    strap_ener_draws = 0.00001555556
    strap_eff_draws = 0.95
    strap_cycles = 100
    
elif scen ==1:
    # Expected values
    masc_solv_draws = 0.1
    masc_ener_draws = 0.0007388887
    masc_eff_draws = 0.875
    masc_cycles = 5

    strap_solv_draws = 0.1
    strap_ener_draws = 0.000023472225
    strap_eff_draws = 0.875
    strap_cycles = 5

elif scen==2:
    #Worst case values
    masc_solv_draws = 0.2
    masc_ener_draws = 0.001108333
    masc_eff_draws = 0.80
    masc_cycles = 1

    strap_solv_draws = 0.2
    strap_ener_draws = 0.000023472225
    strap_eff_draws = 0.80
    strap_cycles = 1
    
#TEA
masc_des = masc_design.loc[(masc_design.Index.isin(['Electricity','Diacetone alcohol']))  | (masc_design.Variable == 'Output efficiency')]
strap_des = strap_design.loc[(strap_design.Index.isin(['Electricity','Ethylene glycol']))  | (strap_design.Variable == 'Output efficiency')]


A_design_scenario = []
masc_unc_costs = []
strap_unc_costs = []
masc_unc_waste = []
strap_unc_waste = []



A_design_scenario = A_design.copy()

## MASC ##

A_design_scenario [A_df_design.index[A_df_design.iloc[:, 1] == 'Diacetone Alcohol'].tolist()[0] - 10,
                 np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'MASC'].tolist()[0] - 10] = - 5.56 * (masc_solv_draws)*(1/masc_eff_draws)*calculate_sum_mi_masc(masc_eff_draws, masc_cycles)

A_design_scenario [A_df_design.index[A_df_design.iloc[:, 0] == 'Electricity, at Grid, US, 2010'].tolist()[0] - 10,
                 np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'MASC'].tolist()[0] - 10] = - 5.56 * 2.7 * (1/masc_eff_draws)*calculate_sum_mi_masc(masc_eff_draws, masc_cycles)

A_design_scenario [A_df_design.index[A_df_design.iloc[:, 1] == 'Ultimate EoL waste'].tolist()[0] - 10,
                 np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'MASC'].tolist()[0] - 10] = ((1-masc_eff_draws)/masc_eff_draws)*calculate_sum_mi_masc(masc_eff_draws, masc_cycles) + calculate_mi_masc(masc_eff_draws,masc_cycles,masc_cycles)

A_design_scenario [A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film'].tolist()[0] - 10,
                 np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'MASC'].tolist()[0] - 10] = 1 - (calculate_mp_masc(masc_eff_draws, masc_cycles))
## STRAP ##

A_design_scenario [A_df_design.index[A_df_design.iloc[:, 0] == 'Electricity, at Grid, US, 2010'].tolist()[0] - 10,
                 np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'STRAP'].tolist()[0] - 10] = - (1/0.9) * 0.648 * (1/strap_eff_draws) * calculate_sum_mi_strap(strap_eff_draws, strap_cycles)

A_design_scenario [A_df_design.index[A_df_design.iloc[:, 1] == 'Ethylene glycol, at plant, kg'].tolist()[0] - 10,
                 np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'STRAP'].tolist()[0] - 10] = - 11.5 * (1/0.9) * (strap_solv_draws)*(1/strap_eff_draws)*calculate_sum_mi_strap(strap_eff_draws, strap_cycles)

A_design_scenario [A_df_design.index[A_df_design.iloc[:, 1] == 'heat, from natural gas'].tolist()[0] - 10,
                 np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'STRAP'].tolist()[0] - 10] = - (1/0.9) * 0.00351 * (1/strap_eff_draws) * calculate_sum_mi_strap(strap_eff_draws, strap_cycles) 

A_design_scenario [A_df_design.index[A_df_design.iloc[:, 1] == 'Ultimate EoL waste'].tolist()[0] - 10,
                 np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'STRAP'].tolist()[0] - 10] = ((1-strap_eff_draws)/strap_eff_draws)*(1/0.9)*calculate_sum_mi_strap(strap_eff_draws, strap_cycles) + calculate_mi_strap(strap_eff_draws,strap_cycles,strap_cycles)

A_design_scenario [A_df_design.index[A_df_design.iloc[:, 1] == 'POLYAMIDE 6 (Nylon 6) (at plant)'].tolist()[0] - 10,
                 np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'STRAP'].tolist()[0] - 10] = 0.26/0.9 - (0.26/0.9)*calculate_mp_strap(strap_eff_draws, strap_cycles)

A_design_scenario [A_df_design.index[A_df_design.iloc[:, 1] == 'Polyethylene, linear low-density, LLDPE, virgin resin, at plant'].tolist()[0] - 10,
                 np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'STRAP'].tolist()[0] - 10] = 0.74/0.9 - (0.74/0.9)*calculate_mp_strap(strap_eff_draws, strap_cycles)



masc_des.loc[masc_des.Index == 'Electricity','Value'] = 2521.974 * masc_ener_draws
masc_des.loc[masc_des.Index == 'Diacetone alcohol','Value'] = 5560 * masc_solv_draws
masc_des.loc[masc_des.Variable == 'Output efficiency','Value'] = masc_eff_draws

_masc = Scenario(
    func_unit = 2.204,
    sens_df = masc_des,
    eol_sc = {'Mechanical and Solvent Cleaning': [1.0, masc_cycles]},
    products = {'Mechanical and Solvent Cleaning': 'Barrier film'}
)


strap_des.loc[strap_des.Index == 'Electricity', 'Value'] = 11400 * strap_ener_draws
strap_des.loc[strap_des.Index == 'Ethylene glycol', 'Value'] = 25132.698 * strap_solv_draws
strap_des.loc[strap_des.Variable == 'Output efficiency', 'Value'] = strap_eff_draws

_strap = Scenario(
    func_unit = 2.204,
    sens_df = strap_des,
    eol_sc = {'Solvent Treatment and Precipitation': [1.0, strap_cycles]},
    products = {'Solvent Treatment and Precipitation':'Polyethylene'}
)

masc_waste = ((1-masc_eff_draws)/masc_eff_draws)*calculate_sum_mi_masc(masc_eff_draws, masc_cycles) + calculate_mi_masc(masc_eff_draws,masc_cycles,masc_cycles)
strap_waste = ((1-strap_eff_draws)/strap_eff_draws)*(1/0.9)*calculate_sum_mi_strap(strap_eff_draws, strap_cycles) + calculate_mi_strap(strap_eff_draws,strap_cycles,strap_cycles)

## 1.4. All uncertainty casess solutions

In [ ]:
M1 = max(Z1_1_best,Z1_2_best,Z1_3_best,Z1_1_worst,Z1_2_worst,Z1_3_worst,Z1_1_median,Z1_2_median,Z1_3_median)
n1 = min(Z1_1_best,Z1_2_best,Z1_3_best,Z1_1_worst,Z1_2_worst,Z1_3_worst,Z1_1_median,Z1_2_median,Z1_3_median)

M2 = max(Z2_1_best,Z2_2_best,Z2_3_best,Z2_1_worst,Z2_2_worst,Z2_3_worst,Z2_1_median,Z2_2_median,Z2_3_median)
n2 = min(Z2_1_best,Z2_2_best,Z2_3_best,Z2_1_worst,Z2_2_worst,Z2_3_worst,Z2_1_median,Z2_2_median,Z2_3_median)

M3 = max(Z3_1_best,Z3_2_best,Z3_3_best,Z3_1_worst,Z3_2_worst,Z3_3_worst,Z3_1_median,Z3_2_median,Z3_3_median)
n3 = min(Z3_1_best,Z3_2_best,Z3_3_best,Z3_1_worst,Z3_2_worst,Z3_3_worst,Z3_1_median,Z3_2_median,Z3_3_median)

In [ ]:
#Z1, Z2, and Z3 values!!! for pareto surface generation

#Best

#model.obj1_aug.activate
model.obj1.activate() 
model.obj2.deactivate() 
model.obj3.deactivate()
model.eps_constraint_1.activate()
model.eps_constraint_2.activate()


r=10

solution_matrix = []

hist1_wo_res = []
hist2_wo_res = []
hist3_wo_res = []



for t in range(0,r):
    for i in range (0,r):
        model.epsilon1 = n2_closest + (M2_closest - n2_closest) * t / (r - 1)
        model.epsilon2 = n3_closest + (M3_closest - n3_closest) * i / (r - 1)
        results = solver.solve(model) # solves and updates instance
        
        if results.solver.termination_condition == TerminationCondition.optimal:
            from pyomo.environ import value
            scenario_obj1_values = {}
            for sc in model.scenarios:
                scenario_obj1_values[sc] = - value(model.obj1_scenario[sc])

            scenario_obj2_values = {}
            for sc in model.scenarios:
                scenario_obj2_values[sc] = - value(model.obj2_scenario[sc])

            scenario_obj3_values = {}
            for sc in model.scenarios:
                scenario_obj3_values[sc] = - value(model.obj3_scenario[sc])
            
            avg_obj_1 = sum(scenario_obj1_values.values()) / len(scenario_obj1_values)

            def distance_to_avg_1(sc):
                return abs(scenario_obj1_values[sc] - avg_obj_1)

            median_sc_obj1 = min(model.scenarios, key=distance_to_avg_1)
            # 3a. Best scenario (min)
            best_sc_obj1 = min(scenario_obj1_values, key=scenario_obj1_values.get)

            # 3b. Worst scenario (max)
            worst_sc_obj1 = max(scenario_obj1_values, key=scenario_obj1_values.get)
            
            
            avg_obj_2 = sum(scenario_obj2_values.values()) / len(scenario_obj2_values)

            def distance_to_avg_2(sc):
                return abs(scenario_obj2_values[sc] - avg_obj)

            median_sc_obj2 = min(model.scenarios, key=distance_to_avg_2)
            # 3a. Best scenario (min)
            best_sc_obj2 = min(scenario_obj2_values, key=scenario_obj2_values.get)

            # 3b. Worst scenario (max)
            worst_sc_obj2 = max(scenario_obj2_values, key=scenario_obj2_values.get)
            
            avg_obj_3 = sum(scenario_obj3_values.values()) / len(scenario_obj3_values)

            def distance_to_avg_3(sc):
                return abs(scenario_obj3_values[sc] - avg_obj)

            median_sc_obj3 = min(model.scenarios, key=distance_to_avg_3)
            # 3a. Best scenario (min)
            best_sc_obj3 = min(scenario_obj3_values, key=scenario_obj3_values.get)

            # 3b. Worst scenario (max)
            worst_sc_obj3 = max(scenario_obj3_values, key=scenario_obj3_values.get)
            
            
            hist1_wo_res.append(scenario_obj1_values)
            hist2_wo_res.append(scenario_obj2_values)
            hist3_wo_res.append(scenario_obj3_values)          

### Multi-objective optimization formulation

In [ ]:
#Model Formulation
# Create the model
model = ConcreteModel()

res = 0

model.s_eps1 = Var(within=NonNegativeReals, bounds=(1e-6, None))  # Slack for epsilon constraint 1
model.s_eps2 = Var(within=NonNegativeReals, bounds=(1e-6, None))  # Slack for epsilon constraint 2


#epsilon parameter
model.epsilon1=Param(initialize=1000,mutable=True)
model.epsilon2=Param(initialize=1000,mutable=True)

# Define the decision variable
model.set_s = RangeSet(len(np.transpose(A_design)))
model.s = Var(model.set_s)
for i in model.set_s:
    model.s[i].set_value(1.0)  # Adjust based on the problem's context
# List of processes with negative scaling factor due to the substitution approach
negative_s_indices = []
positive_s_indices = []
all_s_indices = []
search_elements = [
    'Substitued Electricity US Market',
    'Substitued Polypropylene Market',
    'Substitued Refinery Oil Market'
]

search_elements_zero = [
    #'pyrolysis',
    #'STRAP',
    #'MASC',
    #'landfill (film)',
    #'downcycling (allocation)-s1 (heat pretreatment)',
    #'MASC',
    #'incineration (film)'
]

# Search for elements in the first row of the DataFrame
negative_s_indices = [A_df_design.iloc[0,:].tolist().index(elem) for elem in search_elements]
negative_s_indices = [i-9 for i in negative_s_indices]
all_s_indices = list(range(1, len(np.transpose(A_design))+1))
positive_s_indices = [index for index in all_s_indices if index not in negative_s_indices]
zero_s_indices = [A_df_design.iloc[0,:].tolist().index(elem) for elem in search_elements_zero]
zero_s_indices = [i-9 for i in zero_s_indices]

model.set_negative_scale = Set(initialize=negative_s_indices)
model.set_positive_scale = Set(initialize=positive_s_indices)
model.set_zero_scale = Set(initialize=zero_s_indices)

#model constraint: As=f
model.set_balance = RangeSet(len(f_design))
def cons_rule(model, p):
    return sum(A_design_scenario[p-1,i-1]*model.s[i] for i in model.set_s) == f_design[p-1]
def negative_scale(model, i):
    return (model.s[i]<=0)
def positive_scale(model, i):
    return (model.s[i]>=0)
def zero_scale(model, i):
    return (model.s[i]==0)

##### Resilience Metric
# Define indices
num_rows = 10  # Number of rows/columns in R
indices = range(num_rows)
# Define epsilon for numerical stability
epsilon = 1e-10

def R_rule(model, i, j):
    if i == 0 and j == 1:
        return abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film'].tolist()[0] - 10,
                            A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'multilayer film production'].tolist()[0] - 10]) \
               * abs(model.s[A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'multilayer film production'].tolist()[0] - 9])
    elif i == 0 and j == 9:
        return abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'Ultimate EoL waste'].tolist()[0] - 10,
                            A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'multilayer film production'].tolist()[0] - 10]) \
               * abs(model.s[A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'multilayer film production'].tolist()[0] - 9])
    elif i == 1 and j == 2:
        return abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                            A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'landfill (film)'].tolist()[0] - 10]) \
               * abs(model.s[A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'landfill (film)'].tolist()[0] - 9])
    elif i == 1 and j == 3:
        return abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                            A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'incineration (film)'].tolist()[0] - 10]) \
               * abs(model.s[A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'incineration (film)'].tolist()[0] - 9])
    elif i == 1 and j == 4:
        return abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                            A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'downcycling (allocation)-s1 (heat pretreatment)'].tolist()[0] - 10]) \
               * abs(model.s[A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'downcycling (allocation)-s1 (heat pretreatment)'].tolist()[0] - 9])
    elif i == 1 and j == 5:
        return abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                            A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'downcycling (allocation)-s2 (1/3 discard)'].tolist()[0] - 10]) \
               * abs(model.s[A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'downcycling (allocation)-s2 (1/3 discard)'].tolist()[0] - 9])
    elif i == 1 and j == 6:
        return abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                            A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'STRAP'].tolist()[0] - 10]) \
               * abs(model.s[A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'STRAP'].tolist()[0] - 9])
    elif i == 1 and j == 7:
        return abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                            A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'MASC'].tolist()[0] - 10]) \
               * abs(model.s[A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'MASC'].tolist()[0] - 9])
    elif i == 1 and j == 8:
        return abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                            A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'pyrolysis'].tolist()[0] - 10]) \
               * abs(model.s[A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'pyrolysis'].tolist()[0] - 9])
    elif i == 2 and j == 9:
        return abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                   A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'landfill (film)'].tolist()[0] - 10]) \
           * abs(model.s[A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'landfill (film)'].tolist()[0] - 9])
    elif i == 3 and j == 9:
        return abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                   A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'incineration (film)'].tolist()[0] - 10]) \
           * abs(model.s[A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'incineration (film)'].tolist()[0] - 9])
    elif i == 4 and j == 9:
        return abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                   A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'downcycling (allocation)-s1 (heat pretreatment)'].tolist()[0] - 10]) \
           * abs(model.s[A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'downcycling (allocation)-s1 (heat pretreatment)'].tolist()[0] - 9])
    elif i == 5 and j == 9:
        return abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                   A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'downcycling (allocation)-s2 (1/3 discard)'].tolist()[0] - 10]) \
           * abs(model.s[A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'downcycling (allocation)-s2 (1/3 discard)'].tolist()[0] - 9])
    elif i == 6 and j == 0:
        return abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'Polyethylene, linear low-density, LLDPE, virgin resin, at plant'].tolist()[0] - 10,
                   A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'STRAP'].tolist()[0] - 10]) \
           * abs(model.s[A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'STRAP'].tolist()[0] - 9]) \
     +abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'POLYAMIDE 6 (Nylon 6) (at plant)'].tolist()[0] - 10,
                   A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'STRAP'].tolist()[0] - 10]) \
           * abs(model.s[A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'STRAP'].tolist()[0] - 9])
    elif i == 6 and j == 9:
        return abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'Ultimate EoL waste'].tolist()[0] - 10,
                   A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'STRAP'].tolist()[0] - 10]) \
           * abs(model.s[A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'STRAP'].tolist()[0] - 9])
    elif i == 7 and j == 1:
        return abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film'].tolist()[0] - 10,
                   A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'MASC'].tolist()[0] - 10]) \
           * abs(model.s[A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'MASC'].tolist()[0] - 9])
    elif i == 7 and j == 9:
        return abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'Ultimate EoL waste'].tolist()[0] - 10,
                   A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'MASC'].tolist()[0] - 10]) \
           * abs(model.s[A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'MASC'].tolist()[0] - 9])
    elif i == 8 and j == 9:
        return abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                   A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'pyrolysis'].tolist()[0] - 10]) \
           * abs(model.s[A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'pyrolysis'].tolist()[0] - 9])
    elif i == 9 and j == 0:
        return abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'Polyethylene, linear low-density, LLDPE, virgin resin, at plant'].tolist()[0] - 10,
                            A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'Polyethylene, linear low-density, LLDPE, virgin resin, at plant'].tolist()[0] - 10]) \
               * abs(model.s[A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'Polyethylene, linear low-density, LLDPE, virgin resin, at plant'].tolist()[0] - 9]) \
               + abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'POLYAMIDE 6 (Nylon 6) (at plant)'].tolist()[0] - 10,
                          A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'POLYAMIDE 6 (Nylon 6)'].tolist()[0] - 10]) \
               * abs(model.s[A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'POLYAMIDE 6 (Nylon 6)'].tolist()[0] - 9])
    else:
        return 0

model.R = Expression(indices, indices, rule=R_rule)

# Define the total sum of R
model.R_sum = Expression(expr=sum(model.R[i, j] for i in indices for j in indices))

# Define Development Capacity (DC)
def DC_rule(model):
    return sum(
        -model.R[i, j] * log((model.R[i, j] / model.R_sum) + epsilon) 
        for i in indices for j in indices if i != j
    )
model.DC = Expression(rule=DC_rule)

# Define Ascendency (ASC)
def ASC_rule(model):
    return sum(
        model.R[i, j] * log(
            (model.R[i, j] * model.R_sum) / 
            ((sum(model.R[i, k] for k in indices) + epsilon) * 
             (sum(model.R[k, j] for k in indices) + epsilon)) + epsilon
        ) 
        for i in indices for j in indices if i != j
    )
model.ASC = Expression(rule=ASC_rule)

# Define R_ASC
model.R_ASC = Expression(expr=model.ASC / (model.DC+ epsilon))


model.balance_constraints = Constraint(model.set_balance, rule=cons_rule)
model.negative_scale_constraints = Constraint(model.set_negative_scale, rule=negative_scale)
model.positive_scale_constraints = Constraint(model.set_positive_scale, rule=positive_scale)
model.zero_scale_constraints = Constraint(model.set_zero_scale, rule=zero_scale)
if res==1:  
    model.R_ASC_lower_bound = Constraint(expr=0.213 <= model.R_ASC)
    model.R_ASC_upper_bound = Constraint(expr=model.R_ASC <= 0.589)


model.eps_constraint_1 = Constraint(expr = -(abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'Polyethylene, linear low-density, LLDPE, virgin resin, at plant'].tolist()[0] - 10,
                     np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'Polyethylene, linear low-density, LLDPE, virgin resin, at plant'].tolist()[0] - 10])
    *model.s[np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'Polyethylene, linear low-density, LLDPE, virgin resin, at plant'].tolist()[0] - 9]
        
        + abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'POLYAMIDE 6 (Nylon 6) (at plant)'].tolist()[0] - 10,
                     np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'POLYAMIDE 6 (Nylon 6)'].tolist()[0] - 10])
    *model.s[np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'POLYAMIDE 6 (Nylon 6)'].tolist()[0] - 9]
        
        + (abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                     np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'landfill (film)'].tolist()[0] - 10])
    *model.s[np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'landfill (film)'].tolist()[0] - 9])*(1-0)

                                  + (abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                     np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'incineration (film)'].tolist()[0] - 10])
    *model.s[np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'incineration (film)'].tolist()[0] - 9])*(1-eta_2*gama_2)

                                       + (abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                     np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'pyrolysis'].tolist()[0] - 10])
    *model.s[np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'pyrolysis'].tolist()[0] - 9])*(1-eta_3*gama_3) 

                                  + (abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                     np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'STRAP'].tolist()[0] - 10])
    *model.s[np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'STRAP'].tolist()[0] - 9])*strap_waste

                                  + (abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                     np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'downcycling (allocation)-s1 (heat pretreatment)'].tolist()[0] - 10])
    *model.s[np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'downcycling (allocation)-s1 (heat pretreatment)'].tolist()[0] - 9])*(1-eta_5_s1*gama_5) 

                                     +(abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                     np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'downcycling (allocation)-s2 (1/3 discard)'].tolist()[0] - 10])
    *model.s[np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'downcycling (allocation)-s2 (1/3 discard)'].tolist()[0] - 9])*(1-eta_5_s2*gama_5) 

                                  + (abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                     np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'MASC'].tolist()[0] - 10])
    *model.s[np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'MASC'].tolist()[0] - 9])*masc_waste) - model.s_eps1 == model.epsilon1)


model.eps_constraint_2 = Constraint(expr = -((abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                         np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'landfill (film)'].tolist()[0] - 10])
        *model.s[np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'landfill (film)'].tolist()[0] - 9])
                                      *landf.total_cost

                                      + (abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                         np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'incineration (film)'].tolist()[0] - 10])
        *model.s[np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'incineration (film)'].tolist()[0] - 9])
                                      *landf.total_cost

                                           + (abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                         np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'pyrolysis'].tolist()[0] - 10])
        *model.s[np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'pyrolysis'].tolist()[0] - 9])
                                      *pyrol.total_cost

                                      + (abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                         np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'STRAP'].tolist()[0] - 10])
        *model.s[np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'STRAP'].tolist()[0] - 9])
                                      *_strap.total_cost

                                      + (abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                         np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'downcycling (allocation)-s1 (heat pretreatment)'].tolist()[0] - 10])
        *model.s[np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'downcycling (allocation)-s1 (heat pretreatment)'].tolist()[0] - 9])
                                      *pellet_withheat.total_cost

                                         +(abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                         np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'downcycling (allocation)-s2 (1/3 discard)'].tolist()[0] - 10])
        *model.s[np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'downcycling (allocation)-s2 (1/3 discard)'].tolist()[0] - 9])
                                      *pellet_noheat.total_cost

                                      + (abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                         np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'MASC'].tolist()[0] - 10])
        *model.s[np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'MASC'].tolist()[0] - 9])
                                      *_masc.total_cost) - model.s_eps2 == model.epsilon2)



model.obj1 = Objective(expr = -sum(coef_GWP[i-1]*model.s[i] for i in model.set_s), sense=maximize)

model.obj2 = Objective(expr = -(abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'Polyethylene, linear low-density, LLDPE, virgin resin, at plant'].tolist()[0] - 10,
                     np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'Polyethylene, linear low-density, LLDPE, virgin resin, at plant'].tolist()[0] - 10])
    *model.s[np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'Polyethylene, linear low-density, LLDPE, virgin resin, at plant'].tolist()[0] - 9]
        
        + abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'POLYAMIDE 6 (Nylon 6) (at plant)'].tolist()[0] - 10,
                     np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'POLYAMIDE 6 (Nylon 6)'].tolist()[0] - 10])
    *model.s[np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'POLYAMIDE 6 (Nylon 6)'].tolist()[0] - 9]
        
        + (abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                     np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'landfill (film)'].tolist()[0] - 10])
    *model.s[np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'landfill (film)'].tolist()[0] - 9])*(1-0)

                                  + (abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                     np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'incineration (film)'].tolist()[0] - 10])
    *model.s[np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'incineration (film)'].tolist()[0] - 9])*(1-eta_2*gama_2)

                                       + (abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                     np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'pyrolysis'].tolist()[0] - 10])
    *model.s[np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'pyrolysis'].tolist()[0] - 9])*(1-eta_3*gama_3) 

                                  + (abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                     np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'STRAP'].tolist()[0] - 10])
    *model.s[np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'STRAP'].tolist()[0] - 9])*strap_waste

                                  + (abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                     np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'downcycling (allocation)-s1 (heat pretreatment)'].tolist()[0] - 10])
    *model.s[np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'downcycling (allocation)-s1 (heat pretreatment)'].tolist()[0] - 9])*(1-eta_5_s1*gama_5) 

                                     +(abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                     np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'downcycling (allocation)-s2 (1/3 discard)'].tolist()[0] - 10])
    *model.s[np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'downcycling (allocation)-s2 (1/3 discard)'].tolist()[0] - 9])*(1-eta_5_s2*gama_5) 

                                  + (abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                     np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'MASC'].tolist()[0] - 10])
    *model.s[np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'MASC'].tolist()[0] - 9])*masc_waste), sense=maximize
            )

model.obj3 = Objective(expr = -((abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                         np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'landfill (film)'].tolist()[0] - 10])
        *model.s[np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'landfill (film)'].tolist()[0] - 9])
                                      *landf.total_cost

                                      + (abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                         np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'incineration (film)'].tolist()[0] - 10])
        *model.s[np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'incineration (film)'].tolist()[0] - 9])
                                      *landf.total_cost

                                           + (abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                         np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'pyrolysis'].tolist()[0] - 10])
        *model.s[np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'pyrolysis'].tolist()[0] - 9])
                                      *pyrol.total_cost

                                      + (abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                         np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'STRAP'].tolist()[0] - 10])
        *model.s[np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'STRAP'].tolist()[0] - 9])
                                      *_strap.total_cost

                                      + (abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                         np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'downcycling (allocation)-s1 (heat pretreatment)'].tolist()[0] - 10])
        *model.s[np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'downcycling (allocation)-s1 (heat pretreatment)'].tolist()[0] - 9])
                                      *pellet_withheat.total_cost

                                         +(abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                         np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'downcycling (allocation)-s2 (1/3 discard)'].tolist()[0] - 10])
        *model.s[np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'downcycling (allocation)-s2 (1/3 discard)'].tolist()[0] - 9])
                                      *pellet_noheat.total_cost

                                      + (abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                         np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'MASC'].tolist()[0] - 10])
        *model.s[np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'MASC'].tolist()[0] - 9])
                                      *_masc.total_cost), sense=maximize
         )

#Solver
if res==0:
    solver = SolverFactory('gurobi')
elif res==1:
    solver = SolverFactory('ipopt')

### Epsilon bonds

In [ ]:
model.obj1.activate() 
model.obj2.deactivate()
model.obj3.deactivate()
model.eps_constraint_1.deactivate()
model.eps_constraint_2.deactivate()
results = solver.solve(model, tee=True) # solves and updates instance
Z1_1_worst=value(model.obj1)
Z2_1_worst=value(model.obj2)
Z3_1_worst=value(model.obj3)

In [ ]:
model.obj1.deactivate() 
model.obj2.activate() 
model.obj3.deactivate()
model.eps_constraint_1.deactivate()
model.eps_constraint_2.deactivate()
results = solver.solve(model, tee=True) # solves and updates instance
Z1_2_worst= value(model.obj1)
Z2_2_worst= value(model.obj2)
Z3_2_worst= value(model.obj3)
scaling_factors_circ = []
for j in model.set_s:
    scaling_factors_circ.append(model.s[j].value)

In [ ]:
model.obj1.deactivate() 
model.obj2.deactivate()
model.obj3.activate()
model.eps_constraint_1.deactivate()
model.eps_constraint_2.deactivate()
results = solver.solve(model, tee=True) # solves and updates instance
Z1_3_worst= value(model.obj1)
Z2_3_worst= value(model.obj2)
Z3_3_worst= value(model.obj3)

### Solving the Optimization Problem

In [ ]:
M1_worst = max(Z1_1_worst,Z1_2_worst,Z1_3_worst)
n1_worst = min(Z1_1_worst,Z1_2_worst,Z1_3_worst)

M2_worst = max(Z2_1_worst,Z2_2_worst,Z2_3_worst)
n2_worst = min(Z2_1_worst,Z2_2_worst,Z2_3_worst)

M3_worst = max(Z3_1_worst,Z3_2_worst,Z3_3_worst)
n3_worst = min(Z3_1_worst,Z3_2_worst,Z3_3_worst)

In [ ]:
M_penalty = 1e-4
delta_1 =  M2_worst - n2_worst
delta_2 = M3_worst - n3_worst
model.obj1_aug = Objective(expr = -sum(coef_GWP[i-1]*model.s[i] for i in model.set_s) + M_penalty*(model.s_eps1/delta_1 + model.s_eps2/delta_2), sense=maximize)

In [ ]:
#Z1, Z2, and Z3 values!!! for pareto surface generation

#Best
model.obj1.deactivate()
model.obj1_aug.activate()
model.obj2.deactivate() 
model.obj3.deactivate()
model.eps_constraint_1.activate()
model.eps_constraint_2.activate()


r=100
Z1=[]
Z2=[]
Z3=[]
solution_matrix = []

for t in range(0,r):
    for i in range (0,r):
        model.epsilon1 = n2_worst + (M2_worst - n2_worst) * t / (r - 1)
        model.epsilon2 = n3_worst + (M3_worst - n3_worst) * i / (r - 1)
        results = solver.solve(model) # solves and updates instance
        
        if results.solver.termination_condition == TerminationCondition.optimal:           
            Z1.append(-value(model.obj1))
            Z2.append(-value(model.obj2))
            Z3.append(-value(model.obj3))
            
            
            scaling_factors = []
            for j in model.set_s:
                scaling_factors.append(model.s[j].value)
            solution_matrix.append(scaling_factors) 

Z1_worst.append(-Z1_2_worst)
Z2_worst.append(-Z2_2_worst)
Z3_worst.append(-Z3_2_worst)

In [ ]:
solution_matrix.append(scaling_factors_circ)

### Generating the pareto frontier numbers

In [ ]:
# Assuming Z1, Z2, and Z3 are lists of values

# Create a dictionary to store unique combinations
unique_combinations = {}

# Iterate over the lists and store unique combinations in the dictionary
for z1, z2, z3 in zip(Z1_worst, Z2_worst, Z3_worst):
    combination = (z1, z2, z3)
    if combination not in unique_combinations:
        unique_combinations[combination] = 1  # Value doesn't matter, just using the key for uniqueness

# Extract unique values from the dictionary
unique_Z1, unique_Z2, unique_Z3 = zip(*unique_combinations.keys())

In [ ]:
# Assuming Z1, Z2, and Z3 are lists of values
# Create a dictionary to store unique combinations and their indices
unique_combinations = {}
indices = []
unique_solution_matrix = []

# Iterate over the lists and store unique combinations and their indices in the dictionary
for i, (z1, z2, z3) in enumerate(zip(Z1_worst, Z2_worst, Z3_worst)):
    combination = (z1, z2, z3)
    if combination not in unique_combinations:
        unique_combinations[combination] = [i]  # Store the index as a list
    else:
        unique_combinations[combination].append(i)  # Append the index to the existing list

# Extract unique values and their corresponding indices from the dictionary
indices = list(unique_combinations.values())

# New list with only the first elements of each sublist
unique_solution_indices = [sublist[0] for sublist in indices]

unique_solution_matrix = [solution_matrix[index] for index in unique_solution_indices]

In [ ]:
pareto_GWP = unique_Z1
pareto_circularity = [(1 - (value / (2 * f_design[0]))) for value in unique_Z2]
pareto_cost = unique_Z3

#pareto front points
pareto_points = [[pareto_GWP[i], pareto_circularity[i], pareto_cost[i]] for i in range(len(pareto_GWP))]

In [ ]:
#utopia point

utopia_point_GWP = min(pareto_GWP)
scaled_utopia_point_GWP = (utopia_point_GWP - min(pareto_GWP)) / (max(pareto_GWP) - min(pareto_GWP))

utopia_point_circularity = max(pareto_circularity)
scaled_utopia_point_circularity = (utopia_point_circularity - min(pareto_circularity)) / (max(pareto_circularity) - min(pareto_circularity))

utopia_point_cost = min(pareto_cost)
scaled_utopia_point_cost = (utopia_point_cost - min(pareto_cost)) / (max(pareto_cost) - min(pareto_cost))                                                     

In [ ]:
#Compromised Solution

scaled_pareto_GWP = [(element - min(pareto_GWP)) / (max(pareto_GWP) - min(pareto_GWP)) for element in pareto_GWP]
                                                                     

scaled_pareto_circularity = [(element - min(pareto_circularity)) / (max(pareto_circularity) - min(pareto_circularity)) for element in pareto_circularity]


scaled_pareto_cost = [(element - min(pareto_cost)) / (max(pareto_cost) - min(pareto_cost)) for element in pareto_cost]


# Calculate the distance to the utopia point
distances = []
for i in range(len(scaled_pareto_GWP)):
    distances.append(
        np.sqrt((scaled_pareto_GWP[i] - scaled_utopia_point_GWP) ** 2  
        + (scaled_pareto_circularity[i] - scaled_utopia_point_circularity) ** 2)
        + (scaled_pareto_cost[i] - scaled_utopia_point_cost) ** 2)
index_min_distance = np.argmin(distances)

In [ ]:
#Solutions' scaling factors
index_min_GWP_solution = np.argmin(pareto_GWP)
index_max_circularity_solution = np.argmax(pareto_circularity)
index_min_cost_solution = np.argmin(pareto_cost)
index_compromised_solution = index_min_distance

min_GWP_solution = unique_solution_matrix[index_min_GWP_solution]
max_circularity_solution = unique_solution_matrix[index_max_circularity_solution]
min_cost_solution = unique_solution_matrix[index_min_cost_solution]
compromised_solution = unique_solution_matrix[index_compromised_solution]

# 2. Optimization with resilience constraint

## 2.1. Best solution

In [ ]:
#Uncertainty scenario
scen = 0

if scen ==0:
    #Best case values
    masc_solv_draws = 0.01
    masc_ener_draws = 0.0003694444
    masc_eff_draws = 0.95
    masc_cycles = 100

    strap_solv_draws = 0.002
    strap_ener_draws = 0.00001555556
    strap_eff_draws = 0.95
    strap_cycles = 100
    
elif scen ==1:
    # Expected values
    masc_solv_draws = 0.1
    masc_ener_draws = 0.0007388887
    masc_eff_draws = 0.875
    masc_cycles = 5

    strap_solv_draws = 0.1
    strap_ener_draws = 0.000023472225
    strap_eff_draws = 0.875
    strap_cycles = 5

elif scen==2:
    #Worst case values
    masc_solv_draws = 0.2
    masc_ener_draws = 0.001108333
    masc_eff_draws = 0.80
    masc_cycles = 1

    strap_solv_draws = 0.2
    strap_ener_draws = 0.000023472225
    strap_eff_draws = 0.80
    strap_cycles = 1
    
#TEA
masc_des = masc_design.loc[(masc_design.Index.isin(['Electricity','Diacetone alcohol']))  | (masc_design.Variable == 'Output efficiency')]
strap_des = strap_design.loc[(strap_design.Index.isin(['Electricity','Ethylene glycol']))  | (strap_design.Variable == 'Output efficiency')]


A_design_scenario = []
masc_unc_costs = []
strap_unc_costs = []
masc_unc_waste = []
strap_unc_waste = []



A_design_scenario = A_design.copy()

## MASC ##

A_design_scenario [A_df_design.index[A_df_design.iloc[:, 1] == 'Diacetone Alcohol'].tolist()[0] - 10,
                 np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'MASC'].tolist()[0] - 10] = - 5.56 * (masc_solv_draws)*(1/masc_eff_draws)*calculate_sum_mi_masc(masc_eff_draws, masc_cycles)

A_design_scenario [A_df_design.index[A_df_design.iloc[:, 0] == 'Electricity, at Grid, US, 2010'].tolist()[0] - 10,
                 np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'MASC'].tolist()[0] - 10] = - 5.56 * 2.7 * (1/masc_eff_draws)*calculate_sum_mi_masc(masc_eff_draws, masc_cycles)

A_design_scenario [A_df_design.index[A_df_design.iloc[:, 1] == 'Ultimate EoL waste'].tolist()[0] - 10,
                 np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'MASC'].tolist()[0] - 10] = ((1-masc_eff_draws)/masc_eff_draws)*calculate_sum_mi_masc(masc_eff_draws, masc_cycles) + calculate_mi_masc(masc_eff_draws,masc_cycles,masc_cycles)

A_design_scenario [A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film'].tolist()[0] - 10,
                 np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'MASC'].tolist()[0] - 10] = 1 - (calculate_mp_masc(masc_eff_draws, masc_cycles))
## STRAP ##

A_design_scenario [A_df_design.index[A_df_design.iloc[:, 0] == 'Electricity, at Grid, US, 2010'].tolist()[0] - 10,
                 np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'STRAP'].tolist()[0] - 10] = - (1/0.9) * 0.648 * (1/strap_eff_draws) * calculate_sum_mi_strap(strap_eff_draws, strap_cycles)

A_design_scenario [A_df_design.index[A_df_design.iloc[:, 1] == 'Ethylene glycol, at plant, kg'].tolist()[0] - 10,
                 np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'STRAP'].tolist()[0] - 10] = - 11.5 * (1/0.9) * (strap_solv_draws)*(1/strap_eff_draws)*calculate_sum_mi_strap(strap_eff_draws, strap_cycles)

A_design_scenario [A_df_design.index[A_df_design.iloc[:, 1] == 'heat, from natural gas'].tolist()[0] - 10,
                 np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'STRAP'].tolist()[0] - 10] = - (1/0.9) * 0.00351 * (1/strap_eff_draws) * calculate_sum_mi_strap(strap_eff_draws, strap_cycles) 

A_design_scenario [A_df_design.index[A_df_design.iloc[:, 1] == 'Ultimate EoL waste'].tolist()[0] - 10,
                 np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'STRAP'].tolist()[0] - 10] = ((1-strap_eff_draws)/strap_eff_draws)*(1/0.9)*calculate_sum_mi_strap(strap_eff_draws, strap_cycles) + calculate_mi_strap(strap_eff_draws,strap_cycles,strap_cycles)

A_design_scenario [A_df_design.index[A_df_design.iloc[:, 1] == 'POLYAMIDE 6 (Nylon 6) (at plant)'].tolist()[0] - 10,
                 np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'STRAP'].tolist()[0] - 10] = 0.26/0.9 - (0.26/0.9)*calculate_mp_strap(strap_eff_draws, strap_cycles)

A_design_scenario [A_df_design.index[A_df_design.iloc[:, 1] == 'Polyethylene, linear low-density, LLDPE, virgin resin, at plant'].tolist()[0] - 10,
                 np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'STRAP'].tolist()[0] - 10] = 0.74/0.9 - (0.74/0.9)*calculate_mp_strap(strap_eff_draws, strap_cycles)



masc_des.loc[masc_des.Index == 'Electricity','Value'] = 2521.974 * masc_ener_draws
masc_des.loc[masc_des.Index == 'Diacetone alcohol','Value'] = 5560 * masc_solv_draws
masc_des.loc[masc_des.Variable == 'Output efficiency','Value'] = masc_eff_draws

_masc = Scenario(
    func_unit = 2.204,
    sens_df = masc_des,
    eol_sc = {'Mechanical and Solvent Cleaning': [1.0, masc_cycles]},
    products = {'Mechanical and Solvent Cleaning': 'Barrier film'}
)


strap_des.loc[strap_des.Index == 'Electricity', 'Value'] = 11400 * strap_ener_draws
strap_des.loc[strap_des.Index == 'Ethylene glycol', 'Value'] = 25132.698 * strap_solv_draws
strap_des.loc[strap_des.Variable == 'Output efficiency', 'Value'] = strap_eff_draws

_strap = Scenario(
    func_unit = 2.204,
    sens_df = strap_des,
    eol_sc = {'Solvent Treatment and Precipitation': [1.0, strap_cycles]},
    products = {'Solvent Treatment and Precipitation':'Polyethylene'}
)

masc_waste = ((1-masc_eff_draws)/masc_eff_draws)*calculate_sum_mi_masc(masc_eff_draws, masc_cycles) + calculate_mi_masc(masc_eff_draws,masc_cycles,masc_cycles)
strap_waste = ((1-strap_eff_draws)/strap_eff_draws)*(1/0.9)*calculate_sum_mi_strap(strap_eff_draws, strap_cycles) + calculate_mi_strap(strap_eff_draws,strap_cycles,strap_cycles)

### Multi-objective optimization formulation

In [ ]:
#Model Formulation
# Create the model
model = ConcreteModel()

res = 0

model.s_eps1 = Var(within=NonNegativeReals, bounds=(1e-6, None))  # Slack for epsilon constraint 1
model.s_eps2 = Var(within=NonNegativeReals, bounds=(1e-6, None))  # Slack for epsilon constraint 2


#epsilon parameter
model.epsilon1=Param(initialize=1000,mutable=True)
model.epsilon2=Param(initialize=1000,mutable=True)

# Define the decision variable
model.set_s = RangeSet(len(np.transpose(A_design)))
model.s = Var(model.set_s)
for i in model.set_s:
    model.s[i].set_value(1.0)  # Adjust based on the problem's context
# List of processes with negative scaling factor due to the substitution approach
negative_s_indices = []
positive_s_indices = []
all_s_indices = []
search_elements = [
    'Substitued Electricity US Market',
    'Substitued Polypropylene Market',
    'Substitued Refinery Oil Market'
]

search_elements_zero = [
    #'pyrolysis',
    #'STRAP',
    #'MASC',
    #'landfill (film)',
    #'downcycling (allocation)-s1 (heat pretreatment)',
    #'MASC',
    #'incineration (film)'
]

# Search for elements in the first row of the DataFrame
negative_s_indices = [A_df_design.iloc[0,:].tolist().index(elem) for elem in search_elements]
negative_s_indices = [i-9 for i in negative_s_indices]
all_s_indices = list(range(1, len(np.transpose(A_design))+1))
positive_s_indices = [index for index in all_s_indices if index not in negative_s_indices]
zero_s_indices = [A_df_design.iloc[0,:].tolist().index(elem) for elem in search_elements_zero]
zero_s_indices = [i-9 for i in zero_s_indices]

model.set_negative_scale = Set(initialize=negative_s_indices)
model.set_positive_scale = Set(initialize=positive_s_indices)
model.set_zero_scale = Set(initialize=zero_s_indices)

#model constraint: As=f
model.set_balance = RangeSet(len(f_design))
def cons_rule(model, p):
    return sum(A_design_scenario[p-1,i-1]*model.s[i] for i in model.set_s) == f_design[p-1]
def negative_scale(model, i):
    return (model.s[i]<=0)
def positive_scale(model, i):
    return (model.s[i]>=0)
def zero_scale(model, i):
    return (model.s[i]==0)

##### Resilience Metric
# Define indices
num_rows = 10  # Number of rows/columns in R
indices = range(num_rows)
# Define epsilon for numerical stability
epsilon = 1e-10

def R_rule(model, i, j):
    if i == 0 and j == 1:
        return abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film'].tolist()[0] - 10,
                            A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'multilayer film production'].tolist()[0] - 10]) \
               * abs(model.s[A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'multilayer film production'].tolist()[0] - 9])
    elif i == 0 and j == 9:
        return abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'Ultimate EoL waste'].tolist()[0] - 10,
                            A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'multilayer film production'].tolist()[0] - 10]) \
               * abs(model.s[A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'multilayer film production'].tolist()[0] - 9])
    elif i == 1 and j == 2:
        return abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                            A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'landfill (film)'].tolist()[0] - 10]) \
               * abs(model.s[A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'landfill (film)'].tolist()[0] - 9])
    elif i == 1 and j == 3:
        return abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                            A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'incineration (film)'].tolist()[0] - 10]) \
               * abs(model.s[A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'incineration (film)'].tolist()[0] - 9])
    elif i == 1 and j == 4:
        return abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                            A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'downcycling (allocation)-s1 (heat pretreatment)'].tolist()[0] - 10]) \
               * abs(model.s[A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'downcycling (allocation)-s1 (heat pretreatment)'].tolist()[0] - 9])
    elif i == 1 and j == 5:
        return abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                            A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'downcycling (allocation)-s2 (1/3 discard)'].tolist()[0] - 10]) \
               * abs(model.s[A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'downcycling (allocation)-s2 (1/3 discard)'].tolist()[0] - 9])
    elif i == 1 and j == 6:
        return abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                            A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'STRAP'].tolist()[0] - 10]) \
               * abs(model.s[A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'STRAP'].tolist()[0] - 9])
    elif i == 1 and j == 7:
        return abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                            A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'MASC'].tolist()[0] - 10]) \
               * abs(model.s[A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'MASC'].tolist()[0] - 9])
    elif i == 1 and j == 8:
        return abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                            A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'pyrolysis'].tolist()[0] - 10]) \
               * abs(model.s[A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'pyrolysis'].tolist()[0] - 9])
    elif i == 2 and j == 9:
        return abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                   A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'landfill (film)'].tolist()[0] - 10]) \
           * abs(model.s[A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'landfill (film)'].tolist()[0] - 9])
    elif i == 3 and j == 9:
        return abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                   A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'incineration (film)'].tolist()[0] - 10]) \
           * abs(model.s[A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'incineration (film)'].tolist()[0] - 9])
    elif i == 4 and j == 9:
        return abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                   A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'downcycling (allocation)-s1 (heat pretreatment)'].tolist()[0] - 10]) \
           * abs(model.s[A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'downcycling (allocation)-s1 (heat pretreatment)'].tolist()[0] - 9])
    elif i == 5 and j == 9:
        return abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                   A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'downcycling (allocation)-s2 (1/3 discard)'].tolist()[0] - 10]) \
           * abs(model.s[A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'downcycling (allocation)-s2 (1/3 discard)'].tolist()[0] - 9])
    elif i == 6 and j == 0:
        return abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'Polyethylene, linear low-density, LLDPE, virgin resin, at plant'].tolist()[0] - 10,
                   A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'STRAP'].tolist()[0] - 10]) \
           * abs(model.s[A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'STRAP'].tolist()[0] - 9]) \
     +abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'POLYAMIDE 6 (Nylon 6) (at plant)'].tolist()[0] - 10,
                   A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'STRAP'].tolist()[0] - 10]) \
           * abs(model.s[A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'STRAP'].tolist()[0] - 9])
    elif i == 6 and j == 9:
        return abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'Ultimate EoL waste'].tolist()[0] - 10,
                   A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'STRAP'].tolist()[0] - 10]) \
           * abs(model.s[A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'STRAP'].tolist()[0] - 9])
    elif i == 7 and j == 1:
        return abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film'].tolist()[0] - 10,
                   A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'MASC'].tolist()[0] - 10]) \
           * abs(model.s[A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'MASC'].tolist()[0] - 9])
    elif i == 7 and j == 9:
        return abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'Ultimate EoL waste'].tolist()[0] - 10,
                   A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'MASC'].tolist()[0] - 10]) \
           * abs(model.s[A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'MASC'].tolist()[0] - 9])
    elif i == 8 and j == 9:
        return abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                   A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'pyrolysis'].tolist()[0] - 10]) \
           * abs(model.s[A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'pyrolysis'].tolist()[0] - 9])
    elif i == 9 and j == 0:
        return abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'Polyethylene, linear low-density, LLDPE, virgin resin, at plant'].tolist()[0] - 10,
                            A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'Polyethylene, linear low-density, LLDPE, virgin resin, at plant'].tolist()[0] - 10]) \
               * abs(model.s[A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'Polyethylene, linear low-density, LLDPE, virgin resin, at plant'].tolist()[0] - 9]) \
               + abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'POLYAMIDE 6 (Nylon 6) (at plant)'].tolist()[0] - 10,
                          A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'POLYAMIDE 6 (Nylon 6)'].tolist()[0] - 10]) \
               * abs(model.s[A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'POLYAMIDE 6 (Nylon 6)'].tolist()[0] - 9])
    else:
        return 0

model.R = Expression(indices, indices, rule=R_rule)

# Define the total sum of R
model.R_sum = Expression(expr=sum(model.R[i, j] for i in indices for j in indices))

# Define Development Capacity (DC)
def DC_rule(model):
    return sum(
        -model.R[i, j] * log((model.R[i, j] / model.R_sum) + epsilon) 
        for i in indices for j in indices if i != j
    )
model.DC = Expression(rule=DC_rule)

# Define Ascendency (ASC)
def ASC_rule(model):
    return sum(
        model.R[i, j] * log(
            (model.R[i, j] * model.R_sum) / 
            ((sum(model.R[i, k] for k in indices) + epsilon) * 
             (sum(model.R[k, j] for k in indices) + epsilon)) + epsilon
        ) 
        for i in indices for j in indices if i != j
    )
model.ASC = Expression(rule=ASC_rule)

# Define R_ASC
model.R_ASC = Expression(expr=model.ASC / (model.DC+ epsilon))


model.balance_constraints = Constraint(model.set_balance, rule=cons_rule)
model.negative_scale_constraints = Constraint(model.set_negative_scale, rule=negative_scale)
model.positive_scale_constraints = Constraint(model.set_positive_scale, rule=positive_scale)
model.zero_scale_constraints = Constraint(model.set_zero_scale, rule=zero_scale)
if res==1:  
    model.R_ASC_lower_bound = Constraint(expr=0.213 <= model.R_ASC)
    model.R_ASC_upper_bound = Constraint(expr=model.R_ASC <= 0.589)


model.eps_constraint_1 = Constraint(expr = -(abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'Polyethylene, linear low-density, LLDPE, virgin resin, at plant'].tolist()[0] - 10,
                     np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'Polyethylene, linear low-density, LLDPE, virgin resin, at plant'].tolist()[0] - 10])
    *model.s[np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'Polyethylene, linear low-density, LLDPE, virgin resin, at plant'].tolist()[0] - 9]
        
        + abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'POLYAMIDE 6 (Nylon 6) (at plant)'].tolist()[0] - 10,
                     np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'POLYAMIDE 6 (Nylon 6)'].tolist()[0] - 10])
    *model.s[np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'POLYAMIDE 6 (Nylon 6)'].tolist()[0] - 9]
        
        + (abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                     np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'landfill (film)'].tolist()[0] - 10])
    *model.s[np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'landfill (film)'].tolist()[0] - 9])*(1-0)

                                  + (abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                     np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'incineration (film)'].tolist()[0] - 10])
    *model.s[np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'incineration (film)'].tolist()[0] - 9])*(1-eta_2*gama_2)

                                       + (abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                     np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'pyrolysis'].tolist()[0] - 10])
    *model.s[np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'pyrolysis'].tolist()[0] - 9])*(1-eta_3*gama_3) 

                                  + (abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                     np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'STRAP'].tolist()[0] - 10])
    *model.s[np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'STRAP'].tolist()[0] - 9])*strap_waste

                                  + (abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                     np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'downcycling (allocation)-s1 (heat pretreatment)'].tolist()[0] - 10])
    *model.s[np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'downcycling (allocation)-s1 (heat pretreatment)'].tolist()[0] - 9])*(1-eta_5_s1*gama_5) 

                                     +(abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                     np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'downcycling (allocation)-s2 (1/3 discard)'].tolist()[0] - 10])
    *model.s[np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'downcycling (allocation)-s2 (1/3 discard)'].tolist()[0] - 9])*(1-eta_5_s2*gama_5) 

                                  + (abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                     np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'MASC'].tolist()[0] - 10])
    *model.s[np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'MASC'].tolist()[0] - 9])*masc_waste) - model.s_eps1 == model.epsilon1)


model.eps_constraint_2 = Constraint(expr = -((abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                         np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'landfill (film)'].tolist()[0] - 10])
        *model.s[np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'landfill (film)'].tolist()[0] - 9])
                                      *landf.total_cost

                                      + (abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                         np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'incineration (film)'].tolist()[0] - 10])
        *model.s[np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'incineration (film)'].tolist()[0] - 9])
                                      *landf.total_cost

                                           + (abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                         np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'pyrolysis'].tolist()[0] - 10])
        *model.s[np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'pyrolysis'].tolist()[0] - 9])
                                      *pyrol.total_cost

                                      + (abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                         np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'STRAP'].tolist()[0] - 10])
        *model.s[np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'STRAP'].tolist()[0] - 9])
                                      *_strap.total_cost

                                      + (abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                         np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'downcycling (allocation)-s1 (heat pretreatment)'].tolist()[0] - 10])
        *model.s[np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'downcycling (allocation)-s1 (heat pretreatment)'].tolist()[0] - 9])
                                      *pellet_withheat.total_cost

                                         +(abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                         np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'downcycling (allocation)-s2 (1/3 discard)'].tolist()[0] - 10])
        *model.s[np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'downcycling (allocation)-s2 (1/3 discard)'].tolist()[0] - 9])
                                      *pellet_noheat.total_cost

                                      + (abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                         np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'MASC'].tolist()[0] - 10])
        *model.s[np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'MASC'].tolist()[0] - 9])
                                      *_masc.total_cost) - model.s_eps2 == model.epsilon2)



model.obj1 = Objective(expr = -sum(coef_GWP[i-1]*model.s[i] for i in model.set_s), sense=maximize)

model.obj2 = Objective(expr = -(abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'Polyethylene, linear low-density, LLDPE, virgin resin, at plant'].tolist()[0] - 10,
                     np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'Polyethylene, linear low-density, LLDPE, virgin resin, at plant'].tolist()[0] - 10])
    *model.s[np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'Polyethylene, linear low-density, LLDPE, virgin resin, at plant'].tolist()[0] - 9]
        
        + abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'POLYAMIDE 6 (Nylon 6) (at plant)'].tolist()[0] - 10,
                     np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'POLYAMIDE 6 (Nylon 6)'].tolist()[0] - 10])
    *model.s[np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'POLYAMIDE 6 (Nylon 6)'].tolist()[0] - 9]
        
        + (abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                     np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'landfill (film)'].tolist()[0] - 10])
    *model.s[np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'landfill (film)'].tolist()[0] - 9])*(1-0)

                                  + (abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                     np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'incineration (film)'].tolist()[0] - 10])
    *model.s[np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'incineration (film)'].tolist()[0] - 9])*(1-eta_2*gama_2)

                                       + (abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                     np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'pyrolysis'].tolist()[0] - 10])
    *model.s[np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'pyrolysis'].tolist()[0] - 9])*(1-eta_3*gama_3) 

                                  + (abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                     np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'STRAP'].tolist()[0] - 10])
    *model.s[np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'STRAP'].tolist()[0] - 9])*strap_waste

                                  + (abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                     np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'downcycling (allocation)-s1 (heat pretreatment)'].tolist()[0] - 10])
    *model.s[np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'downcycling (allocation)-s1 (heat pretreatment)'].tolist()[0] - 9])*(1-eta_5_s1*gama_5) 

                                     +(abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                     np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'downcycling (allocation)-s2 (1/3 discard)'].tolist()[0] - 10])
    *model.s[np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'downcycling (allocation)-s2 (1/3 discard)'].tolist()[0] - 9])*(1-eta_5_s2*gama_5) 

                                  + (abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                     np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'MASC'].tolist()[0] - 10])
    *model.s[np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'MASC'].tolist()[0] - 9])*masc_waste), sense=maximize
            )

model.obj3 = Objective(expr = -((abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                         np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'landfill (film)'].tolist()[0] - 10])
        *model.s[np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'landfill (film)'].tolist()[0] - 9])
                                      *landf.total_cost

                                      + (abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                         np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'incineration (film)'].tolist()[0] - 10])
        *model.s[np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'incineration (film)'].tolist()[0] - 9])
                                      *landf.total_cost

                                           + (abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                         np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'pyrolysis'].tolist()[0] - 10])
        *model.s[np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'pyrolysis'].tolist()[0] - 9])
                                      *pyrol.total_cost

                                      + (abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                         np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'STRAP'].tolist()[0] - 10])
        *model.s[np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'STRAP'].tolist()[0] - 9])
                                      *_strap.total_cost

                                      + (abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                         np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'downcycling (allocation)-s1 (heat pretreatment)'].tolist()[0] - 10])
        *model.s[np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'downcycling (allocation)-s1 (heat pretreatment)'].tolist()[0] - 9])
                                      *pellet_withheat.total_cost

                                         +(abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                         np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'downcycling (allocation)-s2 (1/3 discard)'].tolist()[0] - 10])
        *model.s[np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'downcycling (allocation)-s2 (1/3 discard)'].tolist()[0] - 9])
                                      *pellet_noheat.total_cost

                                      + (abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                         np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'MASC'].tolist()[0] - 10])
        *model.s[np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'MASC'].tolist()[0] - 9])
                                      *_masc.total_cost), sense=maximize
         )

#Solver
if res==0:
    solver = SolverFactory('gurobi')
elif res==1:
    solver = SolverFactory('ipopt')

### Epsilon bonds

In [ ]:
model.obj1.activate() 
model.obj2.deactivate()
model.obj3.deactivate()
model.eps_constraint_1.deactivate()
model.eps_constraint_2.deactivate()
results = solver.solve(model, tee=True) # solves and updates instance
Z1_1_best=value(model.obj1)
Z2_1_best=value(model.obj2)
Z3_1_best=value(model.obj3)

In [ ]:
model.obj1.deactivate() 
model.obj2.activate() 
model.obj3.deactivate()
model.eps_constraint_1.deactivate()
model.eps_constraint_2.deactivate()
results = solver.solve(model, tee=True) # solves and updates instance
Z1_2_best= value(model.obj1)
Z2_2_best= value(model.obj2)
Z3_2_best= value(model.obj3)
scaling_factors_circ = []
for j in model.set_s:
    scaling_factors_circ.append(model.s[j].value)

In [ ]:
model.obj1.deactivate() 
model.obj2.deactivate()
model.obj3.activate()
model.eps_constraint_1.deactivate()
model.eps_constraint_2.deactivate()
results = solver.solve(model, tee=True) # solves and updates instance
Z1_3_best= value(model.obj1)
Z2_3_best= value(model.obj2)
Z3_3_best= value(model.obj3)

### Solving the Optimization Problem

In [ ]:
M1_best = max(Z1_1_best,Z1_2_best,Z1_3_best)
n1_best = min(Z1_1_best,Z1_2_best,Z1_3_best)

M2_best = max(Z2_1_best,Z2_2_best,Z2_3_best)
n2_best = min(Z2_1_best,Z2_2_best,Z2_3_best)

M3_best = max(Z3_1_best,Z3_2_best,Z3_3_best)
n3_best = min(Z3_1_best,Z3_2_best,Z3_3_best)

In [ ]:
M_penalty = 1e-4
delta_1 =  M2_best - n2_best
delta_2 = M3_best - n3_best
model.obj1_aug = Objective(expr = -sum(coef_GWP[i-1]*model.s[i] for i in model.set_s) + M_penalty*(model.s_eps1/delta_1 + model.s_eps2/delta_2), sense=maximize)

In [ ]:
#Z1, Z2, and Z3 values!!! for pareto surface generation

#Best
model.obj1.deactivate()
model.obj1_aug.activate()
model.obj2.deactivate() 
model.obj3.deactivate()
model.eps_constraint_1.activate()
model.eps_constraint_2.activate()


r=100
Z1=[]
Z2=[]
Z3=[]
solution_matrix = []

for t in range(0,r):
    for i in range (0,r):
        model.epsilon1 = n2_best + (M2_best - n2_best) * t / (r - 1)
        model.epsilon2 = n3_best + (M3_best - n3_best) * i / (r - 1)
        results = solver.solve(model) # solves and updates instance
        
        if results.solver.termination_condition == TerminationCondition.optimal:           
            Z1.append(-value(model.obj1))
            Z2.append(-value(model.obj2))
            Z3.append(-value(model.obj3))
            
            
            scaling_factors = []
            for j in model.set_s:
                scaling_factors.append(model.s[j].value)
            solution_matrix.append(scaling_factors) 

Z1.append(-Z1_2_best)
Z2.append(-Z2_2_best)
Z3.append(-Z3_2_best)

In [ ]:
solution_matrix.append(scaling_factors_circ)

### Generating the pareto frontier numbers

In [ ]:
# Assuming Z1, Z2, and Z3 are lists of values

# Create a dictionary to store unique combinations
unique_combinations = {}

# Iterate over the lists and store unique combinations in the dictionary
for z1, z2, z3 in zip(Z1, Z2, Z3):
    combination = (z1, z2, z3)
    if combination not in unique_combinations:
        unique_combinations[combination] = 1  # Value doesn't matter, just using the key for uniqueness

# Extract unique values from the dictionary
unique_Z1, unique_Z2, unique_Z3 = zip(*unique_combinations.keys())

In [ ]:
# Assuming Z1, Z2, and Z3 are lists of values
# Create a dictionary to store unique combinations and their indices
unique_combinations = {}
indices = []
unique_solution_matrix = []

# Iterate over the lists and store unique combinations and their indices in the dictionary
for i, (z1, z2, z3) in enumerate(zip(Z1, Z2, Z3)):
    combination = (z1, z2, z3)
    if combination not in unique_combinations:
        unique_combinations[combination] = [i]  # Store the index as a list
    else:
        unique_combinations[combination].append(i)  # Append the index to the existing list

# Extract unique values and their corresponding indices from the dictionary
indices = list(unique_combinations.values())

# New list with only the first elements of each sublist
unique_solution_indices = [sublist[0] for sublist in indices]

unique_solution_matrix = [solution_matrix[index] for index in unique_solution_indices]

In [ ]:
pareto_GWP = unique_Z1
pareto_circularity = [(1 - (value / (2 * f_design[0]))) for value in unique_Z2]
pareto_cost = unique_Z3

#pareto front points
pareto_points = [[pareto_GWP[i], pareto_circularity[i], pareto_cost[i]] for i in range(len(pareto_GWP))]

In [ ]:
#utopia point

utopia_point_GWP = min(pareto_GWP)
scaled_utopia_point_GWP = (utopia_point_GWP - min(pareto_GWP)) / (max(pareto_GWP) - min(pareto_GWP))

utopia_point_circularity = max(pareto_circularity)
scaled_utopia_point_circularity = (utopia_point_circularity - min(pareto_circularity)) / (max(pareto_circularity) - min(pareto_circularity))

utopia_point_cost = min(pareto_cost)
scaled_utopia_point_cost = (utopia_point_cost - min(pareto_cost)) / (max(pareto_cost) - min(pareto_cost))                                                     

In [ ]:
#Compromised Solution

scaled_pareto_GWP = [(element - min(pareto_GWP)) / (max(pareto_GWP) - min(pareto_GWP)) for element in pareto_GWP]
                                                                     

scaled_pareto_circularity = [(element - min(pareto_circularity)) / (max(pareto_circularity) - min(pareto_circularity)) for element in pareto_circularity]


scaled_pareto_cost = [(element - min(pareto_cost)) / (max(pareto_cost) - min(pareto_cost)) for element in pareto_cost]


# Calculate the distance to the utopia point
distances = []
for i in range(len(scaled_pareto_GWP)):
    distances.append(
        np.sqrt((scaled_pareto_GWP[i] - scaled_utopia_point_GWP) ** 2  
        + (scaled_pareto_circularity[i] - scaled_utopia_point_circularity) ** 2)
        + (scaled_pareto_cost[i] - scaled_utopia_point_cost) ** 2)
index_min_distance = np.argmin(distances)

In [ ]:
#Solutions' scaling factors
index_min_GWP_solution = np.argmin(pareto_GWP)
index_max_circularity_solution = np.argmax(pareto_circularity)
index_min_cost_solution = np.argmin(pareto_cost)
index_compromised_solution = index_min_distance

min_GWP_solution = unique_solution_matrix[index_min_GWP_solution]
max_circularity_solution = unique_solution_matrix[index_max_circularity_solution]
min_cost_solution = unique_solution_matrix[index_min_cost_solution]
compromised_solution = unique_solution_matrix[index_compromised_solution]

## 2.2. Median solution

In [ ]:
#Uncertainty scenario
scen = 1

if scen ==0:
    #Best case values
    masc_solv_draws = 0.01
    masc_ener_draws = 0.0003694444
    masc_eff_draws = 0.95
    masc_cycles = 100

    strap_solv_draws = 0.002
    strap_ener_draws = 0.00001555556
    strap_eff_draws = 0.95
    strap_cycles = 100
    
elif scen ==1:
    # Expected values
    masc_solv_draws = 0.1
    masc_ener_draws = 0.0007388887
    masc_eff_draws = 0.875
    masc_cycles = 5

    strap_solv_draws = 0.1
    strap_ener_draws = 0.000023472225
    strap_eff_draws = 0.875
    strap_cycles = 5

elif scen==2:
    #Worst case values
    masc_solv_draws = 0.2
    masc_ener_draws = 0.001108333
    masc_eff_draws = 0.80
    masc_cycles = 1

    strap_solv_draws = 0.2
    strap_ener_draws = 0.000023472225
    strap_eff_draws = 0.80
    strap_cycles = 1
    
#TEA
masc_des = masc_design.loc[(masc_design.Index.isin(['Electricity','Diacetone alcohol']))  | (masc_design.Variable == 'Output efficiency')]
strap_des = strap_design.loc[(strap_design.Index.isin(['Electricity','Ethylene glycol']))  | (strap_design.Variable == 'Output efficiency')]


A_design_scenario = []
masc_unc_costs = []
strap_unc_costs = []
masc_unc_waste = []
strap_unc_waste = []



A_design_scenario = A_design.copy()

## MASC ##

A_design_scenario [A_df_design.index[A_df_design.iloc[:, 1] == 'Diacetone Alcohol'].tolist()[0] - 10,
                 np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'MASC'].tolist()[0] - 10] = - 5.56 * (masc_solv_draws)*(1/masc_eff_draws)*calculate_sum_mi_masc(masc_eff_draws, masc_cycles)

A_design_scenario [A_df_design.index[A_df_design.iloc[:, 0] == 'Electricity, at Grid, US, 2010'].tolist()[0] - 10,
                 np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'MASC'].tolist()[0] - 10] = - 5.56 * 2.7 * (1/masc_eff_draws)*calculate_sum_mi_masc(masc_eff_draws, masc_cycles)

A_design_scenario [A_df_design.index[A_df_design.iloc[:, 1] == 'Ultimate EoL waste'].tolist()[0] - 10,
                 np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'MASC'].tolist()[0] - 10] = ((1-masc_eff_draws)/masc_eff_draws)*calculate_sum_mi_masc(masc_eff_draws, masc_cycles) + calculate_mi_masc(masc_eff_draws,masc_cycles,masc_cycles)

A_design_scenario [A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film'].tolist()[0] - 10,
                 np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'MASC'].tolist()[0] - 10] = 1 - (calculate_mp_masc(masc_eff_draws, masc_cycles))
## STRAP ##

A_design_scenario [A_df_design.index[A_df_design.iloc[:, 0] == 'Electricity, at Grid, US, 2010'].tolist()[0] - 10,
                 np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'STRAP'].tolist()[0] - 10] = - (1/0.9) * 0.648 * (1/strap_eff_draws) * calculate_sum_mi_strap(strap_eff_draws, strap_cycles)

A_design_scenario [A_df_design.index[A_df_design.iloc[:, 1] == 'Ethylene glycol, at plant, kg'].tolist()[0] - 10,
                 np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'STRAP'].tolist()[0] - 10] = - 11.5 * (1/0.9) * (strap_solv_draws)*(1/strap_eff_draws)*calculate_sum_mi_strap(strap_eff_draws, strap_cycles)

A_design_scenario [A_df_design.index[A_df_design.iloc[:, 1] == 'heat, from natural gas'].tolist()[0] - 10,
                 np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'STRAP'].tolist()[0] - 10] = - (1/0.9) * 0.00351 * (1/strap_eff_draws) * calculate_sum_mi_strap(strap_eff_draws, strap_cycles) 

A_design_scenario [A_df_design.index[A_df_design.iloc[:, 1] == 'Ultimate EoL waste'].tolist()[0] - 10,
                 np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'STRAP'].tolist()[0] - 10] = ((1-strap_eff_draws)/strap_eff_draws)*(1/0.9)*calculate_sum_mi_strap(strap_eff_draws, strap_cycles) + calculate_mi_strap(strap_eff_draws,strap_cycles,strap_cycles)

A_design_scenario [A_df_design.index[A_df_design.iloc[:, 1] == 'POLYAMIDE 6 (Nylon 6) (at plant)'].tolist()[0] - 10,
                 np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'STRAP'].tolist()[0] - 10] = 0.26/0.9 - (0.26/0.9)*calculate_mp_strap(strap_eff_draws, strap_cycles)

A_design_scenario [A_df_design.index[A_df_design.iloc[:, 1] == 'Polyethylene, linear low-density, LLDPE, virgin resin, at plant'].tolist()[0] - 10,
                 np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'STRAP'].tolist()[0] - 10] = 0.74/0.9 - (0.74/0.9)*calculate_mp_strap(strap_eff_draws, strap_cycles)



masc_des.loc[masc_des.Index == 'Electricity','Value'] = 2521.974 * masc_ener_draws
masc_des.loc[masc_des.Index == 'Diacetone alcohol','Value'] = 5560 * masc_solv_draws
masc_des.loc[masc_des.Variable == 'Output efficiency','Value'] = masc_eff_draws

_masc = Scenario(
    func_unit = 2.204,
    sens_df = masc_des,
    eol_sc = {'Mechanical and Solvent Cleaning': [1.0, masc_cycles]},
    products = {'Mechanical and Solvent Cleaning': 'Barrier film'}
)


strap_des.loc[strap_des.Index == 'Electricity', 'Value'] = 11400 * strap_ener_draws
strap_des.loc[strap_des.Index == 'Ethylene glycol', 'Value'] = 25132.698 * strap_solv_draws
strap_des.loc[strap_des.Variable == 'Output efficiency', 'Value'] = strap_eff_draws

_strap = Scenario(
    func_unit = 2.204,
    sens_df = strap_des,
    eol_sc = {'Solvent Treatment and Precipitation': [1.0, strap_cycles]},
    products = {'Solvent Treatment and Precipitation':'Polyethylene'}
)

masc_waste = ((1-masc_eff_draws)/masc_eff_draws)*calculate_sum_mi_masc(masc_eff_draws, masc_cycles) + calculate_mi_masc(masc_eff_draws,masc_cycles,masc_cycles)
strap_waste = ((1-strap_eff_draws)/strap_eff_draws)*(1/0.9)*calculate_sum_mi_strap(strap_eff_draws, strap_cycles) + calculate_mi_strap(strap_eff_draws,strap_cycles,strap_cycles)

### Multi-objective optimization formulation

In [ ]:
#Model Formulation
# Create the model
model = ConcreteModel()

res = 0

model.s_eps1 = Var(within=NonNegativeReals, bounds=(1e-6, None))  # Slack for epsilon constraint 1
model.s_eps2 = Var(within=NonNegativeReals, bounds=(1e-6, None))  # Slack for epsilon constraint 2


#epsilon parameter
model.epsilon1=Param(initialize=1000,mutable=True)
model.epsilon2=Param(initialize=1000,mutable=True)

# Define the decision variable
model.set_s = RangeSet(len(np.transpose(A_design)))
model.s = Var(model.set_s)
for i in model.set_s:
    model.s[i].set_value(1.0)  # Adjust based on the problem's context
# List of processes with negative scaling factor due to the substitution approach
negative_s_indices = []
positive_s_indices = []
all_s_indices = []
search_elements = [
    'Substitued Electricity US Market',
    'Substitued Polypropylene Market',
    'Substitued Refinery Oil Market'
]

search_elements_zero = [
    #'pyrolysis',
    #'STRAP',
    #'MASC',
    #'landfill (film)',
    #'downcycling (allocation)-s1 (heat pretreatment)',
    #'MASC',
    #'incineration (film)'
]

# Search for elements in the first row of the DataFrame
negative_s_indices = [A_df_design.iloc[0,:].tolist().index(elem) for elem in search_elements]
negative_s_indices = [i-9 for i in negative_s_indices]
all_s_indices = list(range(1, len(np.transpose(A_design))+1))
positive_s_indices = [index for index in all_s_indices if index not in negative_s_indices]
zero_s_indices = [A_df_design.iloc[0,:].tolist().index(elem) for elem in search_elements_zero]
zero_s_indices = [i-9 for i in zero_s_indices]

model.set_negative_scale = Set(initialize=negative_s_indices)
model.set_positive_scale = Set(initialize=positive_s_indices)
model.set_zero_scale = Set(initialize=zero_s_indices)

#model constraint: As=f
model.set_balance = RangeSet(len(f_design))
def cons_rule(model, p):
    return sum(A_design_scenario[p-1,i-1]*model.s[i] for i in model.set_s) == f_design[p-1]
def negative_scale(model, i):
    return (model.s[i]<=0)
def positive_scale(model, i):
    return (model.s[i]>=0)
def zero_scale(model, i):
    return (model.s[i]==0)

##### Resilience Metric
# Define indices
num_rows = 10  # Number of rows/columns in R
indices = range(num_rows)
# Define epsilon for numerical stability
epsilon = 1e-10

def R_rule(model, i, j):
    if i == 0 and j == 1:
        return abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film'].tolist()[0] - 10,
                            A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'multilayer film production'].tolist()[0] - 10]) \
               * abs(model.s[A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'multilayer film production'].tolist()[0] - 9])
    elif i == 0 and j == 9:
        return abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'Ultimate EoL waste'].tolist()[0] - 10,
                            A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'multilayer film production'].tolist()[0] - 10]) \
               * abs(model.s[A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'multilayer film production'].tolist()[0] - 9])
    elif i == 1 and j == 2:
        return abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                            A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'landfill (film)'].tolist()[0] - 10]) \
               * abs(model.s[A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'landfill (film)'].tolist()[0] - 9])
    elif i == 1 and j == 3:
        return abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                            A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'incineration (film)'].tolist()[0] - 10]) \
               * abs(model.s[A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'incineration (film)'].tolist()[0] - 9])
    elif i == 1 and j == 4:
        return abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                            A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'downcycling (allocation)-s1 (heat pretreatment)'].tolist()[0] - 10]) \
               * abs(model.s[A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'downcycling (allocation)-s1 (heat pretreatment)'].tolist()[0] - 9])
    elif i == 1 and j == 5:
        return abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                            A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'downcycling (allocation)-s2 (1/3 discard)'].tolist()[0] - 10]) \
               * abs(model.s[A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'downcycling (allocation)-s2 (1/3 discard)'].tolist()[0] - 9])
    elif i == 1 and j == 6:
        return abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                            A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'STRAP'].tolist()[0] - 10]) \
               * abs(model.s[A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'STRAP'].tolist()[0] - 9])
    elif i == 1 and j == 7:
        return abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                            A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'MASC'].tolist()[0] - 10]) \
               * abs(model.s[A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'MASC'].tolist()[0] - 9])
    elif i == 1 and j == 8:
        return abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                            A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'pyrolysis'].tolist()[0] - 10]) \
               * abs(model.s[A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'pyrolysis'].tolist()[0] - 9])
    elif i == 2 and j == 9:
        return abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                   A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'landfill (film)'].tolist()[0] - 10]) \
           * abs(model.s[A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'landfill (film)'].tolist()[0] - 9])
    elif i == 3 and j == 9:
        return abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                   A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'incineration (film)'].tolist()[0] - 10]) \
           * abs(model.s[A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'incineration (film)'].tolist()[0] - 9])
    elif i == 4 and j == 9:
        return abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                   A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'downcycling (allocation)-s1 (heat pretreatment)'].tolist()[0] - 10]) \
           * abs(model.s[A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'downcycling (allocation)-s1 (heat pretreatment)'].tolist()[0] - 9])
    elif i == 5 and j == 9:
        return abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                   A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'downcycling (allocation)-s2 (1/3 discard)'].tolist()[0] - 10]) \
           * abs(model.s[A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'downcycling (allocation)-s2 (1/3 discard)'].tolist()[0] - 9])
    elif i == 6 and j == 0:
        return abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'Polyethylene, linear low-density, LLDPE, virgin resin, at plant'].tolist()[0] - 10,
                   A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'STRAP'].tolist()[0] - 10]) \
           * abs(model.s[A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'STRAP'].tolist()[0] - 9]) \
     +abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'POLYAMIDE 6 (Nylon 6) (at plant)'].tolist()[0] - 10,
                   A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'STRAP'].tolist()[0] - 10]) \
           * abs(model.s[A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'STRAP'].tolist()[0] - 9])
    elif i == 6 and j == 9:
        return abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'Ultimate EoL waste'].tolist()[0] - 10,
                   A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'STRAP'].tolist()[0] - 10]) \
           * abs(model.s[A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'STRAP'].tolist()[0] - 9])
    elif i == 7 and j == 1:
        return abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film'].tolist()[0] - 10,
                   A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'MASC'].tolist()[0] - 10]) \
           * abs(model.s[A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'MASC'].tolist()[0] - 9])
    elif i == 7 and j == 9:
        return abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'Ultimate EoL waste'].tolist()[0] - 10,
                   A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'MASC'].tolist()[0] - 10]) \
           * abs(model.s[A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'MASC'].tolist()[0] - 9])
    elif i == 8 and j == 9:
        return abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                   A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'pyrolysis'].tolist()[0] - 10]) \
           * abs(model.s[A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'pyrolysis'].tolist()[0] - 9])
    elif i == 9 and j == 0:
        return abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'Polyethylene, linear low-density, LLDPE, virgin resin, at plant'].tolist()[0] - 10,
                            A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'Polyethylene, linear low-density, LLDPE, virgin resin, at plant'].tolist()[0] - 10]) \
               * abs(model.s[A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'Polyethylene, linear low-density, LLDPE, virgin resin, at plant'].tolist()[0] - 9]) \
               + abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'POLYAMIDE 6 (Nylon 6) (at plant)'].tolist()[0] - 10,
                          A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'POLYAMIDE 6 (Nylon 6)'].tolist()[0] - 10]) \
               * abs(model.s[A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'POLYAMIDE 6 (Nylon 6)'].tolist()[0] - 9])
    else:
        return 0

model.R = Expression(indices, indices, rule=R_rule)

# Define the total sum of R
model.R_sum = Expression(expr=sum(model.R[i, j] for i in indices for j in indices))

# Define Development Capacity (DC)
def DC_rule(model):
    return sum(
        -model.R[i, j] * log((model.R[i, j] / model.R_sum) + epsilon) 
        for i in indices for j in indices if i != j
    )
model.DC = Expression(rule=DC_rule)

# Define Ascendency (ASC)
def ASC_rule(model):
    return sum(
        model.R[i, j] * log(
            (model.R[i, j] * model.R_sum) / 
            ((sum(model.R[i, k] for k in indices) + epsilon) * 
             (sum(model.R[k, j] for k in indices) + epsilon)) + epsilon
        ) 
        for i in indices for j in indices if i != j
    )
model.ASC = Expression(rule=ASC_rule)

# Define R_ASC
model.R_ASC = Expression(expr=model.ASC / (model.DC+ epsilon))


model.balance_constraints = Constraint(model.set_balance, rule=cons_rule)
model.negative_scale_constraints = Constraint(model.set_negative_scale, rule=negative_scale)
model.positive_scale_constraints = Constraint(model.set_positive_scale, rule=positive_scale)
model.zero_scale_constraints = Constraint(model.set_zero_scale, rule=zero_scale)
if res==1:  
    model.R_ASC_lower_bound = Constraint(expr=0.213 <= model.R_ASC)
    model.R_ASC_upper_bound = Constraint(expr=model.R_ASC <= 0.589)


model.eps_constraint_1 = Constraint(expr = -(abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'Polyethylene, linear low-density, LLDPE, virgin resin, at plant'].tolist()[0] - 10,
                     np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'Polyethylene, linear low-density, LLDPE, virgin resin, at plant'].tolist()[0] - 10])
    *model.s[np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'Polyethylene, linear low-density, LLDPE, virgin resin, at plant'].tolist()[0] - 9]
        
        + abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'POLYAMIDE 6 (Nylon 6) (at plant)'].tolist()[0] - 10,
                     np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'POLYAMIDE 6 (Nylon 6)'].tolist()[0] - 10])
    *model.s[np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'POLYAMIDE 6 (Nylon 6)'].tolist()[0] - 9]
        
        + (abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                     np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'landfill (film)'].tolist()[0] - 10])
    *model.s[np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'landfill (film)'].tolist()[0] - 9])*(1-0)

                                  + (abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                     np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'incineration (film)'].tolist()[0] - 10])
    *model.s[np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'incineration (film)'].tolist()[0] - 9])*(1-eta_2*gama_2)

                                       + (abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                     np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'pyrolysis'].tolist()[0] - 10])
    *model.s[np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'pyrolysis'].tolist()[0] - 9])*(1-eta_3*gama_3) 

                                  + (abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                     np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'STRAP'].tolist()[0] - 10])
    *model.s[np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'STRAP'].tolist()[0] - 9])*strap_waste

                                  + (abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                     np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'downcycling (allocation)-s1 (heat pretreatment)'].tolist()[0] - 10])
    *model.s[np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'downcycling (allocation)-s1 (heat pretreatment)'].tolist()[0] - 9])*(1-eta_5_s1*gama_5) 

                                     +(abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                     np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'downcycling (allocation)-s2 (1/3 discard)'].tolist()[0] - 10])
    *model.s[np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'downcycling (allocation)-s2 (1/3 discard)'].tolist()[0] - 9])*(1-eta_5_s2*gama_5) 

                                  + (abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                     np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'MASC'].tolist()[0] - 10])
    *model.s[np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'MASC'].tolist()[0] - 9])*masc_waste) - model.s_eps1 == model.epsilon1)


model.eps_constraint_2 = Constraint(expr = -((abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                         np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'landfill (film)'].tolist()[0] - 10])
        *model.s[np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'landfill (film)'].tolist()[0] - 9])
                                      *landf.total_cost

                                      + (abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                         np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'incineration (film)'].tolist()[0] - 10])
        *model.s[np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'incineration (film)'].tolist()[0] - 9])
                                      *landf.total_cost

                                           + (abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                         np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'pyrolysis'].tolist()[0] - 10])
        *model.s[np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'pyrolysis'].tolist()[0] - 9])
                                      *pyrol.total_cost

                                      + (abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                         np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'STRAP'].tolist()[0] - 10])
        *model.s[np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'STRAP'].tolist()[0] - 9])
                                      *_strap.total_cost

                                      + (abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                         np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'downcycling (allocation)-s1 (heat pretreatment)'].tolist()[0] - 10])
        *model.s[np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'downcycling (allocation)-s1 (heat pretreatment)'].tolist()[0] - 9])
                                      *pellet_withheat.total_cost

                                         +(abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                         np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'downcycling (allocation)-s2 (1/3 discard)'].tolist()[0] - 10])
        *model.s[np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'downcycling (allocation)-s2 (1/3 discard)'].tolist()[0] - 9])
                                      *pellet_noheat.total_cost

                                      + (abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                         np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'MASC'].tolist()[0] - 10])
        *model.s[np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'MASC'].tolist()[0] - 9])
                                      *_masc.total_cost) - model.s_eps2 == model.epsilon2)



model.obj1 = Objective(expr = -sum(coef_GWP[i-1]*model.s[i] for i in model.set_s), sense=maximize)

model.obj2 = Objective(expr = -(abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'Polyethylene, linear low-density, LLDPE, virgin resin, at plant'].tolist()[0] - 10,
                     np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'Polyethylene, linear low-density, LLDPE, virgin resin, at plant'].tolist()[0] - 10])
    *model.s[np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'Polyethylene, linear low-density, LLDPE, virgin resin, at plant'].tolist()[0] - 9]
        
        + abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'POLYAMIDE 6 (Nylon 6) (at plant)'].tolist()[0] - 10,
                     np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'POLYAMIDE 6 (Nylon 6)'].tolist()[0] - 10])
    *model.s[np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'POLYAMIDE 6 (Nylon 6)'].tolist()[0] - 9]
        
        + (abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                     np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'landfill (film)'].tolist()[0] - 10])
    *model.s[np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'landfill (film)'].tolist()[0] - 9])*(1-0)

                                  + (abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                     np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'incineration (film)'].tolist()[0] - 10])
    *model.s[np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'incineration (film)'].tolist()[0] - 9])*(1-eta_2*gama_2)

                                       + (abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                     np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'pyrolysis'].tolist()[0] - 10])
    *model.s[np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'pyrolysis'].tolist()[0] - 9])*(1-eta_3*gama_3) 

                                  + (abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                     np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'STRAP'].tolist()[0] - 10])
    *model.s[np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'STRAP'].tolist()[0] - 9])*strap_waste

                                  + (abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                     np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'downcycling (allocation)-s1 (heat pretreatment)'].tolist()[0] - 10])
    *model.s[np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'downcycling (allocation)-s1 (heat pretreatment)'].tolist()[0] - 9])*(1-eta_5_s1*gama_5) 

                                     +(abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                     np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'downcycling (allocation)-s2 (1/3 discard)'].tolist()[0] - 10])
    *model.s[np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'downcycling (allocation)-s2 (1/3 discard)'].tolist()[0] - 9])*(1-eta_5_s2*gama_5) 

                                  + (abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                     np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'MASC'].tolist()[0] - 10])
    *model.s[np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'MASC'].tolist()[0] - 9])*masc_waste), sense=maximize
            )

model.obj3 = Objective(expr = -((abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                         np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'landfill (film)'].tolist()[0] - 10])
        *model.s[np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'landfill (film)'].tolist()[0] - 9])
                                      *landf.total_cost

                                      + (abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                         np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'incineration (film)'].tolist()[0] - 10])
        *model.s[np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'incineration (film)'].tolist()[0] - 9])
                                      *landf.total_cost

                                           + (abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                         np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'pyrolysis'].tolist()[0] - 10])
        *model.s[np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'pyrolysis'].tolist()[0] - 9])
                                      *pyrol.total_cost

                                      + (abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                         np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'STRAP'].tolist()[0] - 10])
        *model.s[np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'STRAP'].tolist()[0] - 9])
                                      *_strap.total_cost

                                      + (abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                         np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'downcycling (allocation)-s1 (heat pretreatment)'].tolist()[0] - 10])
        *model.s[np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'downcycling (allocation)-s1 (heat pretreatment)'].tolist()[0] - 9])
                                      *pellet_withheat.total_cost

                                         +(abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                         np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'downcycling (allocation)-s2 (1/3 discard)'].tolist()[0] - 10])
        *model.s[np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'downcycling (allocation)-s2 (1/3 discard)'].tolist()[0] - 9])
                                      *pellet_noheat.total_cost

                                      + (abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                         np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'MASC'].tolist()[0] - 10])
        *model.s[np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'MASC'].tolist()[0] - 9])
                                      *_masc.total_cost), sense=maximize
         )

#Solver
if res==0:
    solver = SolverFactory('gurobi')
elif res==1:
    solver = SolverFactory('ipopt')

### Epsilon bonds

In [ ]:
model.obj1.activate() 
model.obj2.deactivate()
model.obj3.deactivate()
model.eps_constraint_1.deactivate()
model.eps_constraint_2.deactivate()
results = solver.solve(model, tee=True) # solves and updates instance
Z1_1_median=value(model.obj1)
Z2_1_median=value(model.obj2)
Z3_1_median=value(model.obj3)

In [ ]:
model.obj1.deactivate() 
model.obj2.activate() 
model.obj3.deactivate()
model.eps_constraint_1.deactivate()
model.eps_constraint_2.deactivate()
results = solver.solve(model, tee=True) # solves and updates instance
Z1_2_median= value(model.obj1)
Z2_2_median= value(model.obj2)
Z3_2_median= value(model.obj3)
scaling_factors_circ = []
for j in model.set_s:
    scaling_factors_circ.append(model.s[j].value)

In [ ]:
model.obj1.deactivate() 
model.obj2.deactivate()
model.obj3.activate()
model.eps_constraint_1.deactivate()
model.eps_constraint_2.deactivate()
results = solver.solve(model, tee=True) # solves and updates instance
Z1_3_median= value(model.obj1)
Z2_3_median= value(model.obj2)
Z3_3_median= value(model.obj3)

### Solving the Optimization Problem

In [ ]:
M1_median = max(Z1_1_median,Z1_2_median,Z1_3_median)
n1_median = min(Z1_1_best,Z1_2_best,Z1_3_best)

M2_median = max(Z2_1_median,Z2_2_median,Z2_3_median)
n2_median = min(Z2_1_median,Z2_2_median,Z2_3_median)

M3_median = max(Z3_1_median,Z3_2_median,Z3_3_median)
n3_median = min(Z3_1_median,Z3_2_median,Z3_3_median)

In [ ]:
M_penalty = 1e-4
delta_1 =  M2_median - n2_median
delta_2 = M3_median - n3_median
model.obj1_aug = Objective(expr = -sum(coef_GWP[i-1]*model.s[i] for i in model.set_s) + M_penalty*(model.s_eps1/delta_1 + model.s_eps2/delta_2), sense=maximize)

In [ ]:
#Z1, Z2, and Z3 values!!! for pareto surface generation

#Best
model.obj1.deactivate()
model.obj1_aug.activate()
model.obj2.deactivate() 
model.obj3.deactivate()
model.eps_constraint_1.activate()
model.eps_constraint_2.activate()


r=100
Z1=[]
Z2=[]
Z3=[]
solution_matrix = []

for t in range(0,r):
    for i in range (0,r):
        model.epsilon1 = n2_median + (M2_median - n2_median) * t / (r - 1)
        model.epsilon2 = n3_median + (M3_median - n3_median) * i / (r - 1)
        results = solver.solve(model) # solves and updates instance
        
        if results.solver.termination_condition == TerminationCondition.optimal:           
            Z1.append(-value(model.obj1))
            Z2.append(-value(model.obj2))
            Z3.append(-value(model.obj3))
            
            
            scaling_factors = []
            for j in model.set_s:
                scaling_factors.append(model.s[j].value)
            solution_matrix.append(scaling_factors) 

Z1_median.append(-Z1_2_median)
Z2_median.append(-Z2_2_median)
Z3_median.append(-Z3_2_median)

In [ ]:
solution_matrix.append(scaling_factors_circ)

### Generating the pareto frontier numbers

In [ ]:
# Assuming Z1, Z2, and Z3 are lists of values

# Create a dictionary to store unique combinations
unique_combinations = {}

# Iterate over the lists and store unique combinations in the dictionary
for z1, z2, z3 in zip(Z1_median, Z2_median, Z3_median):
    combination = (z1, z2, z3)
    if combination not in unique_combinations:
        unique_combinations[combination] = 1  # Value doesn't matter, just using the key for uniqueness

# Extract unique values from the dictionary
unique_Z1, unique_Z2, unique_Z3 = zip(*unique_combinations.keys())

In [ ]:
# Assuming Z1, Z2, and Z3 are lists of values
# Create a dictionary to store unique combinations and their indices
unique_combinations = {}
indices = []
unique_solution_matrix = []

# Iterate over the lists and store unique combinations and their indices in the dictionary
for i, (z1, z2, z3) in enumerate(zip(Z1_median, Z2_median, Z3_median)):
    combination = (z1, z2, z3)
    if combination not in unique_combinations:
        unique_combinations[combination] = [i]  # Store the index as a list
    else:
        unique_combinations[combination].append(i)  # Append the index to the existing list

# Extract unique values and their corresponding indices from the dictionary
indices = list(unique_combinations.values())

# New list with only the first elements of each sublist
unique_solution_indices = [sublist[0] for sublist in indices]

unique_solution_matrix = [solution_matrix[index] for index in unique_solution_indices]

In [ ]:
pareto_GWP = unique_Z1
pareto_circularity = [(1 - (value / (2 * f_design[0]))) for value in unique_Z2]
pareto_cost = unique_Z3

#pareto front points
pareto_points = [[pareto_GWP[i], pareto_circularity[i], pareto_cost[i]] for i in range(len(pareto_GWP))]

In [ ]:
#utopia point

utopia_point_GWP = min(pareto_GWP)
scaled_utopia_point_GWP = (utopia_point_GWP - min(pareto_GWP)) / (max(pareto_GWP) - min(pareto_GWP))

utopia_point_circularity = max(pareto_circularity)
scaled_utopia_point_circularity = (utopia_point_circularity - min(pareto_circularity)) / (max(pareto_circularity) - min(pareto_circularity))

utopia_point_cost = min(pareto_cost)
scaled_utopia_point_cost = (utopia_point_cost - min(pareto_cost)) / (max(pareto_cost) - min(pareto_cost))                                                     

In [ ]:
#Compromised Solution

scaled_pareto_GWP = [(element - min(pareto_GWP)) / (max(pareto_GWP) - min(pareto_GWP)) for element in pareto_GWP]
                                                                     

scaled_pareto_circularity = [(element - min(pareto_circularity)) / (max(pareto_circularity) - min(pareto_circularity)) for element in pareto_circularity]


scaled_pareto_cost = [(element - min(pareto_cost)) / (max(pareto_cost) - min(pareto_cost)) for element in pareto_cost]


# Calculate the distance to the utopia point
distances = []
for i in range(len(scaled_pareto_GWP)):
    distances.append(
        np.sqrt((scaled_pareto_GWP[i] - scaled_utopia_point_GWP) ** 2  
        + (scaled_pareto_circularity[i] - scaled_utopia_point_circularity) ** 2)
        + (scaled_pareto_cost[i] - scaled_utopia_point_cost) ** 2)
index_min_distance = np.argmin(distances)

In [ ]:
#Solutions' scaling factors
index_min_GWP_solution = np.argmin(pareto_GWP)
index_max_circularity_solution = np.argmax(pareto_circularity)
index_min_cost_solution = np.argmin(pareto_cost)
index_compromised_solution = index_min_distance

min_GWP_solution = unique_solution_matrix[index_min_GWP_solution]
max_circularity_solution = unique_solution_matrix[index_max_circularity_solution]
min_cost_solution = unique_solution_matrix[index_min_cost_solution]
compromised_solution = unique_solution_matrix[index_compromised_solution]

## 2.3. Worst solution

In [ ]:
#Uncertainty scenario
scen = 2

if scen ==0:
    #Best case values
    masc_solv_draws = 0.01
    masc_ener_draws = 0.0003694444
    masc_eff_draws = 0.95
    masc_cycles = 100

    strap_solv_draws = 0.002
    strap_ener_draws = 0.00001555556
    strap_eff_draws = 0.95
    strap_cycles = 100
    
elif scen ==1:
    # Expected values
    masc_solv_draws = 0.1
    masc_ener_draws = 0.0007388887
    masc_eff_draws = 0.875
    masc_cycles = 5

    strap_solv_draws = 0.1
    strap_ener_draws = 0.000023472225
    strap_eff_draws = 0.875
    strap_cycles = 5

elif scen==2:
    #Worst case values
    masc_solv_draws = 0.2
    masc_ener_draws = 0.001108333
    masc_eff_draws = 0.80
    masc_cycles = 1

    strap_solv_draws = 0.2
    strap_ener_draws = 0.000023472225
    strap_eff_draws = 0.80
    strap_cycles = 1
    
#TEA
masc_des = masc_design.loc[(masc_design.Index.isin(['Electricity','Diacetone alcohol']))  | (masc_design.Variable == 'Output efficiency')]
strap_des = strap_design.loc[(strap_design.Index.isin(['Electricity','Ethylene glycol']))  | (strap_design.Variable == 'Output efficiency')]


A_design_scenario = []
masc_unc_costs = []
strap_unc_costs = []
masc_unc_waste = []
strap_unc_waste = []



A_design_scenario = A_design.copy()

## MASC ##

A_design_scenario [A_df_design.index[A_df_design.iloc[:, 1] == 'Diacetone Alcohol'].tolist()[0] - 10,
                 np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'MASC'].tolist()[0] - 10] = - 5.56 * (masc_solv_draws)*(1/masc_eff_draws)*calculate_sum_mi_masc(masc_eff_draws, masc_cycles)

A_design_scenario [A_df_design.index[A_df_design.iloc[:, 0] == 'Electricity, at Grid, US, 2010'].tolist()[0] - 10,
                 np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'MASC'].tolist()[0] - 10] = - 5.56 * 2.7 * (1/masc_eff_draws)*calculate_sum_mi_masc(masc_eff_draws, masc_cycles)

A_design_scenario [A_df_design.index[A_df_design.iloc[:, 1] == 'Ultimate EoL waste'].tolist()[0] - 10,
                 np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'MASC'].tolist()[0] - 10] = ((1-masc_eff_draws)/masc_eff_draws)*calculate_sum_mi_masc(masc_eff_draws, masc_cycles) + calculate_mi_masc(masc_eff_draws,masc_cycles,masc_cycles)

A_design_scenario [A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film'].tolist()[0] - 10,
                 np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'MASC'].tolist()[0] - 10] = 1 - (calculate_mp_masc(masc_eff_draws, masc_cycles))
## STRAP ##

A_design_scenario [A_df_design.index[A_df_design.iloc[:, 0] == 'Electricity, at Grid, US, 2010'].tolist()[0] - 10,
                 np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'STRAP'].tolist()[0] - 10] = - (1/0.9) * 0.648 * (1/strap_eff_draws) * calculate_sum_mi_strap(strap_eff_draws, strap_cycles)

A_design_scenario [A_df_design.index[A_df_design.iloc[:, 1] == 'Ethylene glycol, at plant, kg'].tolist()[0] - 10,
                 np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'STRAP'].tolist()[0] - 10] = - 11.5 * (1/0.9) * (strap_solv_draws)*(1/strap_eff_draws)*calculate_sum_mi_strap(strap_eff_draws, strap_cycles)

A_design_scenario [A_df_design.index[A_df_design.iloc[:, 1] == 'heat, from natural gas'].tolist()[0] - 10,
                 np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'STRAP'].tolist()[0] - 10] = - (1/0.9) * 0.00351 * (1/strap_eff_draws) * calculate_sum_mi_strap(strap_eff_draws, strap_cycles) 

A_design_scenario [A_df_design.index[A_df_design.iloc[:, 1] == 'Ultimate EoL waste'].tolist()[0] - 10,
                 np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'STRAP'].tolist()[0] - 10] = ((1-strap_eff_draws)/strap_eff_draws)*(1/0.9)*calculate_sum_mi_strap(strap_eff_draws, strap_cycles) + calculate_mi_strap(strap_eff_draws,strap_cycles,strap_cycles)

A_design_scenario [A_df_design.index[A_df_design.iloc[:, 1] == 'POLYAMIDE 6 (Nylon 6) (at plant)'].tolist()[0] - 10,
                 np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'STRAP'].tolist()[0] - 10] = 0.26/0.9 - (0.26/0.9)*calculate_mp_strap(strap_eff_draws, strap_cycles)

A_design_scenario [A_df_design.index[A_df_design.iloc[:, 1] == 'Polyethylene, linear low-density, LLDPE, virgin resin, at plant'].tolist()[0] - 10,
                 np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'STRAP'].tolist()[0] - 10] = 0.74/0.9 - (0.74/0.9)*calculate_mp_strap(strap_eff_draws, strap_cycles)



masc_des.loc[masc_des.Index == 'Electricity','Value'] = 2521.974 * masc_ener_draws
masc_des.loc[masc_des.Index == 'Diacetone alcohol','Value'] = 5560 * masc_solv_draws
masc_des.loc[masc_des.Variable == 'Output efficiency','Value'] = masc_eff_draws

_masc = Scenario(
    func_unit = 2.204,
    sens_df = masc_des,
    eol_sc = {'Mechanical and Solvent Cleaning': [1.0, masc_cycles]},
    products = {'Mechanical and Solvent Cleaning': 'Barrier film'}
)


strap_des.loc[strap_des.Index == 'Electricity', 'Value'] = 11400 * strap_ener_draws
strap_des.loc[strap_des.Index == 'Ethylene glycol', 'Value'] = 25132.698 * strap_solv_draws
strap_des.loc[strap_des.Variable == 'Output efficiency', 'Value'] = strap_eff_draws

_strap = Scenario(
    func_unit = 2.204,
    sens_df = strap_des,
    eol_sc = {'Solvent Treatment and Precipitation': [1.0, strap_cycles]},
    products = {'Solvent Treatment and Precipitation':'Polyethylene'}
)

masc_waste = ((1-masc_eff_draws)/masc_eff_draws)*calculate_sum_mi_masc(masc_eff_draws, masc_cycles) + calculate_mi_masc(masc_eff_draws,masc_cycles,masc_cycles)
strap_waste = ((1-strap_eff_draws)/strap_eff_draws)*(1/0.9)*calculate_sum_mi_strap(strap_eff_draws, strap_cycles) + calculate_mi_strap(strap_eff_draws,strap_cycles,strap_cycles)

## 2.4. All uncertainty casess solutions

In [ ]:
M1 = max(Z1_1_best,Z1_2_best,Z1_3_best,Z1_1_worst,Z1_2_worst,Z1_3_worst,Z1_1_median,Z1_2_median,Z1_3_median)
n1 = min(Z1_1_best,Z1_2_best,Z1_3_best,Z1_1_worst,Z1_2_worst,Z1_3_worst,Z1_1_median,Z1_2_median,Z1_3_median)

M2 = max(Z2_1_best,Z2_2_best,Z2_3_best,Z2_1_worst,Z2_2_worst,Z2_3_worst,Z2_1_median,Z2_2_median,Z2_3_median)
n2 = min(Z2_1_best,Z2_2_best,Z2_3_best,Z2_1_worst,Z2_2_worst,Z2_3_worst,Z2_1_median,Z2_2_median,Z2_3_median)

M3 = max(Z3_1_best,Z3_2_best,Z3_3_best,Z3_1_worst,Z3_2_worst,Z3_3_worst,Z3_1_median,Z3_2_median,Z3_3_median)
n3 = min(Z3_1_best,Z3_2_best,Z3_3_best,Z3_1_worst,Z3_2_worst,Z3_3_worst,Z3_1_median,Z3_2_median,Z3_3_median)

In [ ]:
#Z1, Z2, and Z3 values!!! for pareto surface generation

#Best

#model.obj1_aug.activate
model.obj1.activate() 
model.obj2.deactivate() 
model.obj3.deactivate()
model.eps_constraint_1.activate()
model.eps_constraint_2.activate()


r=10

solution_matrix = []

hist1_wo_res = []
hist2_wo_res = []
hist3_wo_res = []



for t in range(0,r):
    for i in range (0,r):
        model.epsilon1 = n2_closest + (M2_closest - n2_closest) * t / (r - 1)
        model.epsilon2 = n3_closest + (M3_closest - n3_closest) * i / (r - 1)
        results = solver.solve(model) # solves and updates instance
        
        if results.solver.termination_condition == TerminationCondition.optimal:
            from pyomo.environ import value
            scenario_obj1_values = {}
            for sc in model.scenarios:
                scenario_obj1_values[sc] = - value(model.obj1_scenario[sc])

            scenario_obj2_values = {}
            for sc in model.scenarios:
                scenario_obj2_values[sc] = - value(model.obj2_scenario[sc])

            scenario_obj3_values = {}
            for sc in model.scenarios:
                scenario_obj3_values[sc] = - value(model.obj3_scenario[sc])
            
            avg_obj_1 = sum(scenario_obj1_values.values()) / len(scenario_obj1_values)

            def distance_to_avg_1(sc):
                return abs(scenario_obj1_values[sc] - avg_obj_1)

            median_sc_obj1 = min(model.scenarios, key=distance_to_avg_1)
            # 3a. Best scenario (min)
            best_sc_obj1 = min(scenario_obj1_values, key=scenario_obj1_values.get)

            # 3b. Worst scenario (max)
            worst_sc_obj1 = max(scenario_obj1_values, key=scenario_obj1_values.get)
            
            
            avg_obj_2 = sum(scenario_obj2_values.values()) / len(scenario_obj2_values)

            def distance_to_avg_2(sc):
                return abs(scenario_obj2_values[sc] - avg_obj)

            median_sc_obj2 = min(model.scenarios, key=distance_to_avg_2)
            # 3a. Best scenario (min)
            best_sc_obj2 = min(scenario_obj2_values, key=scenario_obj2_values.get)

            # 3b. Worst scenario (max)
            worst_sc_obj2 = max(scenario_obj2_values, key=scenario_obj2_values.get)
            
            avg_obj_3 = sum(scenario_obj3_values.values()) / len(scenario_obj3_values)

            def distance_to_avg_3(sc):
                return abs(scenario_obj3_values[sc] - avg_obj)

            median_sc_obj3 = min(model.scenarios, key=distance_to_avg_3)
            # 3a. Best scenario (min)
            best_sc_obj3 = min(scenario_obj3_values, key=scenario_obj3_values.get)

            # 3b. Worst scenario (max)
            worst_sc_obj3 = max(scenario_obj3_values, key=scenario_obj3_values.get)
            
            
            hist1_wo_res.append(scenario_obj1_values)
            hist2_wo_res.append(scenario_obj2_values)
            hist3_wo_res.append(scenario_obj3_values)          

### Multi-objective optimization formulation

In [ ]:
#Model Formulation
# Create the model
model = ConcreteModel()

res = 0

model.s_eps1 = Var(within=NonNegativeReals, bounds=(1e-6, None))  # Slack for epsilon constraint 1
model.s_eps2 = Var(within=NonNegativeReals, bounds=(1e-6, None))  # Slack for epsilon constraint 2


#epsilon parameter
model.epsilon1=Param(initialize=1000,mutable=True)
model.epsilon2=Param(initialize=1000,mutable=True)

# Define the decision variable
model.set_s = RangeSet(len(np.transpose(A_design)))
model.s = Var(model.set_s)
for i in model.set_s:
    model.s[i].set_value(1.0)  # Adjust based on the problem's context
# List of processes with negative scaling factor due to the substitution approach
negative_s_indices = []
positive_s_indices = []
all_s_indices = []
search_elements = [
    'Substitued Electricity US Market',
    'Substitued Polypropylene Market',
    'Substitued Refinery Oil Market'
]

search_elements_zero = [
    #'pyrolysis',
    #'STRAP',
    #'MASC',
    #'landfill (film)',
    #'downcycling (allocation)-s1 (heat pretreatment)',
    #'MASC',
    #'incineration (film)'
]

# Search for elements in the first row of the DataFrame
negative_s_indices = [A_df_design.iloc[0,:].tolist().index(elem) for elem in search_elements]
negative_s_indices = [i-9 for i in negative_s_indices]
all_s_indices = list(range(1, len(np.transpose(A_design))+1))
positive_s_indices = [index for index in all_s_indices if index not in negative_s_indices]
zero_s_indices = [A_df_design.iloc[0,:].tolist().index(elem) for elem in search_elements_zero]
zero_s_indices = [i-9 for i in zero_s_indices]

model.set_negative_scale = Set(initialize=negative_s_indices)
model.set_positive_scale = Set(initialize=positive_s_indices)
model.set_zero_scale = Set(initialize=zero_s_indices)

#model constraint: As=f
model.set_balance = RangeSet(len(f_design))
def cons_rule(model, p):
    return sum(A_design_scenario[p-1,i-1]*model.s[i] for i in model.set_s) == f_design[p-1]
def negative_scale(model, i):
    return (model.s[i]<=0)
def positive_scale(model, i):
    return (model.s[i]>=0)
def zero_scale(model, i):
    return (model.s[i]==0)

##### Resilience Metric
# Define indices
num_rows = 10  # Number of rows/columns in R
indices = range(num_rows)
# Define epsilon for numerical stability
epsilon = 1e-10

def R_rule(model, i, j):
    if i == 0 and j == 1:
        return abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film'].tolist()[0] - 10,
                            A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'multilayer film production'].tolist()[0] - 10]) \
               * abs(model.s[A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'multilayer film production'].tolist()[0] - 9])
    elif i == 0 and j == 9:
        return abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'Ultimate EoL waste'].tolist()[0] - 10,
                            A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'multilayer film production'].tolist()[0] - 10]) \
               * abs(model.s[A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'multilayer film production'].tolist()[0] - 9])
    elif i == 1 and j == 2:
        return abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                            A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'landfill (film)'].tolist()[0] - 10]) \
               * abs(model.s[A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'landfill (film)'].tolist()[0] - 9])
    elif i == 1 and j == 3:
        return abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                            A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'incineration (film)'].tolist()[0] - 10]) \
               * abs(model.s[A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'incineration (film)'].tolist()[0] - 9])
    elif i == 1 and j == 4:
        return abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                            A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'downcycling (allocation)-s1 (heat pretreatment)'].tolist()[0] - 10]) \
               * abs(model.s[A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'downcycling (allocation)-s1 (heat pretreatment)'].tolist()[0] - 9])
    elif i == 1 and j == 5:
        return abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                            A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'downcycling (allocation)-s2 (1/3 discard)'].tolist()[0] - 10]) \
               * abs(model.s[A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'downcycling (allocation)-s2 (1/3 discard)'].tolist()[0] - 9])
    elif i == 1 and j == 6:
        return abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                            A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'STRAP'].tolist()[0] - 10]) \
               * abs(model.s[A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'STRAP'].tolist()[0] - 9])
    elif i == 1 and j == 7:
        return abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                            A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'MASC'].tolist()[0] - 10]) \
               * abs(model.s[A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'MASC'].tolist()[0] - 9])
    elif i == 1 and j == 8:
        return abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                            A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'pyrolysis'].tolist()[0] - 10]) \
               * abs(model.s[A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'pyrolysis'].tolist()[0] - 9])
    elif i == 2 and j == 9:
        return abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                   A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'landfill (film)'].tolist()[0] - 10]) \
           * abs(model.s[A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'landfill (film)'].tolist()[0] - 9])
    elif i == 3 and j == 9:
        return abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                   A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'incineration (film)'].tolist()[0] - 10]) \
           * abs(model.s[A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'incineration (film)'].tolist()[0] - 9])
    elif i == 4 and j == 9:
        return abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                   A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'downcycling (allocation)-s1 (heat pretreatment)'].tolist()[0] - 10]) \
           * abs(model.s[A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'downcycling (allocation)-s1 (heat pretreatment)'].tolist()[0] - 9])
    elif i == 5 and j == 9:
        return abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                   A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'downcycling (allocation)-s2 (1/3 discard)'].tolist()[0] - 10]) \
           * abs(model.s[A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'downcycling (allocation)-s2 (1/3 discard)'].tolist()[0] - 9])
    elif i == 6 and j == 0:
        return abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'Polyethylene, linear low-density, LLDPE, virgin resin, at plant'].tolist()[0] - 10,
                   A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'STRAP'].tolist()[0] - 10]) \
           * abs(model.s[A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'STRAP'].tolist()[0] - 9]) \
     +abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'POLYAMIDE 6 (Nylon 6) (at plant)'].tolist()[0] - 10,
                   A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'STRAP'].tolist()[0] - 10]) \
           * abs(model.s[A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'STRAP'].tolist()[0] - 9])
    elif i == 6 and j == 9:
        return abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'Ultimate EoL waste'].tolist()[0] - 10,
                   A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'STRAP'].tolist()[0] - 10]) \
           * abs(model.s[A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'STRAP'].tolist()[0] - 9])
    elif i == 7 and j == 1:
        return abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film'].tolist()[0] - 10,
                   A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'MASC'].tolist()[0] - 10]) \
           * abs(model.s[A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'MASC'].tolist()[0] - 9])
    elif i == 7 and j == 9:
        return abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'Ultimate EoL waste'].tolist()[0] - 10,
                   A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'MASC'].tolist()[0] - 10]) \
           * abs(model.s[A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'MASC'].tolist()[0] - 9])
    elif i == 8 and j == 9:
        return abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                   A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'pyrolysis'].tolist()[0] - 10]) \
           * abs(model.s[A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'pyrolysis'].tolist()[0] - 9])
    elif i == 9 and j == 0:
        return abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'Polyethylene, linear low-density, LLDPE, virgin resin, at plant'].tolist()[0] - 10,
                            A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'Polyethylene, linear low-density, LLDPE, virgin resin, at plant'].tolist()[0] - 10]) \
               * abs(model.s[A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'Polyethylene, linear low-density, LLDPE, virgin resin, at plant'].tolist()[0] - 9]) \
               + abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'POLYAMIDE 6 (Nylon 6) (at plant)'].tolist()[0] - 10,
                          A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'POLYAMIDE 6 (Nylon 6)'].tolist()[0] - 10]) \
               * abs(model.s[A_df_design.T.index[A_df_design.T.iloc[:, 0] == 'POLYAMIDE 6 (Nylon 6)'].tolist()[0] - 9])
    else:
        return 0

model.R = Expression(indices, indices, rule=R_rule)

# Define the total sum of R
model.R_sum = Expression(expr=sum(model.R[i, j] for i in indices for j in indices))

# Define Development Capacity (DC)
def DC_rule(model):
    return sum(
        -model.R[i, j] * log((model.R[i, j] / model.R_sum) + epsilon) 
        for i in indices for j in indices if i != j
    )
model.DC = Expression(rule=DC_rule)

# Define Ascendency (ASC)
def ASC_rule(model):
    return sum(
        model.R[i, j] * log(
            (model.R[i, j] * model.R_sum) / 
            ((sum(model.R[i, k] for k in indices) + epsilon) * 
             (sum(model.R[k, j] for k in indices) + epsilon)) + epsilon
        ) 
        for i in indices for j in indices if i != j
    )
model.ASC = Expression(rule=ASC_rule)

# Define R_ASC
model.R_ASC = Expression(expr=model.ASC / (model.DC+ epsilon))


model.balance_constraints = Constraint(model.set_balance, rule=cons_rule)
model.negative_scale_constraints = Constraint(model.set_negative_scale, rule=negative_scale)
model.positive_scale_constraints = Constraint(model.set_positive_scale, rule=positive_scale)
model.zero_scale_constraints = Constraint(model.set_zero_scale, rule=zero_scale)
if res==1:  
    model.R_ASC_lower_bound = Constraint(expr=0.213 <= model.R_ASC)
    model.R_ASC_upper_bound = Constraint(expr=model.R_ASC <= 0.589)


model.eps_constraint_1 = Constraint(expr = -(abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'Polyethylene, linear low-density, LLDPE, virgin resin, at plant'].tolist()[0] - 10,
                     np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'Polyethylene, linear low-density, LLDPE, virgin resin, at plant'].tolist()[0] - 10])
    *model.s[np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'Polyethylene, linear low-density, LLDPE, virgin resin, at plant'].tolist()[0] - 9]
        
        + abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'POLYAMIDE 6 (Nylon 6) (at plant)'].tolist()[0] - 10,
                     np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'POLYAMIDE 6 (Nylon 6)'].tolist()[0] - 10])
    *model.s[np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'POLYAMIDE 6 (Nylon 6)'].tolist()[0] - 9]
        
        + (abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                     np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'landfill (film)'].tolist()[0] - 10])
    *model.s[np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'landfill (film)'].tolist()[0] - 9])*(1-0)

                                  + (abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                     np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'incineration (film)'].tolist()[0] - 10])
    *model.s[np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'incineration (film)'].tolist()[0] - 9])*(1-eta_2*gama_2)

                                       + (abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                     np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'pyrolysis'].tolist()[0] - 10])
    *model.s[np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'pyrolysis'].tolist()[0] - 9])*(1-eta_3*gama_3) 

                                  + (abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                     np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'STRAP'].tolist()[0] - 10])
    *model.s[np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'STRAP'].tolist()[0] - 9])*strap_waste

                                  + (abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                     np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'downcycling (allocation)-s1 (heat pretreatment)'].tolist()[0] - 10])
    *model.s[np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'downcycling (allocation)-s1 (heat pretreatment)'].tolist()[0] - 9])*(1-eta_5_s1*gama_5) 

                                     +(abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                     np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'downcycling (allocation)-s2 (1/3 discard)'].tolist()[0] - 10])
    *model.s[np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'downcycling (allocation)-s2 (1/3 discard)'].tolist()[0] - 9])*(1-eta_5_s2*gama_5) 

                                  + (abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                     np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'MASC'].tolist()[0] - 10])
    *model.s[np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'MASC'].tolist()[0] - 9])*masc_waste) - model.s_eps1 == model.epsilon1)


model.eps_constraint_2 = Constraint(expr = -((abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                         np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'landfill (film)'].tolist()[0] - 10])
        *model.s[np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'landfill (film)'].tolist()[0] - 9])
                                      *landf.total_cost

                                      + (abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                         np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'incineration (film)'].tolist()[0] - 10])
        *model.s[np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'incineration (film)'].tolist()[0] - 9])
                                      *landf.total_cost

                                           + (abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                         np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'pyrolysis'].tolist()[0] - 10])
        *model.s[np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'pyrolysis'].tolist()[0] - 9])
                                      *pyrol.total_cost

                                      + (abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                         np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'STRAP'].tolist()[0] - 10])
        *model.s[np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'STRAP'].tolist()[0] - 9])
                                      *_strap.total_cost

                                      + (abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                         np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'downcycling (allocation)-s1 (heat pretreatment)'].tolist()[0] - 10])
        *model.s[np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'downcycling (allocation)-s1 (heat pretreatment)'].tolist()[0] - 9])
                                      *pellet_withheat.total_cost

                                         +(abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                         np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'downcycling (allocation)-s2 (1/3 discard)'].tolist()[0] - 10])
        *model.s[np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'downcycling (allocation)-s2 (1/3 discard)'].tolist()[0] - 9])
                                      *pellet_noheat.total_cost

                                      + (abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                         np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'MASC'].tolist()[0] - 10])
        *model.s[np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'MASC'].tolist()[0] - 9])
                                      *_masc.total_cost) - model.s_eps2 == model.epsilon2)



model.obj1 = Objective(expr = -sum(coef_GWP[i-1]*model.s[i] for i in model.set_s), sense=maximize)

model.obj2 = Objective(expr = -(abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'Polyethylene, linear low-density, LLDPE, virgin resin, at plant'].tolist()[0] - 10,
                     np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'Polyethylene, linear low-density, LLDPE, virgin resin, at plant'].tolist()[0] - 10])
    *model.s[np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'Polyethylene, linear low-density, LLDPE, virgin resin, at plant'].tolist()[0] - 9]
        
        + abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'POLYAMIDE 6 (Nylon 6) (at plant)'].tolist()[0] - 10,
                     np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'POLYAMIDE 6 (Nylon 6)'].tolist()[0] - 10])
    *model.s[np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'POLYAMIDE 6 (Nylon 6)'].tolist()[0] - 9]
        
        + (abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                     np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'landfill (film)'].tolist()[0] - 10])
    *model.s[np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'landfill (film)'].tolist()[0] - 9])*(1-0)

                                  + (abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                     np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'incineration (film)'].tolist()[0] - 10])
    *model.s[np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'incineration (film)'].tolist()[0] - 9])*(1-eta_2*gama_2)

                                       + (abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                     np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'pyrolysis'].tolist()[0] - 10])
    *model.s[np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'pyrolysis'].tolist()[0] - 9])*(1-eta_3*gama_3) 

                                  + (abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                     np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'STRAP'].tolist()[0] - 10])
    *model.s[np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'STRAP'].tolist()[0] - 9])*strap_waste

                                  + (abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                     np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'downcycling (allocation)-s1 (heat pretreatment)'].tolist()[0] - 10])
    *model.s[np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'downcycling (allocation)-s1 (heat pretreatment)'].tolist()[0] - 9])*(1-eta_5_s1*gama_5) 

                                     +(abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                     np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'downcycling (allocation)-s2 (1/3 discard)'].tolist()[0] - 10])
    *model.s[np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'downcycling (allocation)-s2 (1/3 discard)'].tolist()[0] - 9])*(1-eta_5_s2*gama_5) 

                                  + (abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                     np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'MASC'].tolist()[0] - 10])
    *model.s[np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'MASC'].tolist()[0] - 9])*masc_waste), sense=maximize
            )

model.obj3 = Objective(expr = -((abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                         np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'landfill (film)'].tolist()[0] - 10])
        *model.s[np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'landfill (film)'].tolist()[0] - 9])
                                      *landf.total_cost

                                      + (abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                         np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'incineration (film)'].tolist()[0] - 10])
        *model.s[np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'incineration (film)'].tolist()[0] - 9])
                                      *landf.total_cost

                                           + (abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                         np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'pyrolysis'].tolist()[0] - 10])
        *model.s[np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'pyrolysis'].tolist()[0] - 9])
                                      *pyrol.total_cost

                                      + (abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                         np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'STRAP'].tolist()[0] - 10])
        *model.s[np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'STRAP'].tolist()[0] - 9])
                                      *_strap.total_cost

                                      + (abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                         np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'downcycling (allocation)-s1 (heat pretreatment)'].tolist()[0] - 10])
        *model.s[np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'downcycling (allocation)-s1 (heat pretreatment)'].tolist()[0] - 9])
                                      *pellet_withheat.total_cost

                                         +(abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                         np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'downcycling (allocation)-s2 (1/3 discard)'].tolist()[0] - 10])
        *model.s[np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'downcycling (allocation)-s2 (1/3 discard)'].tolist()[0] - 9])
                                      *pellet_noheat.total_cost

                                      + (abs(A_design_scenario[A_df_design.index[A_df_design.iloc[:, 1] == 'multilayer film waste'].tolist()[0] - 10,
                         np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'MASC'].tolist()[0] - 10])
        *model.s[np.transpose(A_df_design).index[np.transpose(A_df_design).iloc[:, 0] == 'MASC'].tolist()[0] - 9])
                                      *_masc.total_cost), sense=maximize
         )

#Solver
if res==0:
    solver = SolverFactory('gurobi')
elif res==1:
    solver = SolverFactory('ipopt')

### Epsilon bonds

In [ ]:
model.obj1.activate() 
model.obj2.deactivate()
model.obj3.deactivate()
model.eps_constraint_1.deactivate()
model.eps_constraint_2.deactivate()
results = solver.solve(model, tee=True) # solves and updates instance
Z1_1_worst=value(model.obj1)
Z2_1_worst=value(model.obj2)
Z3_1_worst=value(model.obj3)

In [ ]:
model.obj1.deactivate() 
model.obj2.activate() 
model.obj3.deactivate()
model.eps_constraint_1.deactivate()
model.eps_constraint_2.deactivate()
results = solver.solve(model, tee=True) # solves and updates instance
Z1_2_worst= value(model.obj1)
Z2_2_worst= value(model.obj2)
Z3_2_worst= value(model.obj3)
scaling_factors_circ = []
for j in model.set_s:
    scaling_factors_circ.append(model.s[j].value)

In [ ]:
model.obj1.deactivate() 
model.obj2.deactivate()
model.obj3.activate()
model.eps_constraint_1.deactivate()
model.eps_constraint_2.deactivate()
results = solver.solve(model, tee=True) # solves and updates instance
Z1_3_worst= value(model.obj1)
Z2_3_worst= value(model.obj2)
Z3_3_worst= value(model.obj3)

### Solving the Optimization Problem

In [ ]:
M1_worst = max(Z1_1_worst,Z1_2_worst,Z1_3_worst)
n1_worst = min(Z1_1_worst,Z1_2_worst,Z1_3_worst)

M2_worst = max(Z2_1_worst,Z2_2_worst,Z2_3_worst)
n2_worst = min(Z2_1_worst,Z2_2_worst,Z2_3_worst)

M3_worst = max(Z3_1_worst,Z3_2_worst,Z3_3_worst)
n3_worst = min(Z3_1_worst,Z3_2_worst,Z3_3_worst)

In [ ]:
M_penalty = 1e-4
delta_1 =  M2_worst - n2_worst
delta_2 = M3_worst - n3_worst
model.obj1_aug = Objective(expr = -sum(coef_GWP[i-1]*model.s[i] for i in model.set_s) + M_penalty*(model.s_eps1/delta_1 + model.s_eps2/delta_2), sense=maximize)

In [ ]:
#Z1, Z2, and Z3 values!!! for pareto surface generation

#Best
model.obj1.deactivate()
model.obj1_aug.activate()
model.obj2.deactivate() 
model.obj3.deactivate()
model.eps_constraint_1.activate()
model.eps_constraint_2.activate()


r=100
Z1=[]
Z2=[]
Z3=[]
solution_matrix = []

for t in range(0,r):
    for i in range (0,r):
        model.epsilon1 = n2_worst + (M2_worst - n2_worst) * t / (r - 1)
        model.epsilon2 = n3_worst + (M3_worst - n3_worst) * i / (r - 1)
        results = solver.solve(model) # solves and updates instance
        
        if results.solver.termination_condition == TerminationCondition.optimal:           
            Z1.append(-value(model.obj1))
            Z2.append(-value(model.obj2))
            Z3.append(-value(model.obj3))
            
            
            scaling_factors = []
            for j in model.set_s:
                scaling_factors.append(model.s[j].value)
            solution_matrix.append(scaling_factors) 

Z1_worst.append(-Z1_2_worst)
Z2_worst.append(-Z2_2_worst)
Z3_worst.append(-Z3_2_worst)

In [ ]:
solution_matrix.append(scaling_factors_circ)

### Generating the pareto frontier numbers

In [ ]:
# Assuming Z1, Z2, and Z3 are lists of values

# Create a dictionary to store unique combinations
unique_combinations = {}

# Iterate over the lists and store unique combinations in the dictionary
for z1, z2, z3 in zip(Z1_worst, Z2_worst, Z3_worst):
    combination = (z1, z2, z3)
    if combination not in unique_combinations:
        unique_combinations[combination] = 1  # Value doesn't matter, just using the key for uniqueness

# Extract unique values from the dictionary
unique_Z1, unique_Z2, unique_Z3 = zip(*unique_combinations.keys())

In [ ]:
# Assuming Z1, Z2, and Z3 are lists of values
# Create a dictionary to store unique combinations and their indices
unique_combinations = {}
indices = []
unique_solution_matrix = []

# Iterate over the lists and store unique combinations and their indices in the dictionary
for i, (z1, z2, z3) in enumerate(zip(Z1_worst, Z2_worst, Z3_worst)):
    combination = (z1, z2, z3)
    if combination not in unique_combinations:
        unique_combinations[combination] = [i]  # Store the index as a list
    else:
        unique_combinations[combination].append(i)  # Append the index to the existing list

# Extract unique values and their corresponding indices from the dictionary
indices = list(unique_combinations.values())

# New list with only the first elements of each sublist
unique_solution_indices = [sublist[0] for sublist in indices]

unique_solution_matrix = [solution_matrix[index] for index in unique_solution_indices]

In [ ]:
pareto_GWP = unique_Z1
pareto_circularity = [(1 - (value / (2 * f_design[0]))) for value in unique_Z2]
pareto_cost = unique_Z3

#pareto front points
pareto_points = [[pareto_GWP[i], pareto_circularity[i], pareto_cost[i]] for i in range(len(pareto_GWP))]

In [ ]:
#utopia point

utopia_point_GWP = min(pareto_GWP)
scaled_utopia_point_GWP = (utopia_point_GWP - min(pareto_GWP)) / (max(pareto_GWP) - min(pareto_GWP))

utopia_point_circularity = max(pareto_circularity)
scaled_utopia_point_circularity = (utopia_point_circularity - min(pareto_circularity)) / (max(pareto_circularity) - min(pareto_circularity))

utopia_point_cost = min(pareto_cost)
scaled_utopia_point_cost = (utopia_point_cost - min(pareto_cost)) / (max(pareto_cost) - min(pareto_cost))                                                     

In [ ]:
#Compromised Solution

scaled_pareto_GWP = [(element - min(pareto_GWP)) / (max(pareto_GWP) - min(pareto_GWP)) for element in pareto_GWP]
                                                                     

scaled_pareto_circularity = [(element - min(pareto_circularity)) / (max(pareto_circularity) - min(pareto_circularity)) for element in pareto_circularity]


scaled_pareto_cost = [(element - min(pareto_cost)) / (max(pareto_cost) - min(pareto_cost)) for element in pareto_cost]


# Calculate the distance to the utopia point
distances = []
for i in range(len(scaled_pareto_GWP)):
    distances.append(
        np.sqrt((scaled_pareto_GWP[i] - scaled_utopia_point_GWP) ** 2  
        + (scaled_pareto_circularity[i] - scaled_utopia_point_circularity) ** 2)
        + (scaled_pareto_cost[i] - scaled_utopia_point_cost) ** 2)
index_min_distance = np.argmin(distances)

In [ ]:
#Solutions' scaling factors
index_min_GWP_solution = np.argmin(pareto_GWP)
index_max_circularity_solution = np.argmax(pareto_circularity)
index_min_cost_solution = np.argmin(pareto_cost)
index_compromised_solution = index_min_distance

min_GWP_solution = unique_solution_matrix[index_min_GWP_solution]
max_circularity_solution = unique_solution_matrix[index_max_circularity_solution]
min_cost_solution = unique_solution_matrix[index_min_cost_solution]
compromised_solution = unique_solution_matrix[index_compromised_solution]

# Visualization

In [ ]:
# Data values for the indicators
indicators = ['Without Resilience', 'With Resilience']
values = [GWP_wo_res_single, GWP_with_res_single]  # Example values

# Create the bar plot
fig, ax = plt.subplots(figsize=(6, 4))
ax.bar(indicators, values, color=['blue', 'green'])

# Labels and title
ax.set_ylabel('GWP (kg CO2 eq.)')
ax.set_ylim(0, 2000)  # Set y-axis range

if savefigs:
    plt.savefig('single-obj-gwp.svg')
# Show the plot
plt.show()

In [ ]:
color_link = ['rgba(0, 0, 0, 0)','#A6E3D7','#A6E3D7','#A6E3D7','#A6E3D7','#A6E3D7','#A6E3D7','#A6E3D7','#A6E3D7','#A6E3D7', '#A6E3D7','#A6E3D7','#A6E3D7','#A6E3D7','#A6E3D7','#A6E3D7','#A6E3D7','#A6E3D7','#A6E3D7','#A6E3D7']
label = [ "Open-loop Recycling 2","Virgin PE", "Virgin PA6","Open-loop Recycling 1", "Manufacturer", "Use", "Landfill", "Incineration", "Pelletizing-a", "Pelletizing-b", "STRAP", "MASC","Pyrolysis"]

#0: Open-loop Recycling 2
#1: Virgin PE
#2: Virgin PA6
#3: Open-loop Recycling 1
#4: Manufacturer
#5: Use
#6: Landfill
#7: Incineration
#8: Pelletizing-a
#9: Pelletizing-b
#10: STRAP
#11: MASC
#12: Pyrolysis

source = [0,1,2,0,3,4,5,5,5,5,5,5,10,11,8,9,10,11,4, 5, 8, 9,12]
target = [4,4,4,3,5,5,6,7,8,9,10,11,4,5,6,6,6,6,6,12,3,3,0]

value = [0.01,
         abs(round(scaling_factors_circularity_wo_res_dict ['Polyethylene, linear low-density, LLDPE, virgin resin, at plant']))+0.01, #0>>2
         abs(round(scaling_factors_circularity_wo_res_dict ['POLYAMIDE 6 (Nylon 6)']))+0.01, 
         0.01,
         0.01,
         abs(round(scaling_factors_circularity_wo_res_dict ['multilayer film production']))+0.01,
         abs(round(scaling_factors_circularity_wo_res_dict ['landfill (film)']))+0.01,
         abs(round(scaling_factors_circularity_wo_res_dict ['incineration (film)'])),
         abs(round(scaling_factors_circularity_wo_res_dict ['downcycling (allocation)-s1 (heat pretreatment)']))+0.01,
         abs(round(scaling_factors_circularity_wo_res_dict ['downcycling (allocation)-s2 (1/3 discard)']))+0.01,
         abs(round(scaling_factors_circularity_wo_res_dict ['STRAP']))+0.01,
         abs(round(scaling_factors_circularity_wo_res_dict ['MASC']))+0.01,
         0.92*abs(round(scaling_factors_circularity_wo_res_dict ['STRAP']))+0.01,
         0.95*abs(round(scaling_factors_circularity_wo_res_dict ['MASC']))+0.01,
         0.05*abs(round(scaling_factors_circularity_wo_res_dict ['downcycling (allocation)-s1 (heat pretreatment)']))+0.01,
         0.3635*abs(round(scaling_factors_circularity_wo_res_dict ['downcycling (allocation)-s2 (1/3 discard)']))+0.01,
         0.08*abs(round(scaling_factors_circularity_wo_res_dict ['STRAP']))+0.01,
         0.05*abs(round(scaling_factors_circularity_wo_res_dict ['MASC']))+0.01,
         (0.1/0.9)*abs(round(scaling_factors_circularity_wo_res_dict ['multilayer film production']))+0.01,
         abs(round(scaling_factors_circularity_wo_res_dict ['pyrolysis']))+0.01,
         0.95*abs(round(scaling_factors_circularity_wo_res_dict ['downcycling (allocation)-s1 (heat pretreatment)']))+0.01,
         (1-0.3635)*abs(round(scaling_factors_circularity_wo_res_dict ['downcycling (allocation)-s2 (1/3 discard)']))+0.01,
         0.7*abs(round(scaling_factors_circularity_wo_res_dict ['pyrolysis']))+0.01
        ]
         
#data to dict, dict to sankey
link = dict(source = source, target = target, value = value, color = color_link)
node = dict(
    label = label,
    pad = 50,
    thickness = 15,
    x = [  0,  0,  0,0.2,0.2,0.4,  1,  1,0.8,0.8,0.6,0.6,0.8],  # Adjust x positions
    y = [0.2,0.4,0.6,0.8,0.5,0.5,0.6,0.4,0.6,0.8,0.6,0.4,0.4]   # Adjust y positions
)

data = go.Sankey(
    arrangement="snap",  # Ensures x and y positions are used
    link=link,
    node=node,
    valueformat=".2f",
    valuesuffix=" MT"
)
# graph the Sankey

fig = go.Figure(data)
fig.update_layout(width=1200, height=600)
fig.update_layout(
    hovermode = 'x',
    paper_bgcolor = '#FFFFFF', 
)
# Save as SVG
fig.write_image("sankey-wo-res-gwp.svg", format="svg")

fig.update_layout(font_size=18,font_color='black')
fig.show()

if savefigs:
    fig.write_image("sankey-wo-res-circularity.svg", format="svg")

In [ ]:
# Data values for the indicators
indicators = ['Without Resilience', 'With Resilience']
values = [circularity_wo_res_single, circularity_with_res_single]  # Example values

# Create the bar plot
fig, ax = plt.subplots(figsize=(6, 4))
ax.bar(indicators, values, color=['blue', 'green'])

# Labels and title
ax.set_ylabel('Circularity')
ax.set_ylim(0, 1)  # Set y-axis range

if savefigs:
    plt.savefig('single-obj-circularity.svg')
# Show the plot
plt.show()

In [ ]:
color_link = ['rgba(0, 0, 0, 0)','#A6E3D7','#A6E3D7','#A6E3D7','#A6E3D7','#A6E3D7','#A6E3D7','#A6E3D7','#A6E3D7','#A6E3D7', '#A6E3D7','#A6E3D7','#A6E3D7','#A6E3D7','#A6E3D7','#A6E3D7','#A6E3D7','#A6E3D7','#A6E3D7','#A6E3D7']
label = [ "Open-loop Recycling 2","Virgin PE", "Virgin PA6","Open-loop Recycling 1", "Manufacturer", "Use", "Landfill", "Incineration", "Pelletizing-a", "Pelletizing-b", "STRAP", "MASC","Pyrolysis"]

#0: Open-loop Recycling 2
#1: Virgin PE
#2: Virgin PA6
#3: Open-loop Recycling 2
#4: Manufacturer
#5: Use
#6: Landfill
#7: Incineration
#8: Pelletizing-a
#9: Pelletizing-b
#10: STRAP
#11: MASC
#12: Pyrolysis

source = [0,1,2,0,3,4,5,5,5,5,5,5,10,11,8,9,10,11,4, 5, 8, 9,12]
target = [4,4,4,3,5,5,6,7,8,9,10,11,4,5,6,6,6,6,6,12,3,3,0]

value = [0.01,
         abs(round(scaling_factors_circularity_with_res_dict ['Polyethylene, linear low-density, LLDPE, virgin resin, at plant']))+0.01, #0>>2
         abs(round(scaling_factors_circularity_with_res_dict ['POLYAMIDE 6 (Nylon 6)']))+0.01, 
         0.01,
         0.01,
         abs(round(scaling_factors_circularity_with_res_dict ['multilayer film production']))+0.01,
         abs(round(scaling_factors_circularity_with_res_dict ['landfill (film)']))+0.01,
         abs(round(scaling_factors_circularity_with_res_dict ['incineration (film)']))+0.01,
         abs(round(scaling_factors_circularity_with_res_dict ['downcycling (allocation)-s1 (heat pretreatment)']))+0.01,
         abs(round(scaling_factors_circularity_with_res_dict ['downcycling (allocation)-s2 (1/3 discard)']))+0.01,
         abs(round(scaling_factors_circularity_with_res_dict ['STRAP']))+0.01,
         abs(round(scaling_factors_circularity_with_res_dict ['MASC']))+0.01,
         0.92*abs(round(scaling_factors_circularity_with_res_dict ['STRAP']))+0.01,
         0.95*abs(round(scaling_factors_circularity_with_res_dict ['MASC']))+0.01,
         0.05*abs(round(scaling_factors_circularity_with_res_dict ['downcycling (allocation)-s1 (heat pretreatment)']))+0.01,
         0.3635*abs(round(scaling_factors_circularity_with_res_dict ['downcycling (allocation)-s2 (1/3 discard)']))+0.01,
         0.08*abs(round(scaling_factors_circularity_with_res_dict ['STRAP']))+0.01,
         0.05*abs(round(scaling_factors_circularity_with_res_dict ['MASC']))+0.01,
         (0.1/0.9)*abs(round(scaling_factors_circularity_with_res_dict ['multilayer film production']))+0.01,
         abs(round(scaling_factors_circularity_with_res_dict ['pyrolysis']))+0.01,
         0.95*abs(round(scaling_factors_circularity_with_res_dict ['downcycling (allocation)-s1 (heat pretreatment)']))+0.01,
         (1-0.3635)*abs(round(scaling_factors_circularity_with_res_dict ['downcycling (allocation)-s2 (1/3 discard)']))+0.01,
         0.7*abs(round(scaling_factors_circularity_with_res_dict ['pyrolysis']))+0.01
        ]
         
#data to dict, dict to sankey
link = dict(source = source, target = target, value = value, color = color_link)
node = dict(
    label = label,
    pad = 50,
    thickness = 15,
    x = [  0,  0,  0,0.2,0.2,0.4,  1,  1,0.8,0.8,0.6,0.6,0.8],  # Adjust x positions
    y = [0.2,0.4,0.6,0.8,0.5,0.5,0.6,0.4,0.6,0.8,0.6,0.4,0.4]   # Adjust y positions
)

data = go.Sankey(
    arrangement="snap",  # Ensures x and y positions are used
    link=link,
    node=node,
    valueformat=".2f",
    valuesuffix=" MT"
)
# graph the Sankey

fig = go.Figure(data)
fig.update_layout(width=1000, height=600)
fig.update_layout(
    hovermode = 'x',
    paper_bgcolor = '#FFFFFF', 
)
# Save as SVG
fig.write_image("sankey-wo-res-gwp.svg", format="svg")

fig.update_layout(font_size=16,font_color='black')
fig.show()


if savefigs:
    fig.write_image("sankey-with-res-circularity.svg", format="svg")

In [ ]:
# Data values for the indicators
indicators = ['Without Resilience', 'With Resilience']
values = [cost_wo_res_single, cost_with_res_single]  # Example values

# Create the bar plot
fig, ax = plt.subplots(figsize=(6, 4))
ax.bar(indicators, values, color=['blue', 'green'])

# Labels and title
ax.set_ylabel('Cost (USD)')
ax.set_ylim(0, 1000)  # Set y-axis range

if savefigs:
    plt.savefig('single-obj-cost.svg')
# Show the plot
plt.show()

In [ ]:
color_link = ['rgba(0, 0, 0, 0)','#A6E3D7','#A6E3D7','#A6E3D7','#A6E3D7','#A6E3D7','#A6E3D7','#A6E3D7','#A6E3D7','#A6E3D7', '#A6E3D7','#A6E3D7','#A6E3D7','#A6E3D7','#A6E3D7','#A6E3D7','#A6E3D7','#A6E3D7','#A6E3D7','#A6E3D7']
label = [ "Open-loop Recycling 2","Virgin PE", "Virgin PA6","Open-loop Recycling 1", "Manufacturer", "Use", "Landfill", "Incineration", "Pelletizing-a", "Pelletizing-b", "STRAP", "MASC","Pyrolysis"]

#0: Open-loop Recycling 2
#1: Virgin PE
#2: Virgin PA6
#3: Open-loop Recycling 1
#4: Manufacturer
#5: Use
#6: Landfill
#7: Incineration
#8: Pelletizing-a
#9: Pelletizing-b
#10: STRAP
#11: MASC
#12: Pyrolysis

source = [0,1,2,0,3,4,5,5,5,5,5,5,10,11,8,9,10,11,4, 5, 8, 9,12]
target = [4,4,4,3,5,5,6,7,8,9,10,11,4,5,6,6,6,6,6,12,3,3,0]

value = [0.01,
         abs(round(scaling_factors_cost_with_res_dict ['Polyethylene, linear low-density, LLDPE, virgin resin, at plant'])), #0>>2
         abs(round(scaling_factors_cost_with_res_dict ['POLYAMIDE 6 (Nylon 6)'])), 
         0.01,
         0.01,
         abs(round(scaling_factors_cost_with_res_dict ['multilayer film production'])),
         abs(round(scaling_factors_cost_with_res_dict ['landfill (film)'])),
         abs(round(scaling_factors_cost_with_res_dict ['incineration (film)'])),
         abs(round(scaling_factors_cost_with_res_dict ['downcycling (allocation)-s1 (heat pretreatment)'])),
         abs(round(scaling_factors_cost_with_res_dict ['downcycling (allocation)-s2 (1/3 discard)'])),
         abs(round(scaling_factors_cost_with_res_dict ['STRAP'])),
         abs(round(scaling_factors_cost_with_res_dict ['MASC'])),
         0.92*abs(round(scaling_factors_cost_with_res_dict ['STRAP'])),
         0.95*abs(round(scaling_factors_cost_with_res_dict ['MASC'])),
         0.05*abs(round(scaling_factors_cost_with_res_dict ['downcycling (allocation)-s1 (heat pretreatment)'])),
         0.3635*abs(round(scaling_factors_cost_with_res_dict ['downcycling (allocation)-s2 (1/3 discard)'])),
         0.08*abs(round(scaling_factors_cost_with_res_dict ['STRAP'])),
         0.05*abs(round(scaling_factors_cost_with_res_dict ['MASC'])),
         (0.1/0.9)*abs(round(scaling_factors_cost_with_res_dict ['multilayer film production'])),
         abs(round(scaling_factors_cost_with_res_dict ['pyrolysis'])),
         0.95*abs(round(scaling_factors_cost_with_res_dict ['downcycling (allocation)-s1 (heat pretreatment)'])),
         (1-0.3635)*abs(round(scaling_factors_cost_with_res_dict ['downcycling (allocation)-s2 (1/3 discard)'])),
         0.7*abs(round(scaling_factors_cost_with_res_dict ['pyrolysis']))
        ]
         
#data to dict, dict to sankey
link = dict(source = source, target = target, value = value, color = color_link)
node = dict(
    label = label,
    pad = 50,
    thickness = 15,
    x = [  0,  0,  0,0.2,0.2,0.4,  1,  1,0.8,0.8,0.6,0.6,0.8],  # Adjust x positions
    y = [0.2,0.4,0.6,0.8,0.5,0.5,0.6,0.4,0.6,0.8,0.6,0.4,0.4]   # Adjust y positions
)

data = go.Sankey(
    arrangement="snap",  # Ensures x and y positions are used
    link=link,
    node=node,
    valueformat=".2f",
    valuesuffix=" MT"
)
# graph the Sankey

fig = go.Figure(data)
fig.update_layout(width=1200, height=600)
fig.update_layout(
    hovermode = 'x',
    paper_bgcolor = '#FFFFFF', 
)
# Save as SVG
fig.write_image("sankey-wo-res-gwp.svg", format="svg")

fig.update_layout(font_size=18,font_color='black')
fig.show()

if savefigs:
    fig.write_image("sankey-with-res-cost.svg", format="svg")

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.ticker as ticker

# Sample data
data = {
    'GWP': pareto_GWP,
    'Circularity': pareto_circularity,
    'Cost': pareto_cost  # Third axis data
}

# Convert data to DataFrame
df = pd.DataFrame(data)

# Create figure and axes with larger figure size
fig, ax = plt.subplots(figsize=(10, 8))

# Increase font sizes globally
plt.rcParams.update({'font.size': 16})

# Define a logarithmic normalization for the color mapping
norm = mcolors.LogNorm(vmin=df['Cost'].min(), vmax=df['Cost'].max())

# Scale the marker sizes based on the cost values, applying a minimum size threshold
min_marker_size = 50  # Adjust this value as needed
marker_sizes = min_marker_size + 100 * (df['Cost'] - df['Cost'].min()) / (df['Cost'].max() - df['Cost'].min())

# Create scatter plot with logarithmic color mapping and marker sizes
scatter = ax.scatter(df['GWP'], df['Circularity'], c=df['Cost'], cmap='viridis', norm=norm, s=marker_sizes)

# Set x-axis and y-axis limits
ax.set_xlim(left=1000)
ax.set_ylim(0,1)

# Add labels and title with increased font size
ax.set_xlabel('GWP (kg CO2 eq)', fontsize=16)
ax.set_ylabel('Circularity', fontsize=16)
ax.set_title('Pareto Front without resilience', fontsize=16)

# Add legend explaining the sizes with increased font size
legend_labels = ['Low', 'Medium', 'High']
legend_sizes = [0.1, 0.2, 0.3]  # Adjust these sizes accordingly
legend_handles = [plt.scatter([], [], s=size*500, color='black', alpha=0.5) for size in legend_sizes]
ax.legend(legend_handles, legend_labels, title='Cost', loc='upper left', fontsize=16, title_fontsize=16)

# Create color bar with rounded tick values
cbar = fig.colorbar(scatter, ax=ax, orientation='vertical')
cbar.set_label('Cost (USD)', fontsize=16)
cbar.ax.yaxis.set_label_position('left')  # Place the label on the left side

# Generate rounded tick values for the color bar
cost_min = df['Cost'].min()
cost_max = df['Cost'].max()
cbar_ticks = np.geomspace(cost_min, cost_max, num=5)  # Log-spaced values
cbar_ticks = np.round(cbar_ticks, -int(np.floor(np.log10(cbar_ticks[0]))))  # Round to nearest 10s, 100s, etc.
cbar.set_ticks(cbar_ticks)
cbar.ax.set_yticklabels([f'{int(tick):,}' for tick in cbar_ticks])  # Format as integer with commas

# Adjust the layout to prevent clipping
plt.tight_layout()

# Save the plot as a PDF file
plt.savefig('pareto_front_closest_wo_res.svg', format='svg')

# Show plot
plt.show()

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.ticker as ticker

# Sample data
data = {
    'GWP': pareto_GWP_with_res,
    'Circularity': pareto_circularity_with_res,
    'Cost': pareto_cost_with_res  # Third axis data
}

# Convert data to DataFrame
df = pd.DataFrame(data)

# Create figure and axes with larger figure size
fig, ax = plt.subplots(figsize=(10, 8))

# Increase font sizes globally
plt.rcParams.update({'font.size': 16})

# Determine the maximum cost and round up to the nearest 1000
cost_max = np.ceil(df['Cost'].max() / 1000) * 1000  # Ensure max color bar value is a multiple of 1000

# Define a linear normalization for the color mapping
norm = mcolors.Normalize(vmin=0, vmax=cost_max)

# Scale the marker sizes based on the cost values
min_marker_size = 50
marker_sizes = min_marker_size + 100 * (df['Cost'] - df['Cost'].min()) / (df['Cost'].max() - df['Cost'].min())

# Create scatter plot with linear color mapping and marker sizes
scatter = ax.scatter(df['GWP'], df['Circularity'], c=df['Cost'], cmap='viridis', norm=norm, s=marker_sizes)

# Set x-axis and y-axis limits
ax.set_xlim(left=1000)
ax.set_ylim(0, 1)

# Add labels and title
ax.set_xlabel('GWP (kg CO2 eq)', fontsize=16)
ax.set_ylabel('Circularity (%)', fontsize=16)
ax.set_title('Pareto Front with resilience', fontsize=16)

# Add legend for marker sizes
legend_labels = ['Low', 'Medium', 'High']
legend_sizes = [0.1, 0.2, 0.3]  
legend_handles = [plt.scatter([], [], s=size*500, color='black', alpha=0.5) for size in legend_sizes]
ax.legend(legend_handles, legend_labels, title='Cost', loc='upper left', fontsize=16, title_fontsize=16)

# Create color bar with linear scale and tick marks every 1000
cbar = fig.colorbar(scatter, ax=ax, orientation='vertical')
cbar.set_label('Cost ($)', fontsize=16)
cbar.ax.yaxis.set_label_position('left')

# Set tick values every 1000
tick_spacing = 1000
cbar_ticks = np.arange(0, cost_max + tick_spacing, tick_spacing)
cbar.set_ticks(cbar_ticks)
cbar.ax.set_yticklabels([f'{int(tick):,}' for tick in cbar_ticks])  

# Adjust layout
plt.tight_layout()

# Save and show the plot
plt.savefig('pareto_front_closest_with_res.svg', format='svg')
plt.show()

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Sample data
data_wo_res = {
    'GWP': pareto_GWP,
    'Circularity': pareto_circularity,
    'Cost': pareto_cost
}

data_with_res = {
    'GWP': pareto_GWP_with_res,
    'Circularity': pareto_circularity_with_res,
    'Cost': pareto_cost_with_res
}

# Convert data to DataFrame
df_wo_res = pd.DataFrame(data_wo_res)
df_with_res = pd.DataFrame(data_with_res)

# Create figure
fig, ax = plt.subplots(figsize=(10, 8))

# Increase font sizes globally
plt.rcParams.update({'font.size': 16})

# Marker size scaling
min_marker_size = 50
max_marker_size = 300  # Adjust max size if needed
cost_min = min(df_wo_res['Cost'].min(), df_with_res['Cost'].min())
cost_max = max(df_wo_res['Cost'].max(), df_with_res['Cost'].max())

marker_sizes_wo_res = min_marker_size + (max_marker_size - min_marker_size) * (
    (df_wo_res['Cost'] - cost_min) / (cost_max - cost_min)
)

marker_sizes_with_res = min_marker_size + (max_marker_size - min_marker_size) * (
    (df_with_res['Cost'] - cost_min) / (cost_max - cost_min)
)

# Scatter plots
ax.scatter(df_wo_res['GWP'], df_wo_res['Circularity'], 
           s=marker_sizes_wo_res, color='blue', alpha=0.7, 
           edgecolors='black', label="Without Resilience")

ax.scatter(df_with_res['GWP'], df_with_res['Circularity'], 
           s=marker_sizes_with_res, color='red', alpha=0.7, 
           edgecolors='black', label="With Resilience")

# Set x-axis and y-axis limits
ax.set_xlim(left=1000)
ax.set_ylim(0, 1)

# Labels and title
ax.set_xlabel('GWP (kg CO2 eq)', fontsize=16)
ax.set_ylabel('Circularity', fontsize=16)
ax.set_title('Pareto Front Comparison: With vs. Without Resilience', fontsize=16)

# Legend for cost representation
legend_sizes = [cost_min, (cost_min + cost_max) / 2, cost_max]  # Example values
legend_markers = [min_marker_size, (min_marker_size + max_marker_size) / 2, max_marker_size]
legend_handles = [plt.scatter([], [], s=size, color='black', alpha=0.5) for size in legend_markers]

ax.legend(legend_handles, [f'${int(value):,}' for value in legend_sizes], 
          title='Cost (USD)', loc='upper left', fontsize=14, title_fontsize=16)

# Layout adjustments
plt.tight_layout()

# Save the plot
plt.savefig('pareto_front_comparison_size_only.svg', format='svg')

# Show plot
plt.show()


In [ ]:
# Create 3D figure
fig = plt.figure(figsize=(12, 9))
ax = fig.add_subplot(111, projection='3d')

# Increase font sizes globally
plt.rcParams.update({'font.size': 14})

# Function to plot triangular surface
def plot_triangular_surface(df, color, alpha=0.6):
    points = np.column_stack((df['GWP'], df['Circularity']))
    tri = Delaunay(points)  # Create a triangular mesh
    ax.plot_trisurf(df['GWP'], df['Circularity'], df['Cost'], triangles=tri.simplices, color=color, alpha=alpha)

# Plot triangular surfaces
plot_triangular_surface(df_wo_res, color='blue', alpha=0.6)  
plot_triangular_surface(df_with_res, color='red', alpha=0.6)  

# Scatter optimal points
ax.scatter(min_cost_wo_res['GWP'], min_cost_wo_res['Circularity'], min_cost_wo_res['Cost'], color='black', s=100, label="Min Cost (No Resilience)", marker='o')
ax.scatter(max_circularity_wo_res['GWP'], max_circularity_wo_res['Circularity'], max_circularity_wo_res['Cost'], color='black', s=100, label="Max Circularity (No Resilience)", marker='^')
ax.scatter(min_gwp_wo_res['GWP'], min_gwp_wo_res['Circularity'], min_gwp_wo_res['Cost'], color='black', s=100, label="Min GWP (No Resilience)", marker='s')

ax.scatter(min_cost_with_res['GWP'], min_cost_with_res['Circularity'], min_cost_with_res['Cost'], color='yellow', s=100, label="Min Cost (With Resilience)", marker='o')
ax.scatter(max_circularity_with_res['GWP'], max_circularity_with_res['Circularity'], max_circularity_with_res['Cost'], color='yellow', s=100, label="Max Circularity (With Resilience)", marker='^')
ax.scatter(min_gwp_with_res['GWP'], min_gwp_with_res['Circularity'], min_gwp_with_res['Cost'], color='yellow', s=100, label="Min GWP (With Resilience)", marker='s')

# Labels and title
ax.set_xlabel('GWP (kg CO2 eq)', fontsize=14)
ax.set_ylabel('Circularity', fontsize=14)
ax.set_zlabel('Cost (USD)', fontsize=14)
ax.set_title('3D Pareto Front with Optimal Solutions Highlighted', fontsize=16)

# Adjust view angle
ax.view_init(elev=10, azim=115)  

# Show legend
ax.legend(fontsize=12)

# Save the plot
plt.savefig('pareto_front_3d_triangular_surface.svg', format='svg')

# Show plot
plt.show()

In [ ]:
import matplotlib.pyplot as plt

# Flatten all values from both dictionaries into separate lists
all_values_wo_res = [value for d in hist1_wo_res for value in d.values()]
all_values_with_res = [value for d in hist1_with_res for value in d.values()]

# Create figure and axis
fig, ax = plt.subplots(figsize=(8, 6))

# Create a combined box plot
ax.boxplot([all_values_wo_res, all_values_with_res], vert=True, patch_artist=True,
           boxprops=dict(facecolor='lightblue'))

# Set labels and title
ax.set_ylabel('GWP (kg CO2 eq)', fontsize=14)
ax.set_xticklabels(['Without Resilience', 'With Resilience'], fontsize=12)

# Save and show the plot
plt.savefig('histogram-gwp.svg', format='svg')

# Adjust layout and show plot
plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt

# Flatten all values from both dictionaries into separate lists and apply the transformation
all_values_wo_res = [(1 - (value / 2000)) for d in hist2_wo_res for value in d.values()]
all_values_with_res = [(1 - (value / 2000)) for d in hist2_with_res for value in d.values()]

# Create figure and axis
fig, ax = plt.subplots(figsize=(8, 6))

# Create a combined box plot
ax.boxplot([all_values_wo_res, all_values_with_res], vert=True, patch_artist=True,
           boxprops=dict(facecolor='lightblue'))

# Set labels and title
ax.set_ylabel('Circularity', fontsize=14)
ax.set_xticklabels(['Without Resilience', 'With Resilience'], fontsize=12)

# Save and show the plot
plt.savefig('histogram-circularity.svg', format='svg')

# Adjust layout and show plot
plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt

# Flatten all values from both dictionaries into separate lists
all_values_wo_res = [value for d in hist3_wo_res for value in d.values()]
all_values_with_res = [value for d in hist3_with_res for value in d.values()]

# Create figure and axis
fig, ax = plt.subplots(figsize=(8, 6))

# Create a combined box plot
ax.boxplot([all_values_wo_res, all_values_with_res], vert=True, patch_artist=True,
           boxprops=dict(facecolor='lightblue'))

# Set labels and title
ax.set_ylabel('Cost (USD)', fontsize=14)
ax.set_xticklabels(['Without Resilience', 'With Resilience'], fontsize=12)

# Save and show the plot
plt.savefig('histogram-cost.svg', format='svg')

# Adjust layout and show plot
plt.tight_layout()
plt.show()